## 침묵 구간으로 문장 자르기

In [ ]:
!git init 
!git pull https://github.com/carpedm20/multi-Speaker-tacotron-tensorflow

Reinitialized existing Git repository in /content/.git/
From https://github.com/carpedm20/multi-Speaker-tacotron-tensorflow
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [ ]:
!pip3 install -r requirements.txt
!python3 -c "import nltk; nltk.download('punkt')"

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15.4 # 1.15 버전 Tensorflow 설치

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 110.5MB 49kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# audio/silence.py

import os
import re
import sys
import json
import librosa
import argparse
import numpy as np
from tqdm import tqdm
from glob import glob
from pydub import silence
from pydub import AudioSegment
from functools import partial

from hparams import hparams
from utils import parallel_run, add_postfix
from audio import load_audio, save_audio, get_duration, get_silence

def abs_mean(x):
    return abs(x).mean()

def remove_breath(audio):
    edges = librosa.effects.split(
            audio, top_db=40, frame_length=128, hop_length=32)

    for idx in range(len(edges)):
        start_idx, end_idx = edges[idx][0], edges[idx][1]
        if start_idx < len(audio):
            if abs_mean(audio[start_idx:end_idx]) < abs_mean(audio) - 0.05:
                audio[start_idx:end_idx] = 0

    return audio

def split_on_silence_with_librosa(
        audio_path, top_db=40, frame_length=1024, hop_length=256,
        skip_idx=0, out_ext="wav",
        min_segment_length=3, max_segment_length=8,
        pre_silence_length=0, post_silence_length=0):

    filename = os.path.basename(audio_path).split('.', 1)[0]
    in_ext = audio_path.rsplit(".")[1]

    audio = load_audio(audio_path)

    edges = librosa.effects.split(audio,
            top_db=top_db, frame_length=frame_length, hop_length=hop_length)

    new_audio = np.zeros_like(audio)
    for idx, (start, end) in enumerate(edges[skip_idx:]):
        new_audio[start:end] = remove_breath(audio[start:end])
        
    save_audio(new_audio, add_postfix(audio_path, "no_breath"))
    audio = new_audio
    edges = librosa.effects.split(audio,
            top_db=top_db, frame_length=frame_length, hop_length=hop_length)

    audio_paths = []
    for idx, (start, end) in enumerate(edges[skip_idx:]):
        segment = audio[start:end]
        duration = get_duration(segment)

        if duration <= min_segment_length or duration >= max_segment_length:
            continue

        output_path = "{}/{}.{:04d}.{}".format(
                os.path.dirname(audio_path), filename, idx, out_ext)

        padded_segment = np.concatenate([
                get_silence(pre_silence_length),
                segment,
                get_silence(post_silence_length),
        ])


        
        save_audio(padded_segment, output_path)
        audio_paths.append(output_path)

    return audio_paths

def read_audio(audio_path):
    return AudioSegment.from_file(audio_path)

def split_on_silence_with_pydub(
        audio_path, skip_idx=0, out_ext="wav",
        silence_thresh=-40, min_silence_len=400,
        silence_chunk_len=100, keep_silence=100):

    filename = os.path.basename(audio_path).split('.', 1)[0]
    in_ext = audio_path.rsplit(".")[1]

    audio = read_audio(audio_path)
    not_silence_ranges = silence.detect_nonsilent(
        audio, min_silence_len=silence_chunk_len,
        silence_thresh=silence_thresh)

    edges = [not_silence_ranges[0]]

    for idx in range(1, len(not_silence_ranges)-1):
        cur_start = not_silence_ranges[idx][0]
        prev_end = edges[-1][1]

        if cur_start - prev_end < min_silence_len:
            edges[-1][1] = not_silence_ranges[idx][1]
        else:
            edges.append(not_silence_ranges[idx])
    
    audio_paths = []
    for idx, (start_idx, end_idx) in enumerate(edges[skip_idx:]):
        start_idx = max(0, start_idx - keep_silence)
        end_idx += keep_silence

        target_audio_path = "{}/{}.{:04d}.{}".format(
                os.path.dirname(audio_path), filename, idx, out_ext)

        segment=audio[start_idx:end_idx]
        
        segment.export(target_audio_path, out_ext)  # for soundsegment

        audio_paths.append(target_audio_path)

    return audio_paths

def split_on_silence_batch(audio_paths, method, **kargv):
    audio_paths.sort()
    method = method.lower()

    if method == "librosa":
        fn = partial(split_on_silence_with_librosa, **kargv)
    elif method == "pydub":
        fn = partial(split_on_silence_with_pydub, **kargv)

    parallel_run(fn, audio_paths,
            desc="Split on silence", parallel=False)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--audio_pattern', required=True)
    parser.add_argument('--out_ext', default='wav')
    parser.add_argument('--method', choices=['librosa', 'pydub'], required=True)
    config = parser.parse_args()

    audio_paths = glob(config.audio_pattern)

    split_on_silence_batch(
            audio_paths, config.method,
            out_ext=config.out_ext,
    )

In [ ]:
# silence 모듈 실행
!python3 -m audio.silence --audio_pattern "/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/*.wav" --method=pydub

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Split on silence: 100% 13/13 [01:14<00:00,  5.72s/it]


In [ ]:
!pip install pydub

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

--2020-10-19 03:53:15--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?ZICD9xT5mRW4kzEBec1_txQ1AST4fl4pSkiu-FkhOwdzPLeo-zhKsRSNU0QgByRzCvTh0tvFDJXQLg4ffEDuTorVT7yuHtPIglxd3H0JEqpm8lUZJzuDv-iDgGkxNbsKlO_y5FlBbKfZsplh89yIMMt9elaYoww4MTAp7HryZPMJyRjqaaVoYVf7ZV2lwRl8DR1T9zXTua_wf7_1LdEW [following]
--2020-10-19 03:53:16--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?ZICD9xT5mRW4kzEBec1_txQ1AST4fl4pSkiu-FkhOwdzPLeo-zhKsRSNU0QgByRzCvTh0tvFDJXQLg4ffEDuTorVT7yuHtPIglxd3H0JEqpm8lUZJzuD

In [ ]:
import requests
import json
from tqdm.notebook import tqdm
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

rest_api_key = 'dce0d8d49cffcdea376d76f9a8e80d60'

headers = {
    "Content-Type": "application/octet-stream",
    "X-DSS-Service": "DICTATION",
    "Authorization": "KakaoAK " + rest_api_key,
}

import os

path = '/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/'
file_list = os.listdir(path)
file_list_wav = [file for file in file_list if file.endswith(".wav")]
recognition_all = {}

for i in tqdm(file_list_wav):
  with open('/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/{}'.format(i), 'rb') as fp:
      audio = fp.read()
  res = requests.post(kakao_speech_url, headers=headers, data=audio)
  try: 
    result_json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
    result = json.loads(result_json_string)
    text = result['value']
    recognition_all['./content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/{}'.format(i)] = text
  except:
    print('에러발생 :', '/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/{}'.format(i))

  with open('/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/recognition_all.json','w',encoding='utf-8') as f:
      json.dump(recognition_all, f, ensure_ascii=False)

에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0002.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0004.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0006.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0007.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0008.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0010.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0025.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0026.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0027.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0033.wav
에러발생 : /content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적

In [ ]:
!python3 -m recognition.google --audio_pattern "/content/gdrive/My Drive/stt플젝데이터/*.*.wav"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

text_recognition_batch: 0it [00:00, ?it/s]
 [*] /content/gdrive/My Drive/recognition.json has backup: /content/gdrive/My Drive/recognition.backup_2020-10-18_19-56-10.json


In [ ]:
!pip install SpeechRecognition
!pip install pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-1rak4e1m/pyaudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-1rak4e1m/pyaudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-fqkp56o_/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [ ]:
import speech_recognition as sr

recognition_all = {}
path = '/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/'
file_list = os.listdir(path)
file_list_wav = [file for file in file_list if file.endswith(".wav")]
for i in tqdm(file_list_wav):
  AUDIO_FILE = f"/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/{i}"

  # audio file을 audio source로 사용합니다
  r = sr.Recognizer()
  with sr.AudioFile(AUDIO_FILE) as source:
      audio = r.record(source)  # 전체 audio file 읽기

  # 구글 웹 음성 API로 인식하기
  try:
      print(f"/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/{i} : " + r.recognize_google(audio, language='ko'))
  except sr.UnknownValueError:
      pass
  except sr.RequestError as e:
      pass
  
  with open('/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/유인나_1904_1605.json','w',encoding='utf-8') as f:
      json.dump(recognition_all, f, ensure_ascii=False)

/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0000.wav : KBS 쿨 FM 유인나의 볼륨을 높여요 4월 11일 일요일 초코 그러는 뜨거운 감자의 고백 들으셨습니다
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0001.wav : 많다 말장난 고백 은근 많네요 참 진짜 머리도 좋아요
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0003.wav : 난 라면이야 너와 함께라면
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0005.wav : 난 푸딩이야 오늘도 내가 보고 싶은 띵
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0009.wav : 미안합니다
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0010.wav : 아우 닭살 현실감이 조금 떨어지긴 어때요 기분은 괜찮죠
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0011.wav : 이런 말 언제 들어 보겠어요
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_01.0013.wav : 이제 밥 살 좀 털어내고 광고 듣고 와서 볼륨 가족들 사연 만나 보자 잠깐만
/content/gdrive/My Drive/stt플젝/stt플젝데이터/converter적용/160501_0

## 유인나 Dataset json화

In [ ]:
import os
from tqdm import tqdm
import json

In [ ]:
inna_json = {"0402_2_vocals.0013.wav": "음", "0402_2_vocals.0031.wav": "아", "0402_2_vocals.0064.wav": "뭐야", "0402_2_vocals.0066.wav": "으아", "0402_2_vocals.0071.wav": "에", "0402_2_vocals.0078.wav": "야", "0402_2_vocals.0081.wav": "음", "0402_2_vocals.0083.wav": "에", "0402_2_vocals.0089.wav": "봄비", "0402_3_vocals.0032.wav": "음", "0402_3_vocals.0048.wav": "뚜아", "0402_3_vocals.0057.wav": "대신", "0402_3_vocals.0070.wav": "음", "0402_3_vocals.0078.wav": "음", "0402_3_vocals.0082.wav": "아이엘", "0402_3_vocals.0090.wav": "아 음어 야", "0402_3_vocals.0114.wav": "흠", "0402_3_vocals.0115.wav": "do 5", "0402_3_vocals.0128.wav": "흠", "0402_3_vocals.0137.wav": "그런데", "0402_3_vocals.0145.wav": "보우", "0402_3_vocals.0146.wav": "퀵", "0402_4_vocals.0002.wav": "음", "0402_4_vocals.0015.wav": "이 말 좋은 거 같아요 많은 풍경이 떠오르기도 하구", "0402_4_vocals.0032.wav": "뚝", "0402_4_vocals.0082.wav": "응", "0402_4_vocals.0089.wav": "음", "0402_5_vocals.0025.wav": "네", "0402_5_vocals.0039.wav": "음", "0402_5_vocals.0050.wav": "5 두부", "0402_5_vocals.0073.wav": "차야", "0402_5_vocals.0077.wav": "응", "0402_5_vocals.0085.wav": "왜", "0402_5_vocals.0103.wav": "야 잘 해봐요", "0402_5_vocals.0108.wav": "뭐", "0409_1_vocals.0023.wav": "음", "0409_1_vocals.0038.wav": "아 에에 에", "0409_1_vocals.0040.wav": "네에", "0409_1_vocals.0041.wav": "서은 홍", "0409_1_vocals.0042.wav": "20 오 오", "0409_1_vocals.0043.wav": "응응 와아 하시오", "0409_1_vocals.0044.wav": "우우와", "0409_1_vocals.0063.wav": "술", "0409_1_vocals.0074.wav": "이태원 술탄 씨", "0409_1_vocals.0076.wav": "아", "0409_1_vocals.0113.wav": "응 테인 hectic 그 들", "0409_1_vocals.0122.wav": "아이유", "0409_1_vocals.0133.wav": "응", "0409_1_vocals.0136.wav": "응 응", "0409_1_vocals.0137.wav": "아", "0409_1_vocals.0161.wav": "흠", "0409_2_vocals.0007.wav": "음", "0409_2_vocals.0018.wav": "음", "0409_2_vocals.0041.wav": "음 아 음", "0409_2_vocals.0047.wav": "음", "0409_3_vocals.0137.wav": "아이유에에", "0409_4_vocals.0011.wav": "아이유를", "0409_4_vocals.0079.wav": "흠", "0409_4_vocals.0107.wav": "응 위", "0409_5_vocals.0027.wav": "조용히 해", "0409_5_vocals.0051.wav": "응응 재생해 주세요아 흑 꼭 다음은 누구야", "0409_5_vocals.0055.wav": "음 예", "0409_5_vocals.0063.wav": "후우 링", "0409_5_vocals.0064.wav": "예에 에에에 호에에에", "0409_5_vocals.0065.wav": "음 와", "0409_5_vocals.0066.wav": "응 와아 네에", "0409_5_vocals.0067.wav": "음 와", "0501_1.0009.wav": "응", "0501_1.0023.wav": "어", "0501_1.0031.wav": "응", "0501_1.0052.wav": "음", "0501_1.0063.wav": "응", "0501_1.0070.wav": "휴", "0501_1.0082.wav": "아잇", "0501_1.0084.wav": "응", "0501_1.0087.wav": "으아", "0501_1.0093.wav": "응", "0501_2.0029.wav": "응", "0501_2.0033.wav": "군 하핫", "0501_2.0079.wav": "음해", "0501_4.0004.wav": "응", "0501_4.0006.wav": "응해", "0501_4.0009.wav": "응", "0501_4.0010.wav": "으아아", "0501_4.0018.wav": "왜", "0501_4.0030.wav": "음", "0501_4.0038.wav": "응 응", "0501_4.0041.wav": "응", "0501_4.0068.wav": "응", "0501_4.0078.wav": "음", "0501_4.0084.wav": "응", "0501_4.0089.wav": "90", "0501_4.0092.wav": "에에 a", "0501_4.0095.wav": "중흥", "0501_4.0108.wav": "음", "0501_4.0120.wav": "응 했다 그", "0501_4.0128.wav": "음", "0502_1_vocals.0000.wav": "어우", "0502_1_vocals.0014.wav": "쇼메", "0502_1_vocals.0015.wav": "그 행복한 사람 있나요", "0502_1_vocals.0019.wav": "어우", "0502_1_vocals.0028.wav": "꿀 ooo", "0502_1_vocals.0034.wav": "음 퓨", "0502_1_vocals.0061.wav": "유유 오우", "0502_1_vocals.0063.wav": "6616 님 4일간의 시간여행 끝까지 함께 할게요", "0502_2_vocals.0012.wav": "야", "0502_2_vocals.0014.wav": "응", "0502_2_vocals.0015.wav": "아", "0502_2_vocals.0018.wav": "오", "0502_2_vocals.0021.wav": "ooouuu가 있대", "0502_2_vocals.0027.wav": "음", "0502_2_vocals.0030.wav": "5522", "0502_2_vocals.0036.wav": "숙대 hook야", "0502_2_vocals.0037.wav": "종란", "0502_2_vocals.0044.wav": "탓으로 저렇게 잘 컸는지", "0502_2_vocals.0046.wav": "8912 님이", "0502_2_vocals.0050.wav": "와", "0502_2_vocals.0065.wav": "신혁 씨 내일까지 미역국 한 번 만들어 보고", "0502_2_vocals.0070.wav": "하하하", "0502_2_vocals.0083.wav": "응 가", "0502_2_vocals.0103.wav": "fm", "0502_3_vocals.0009.wav": "이걸", "0502_3_vocals.0030.wav": "아", "0502_3_vocals.0038.wav": "송영진 씨가 유디 목소리로 eu가 짱 목소리 톤이야", "0502_3_vocals.0047.wav": "야", "0502_3_vocals.0053.wav": "음", "0502_4_vocals.0002.wav": "그래도", "0502_4_vocals.0007.wav": "아", "0502_4_vocals.0013.wav": "음", "0502_4_vocals.0020.wav": "어", "0502_4_vocals.0021.wav": "응", "0502_4_vocals.0048.wav": "어원", "0502_4_vocals.0061.wav": "어어", "0502_4_vocals.0065.wav": "아파트 시세", "0502_4_vocals.0067.wav": "와", "0502_4_vocals.0073.wav": "좀 만나볼게요 우리 노래를 하는 정도는 빼도 될 거 같아요 얘기를 나누고 싶으니까요", "0502_4_vocals.0074.wav": "어", "0502_4_vocals.0078.wav": "그래", "0502_4_vocals.0079.wav": "네", "0502_4_vocals.0081.wav": "오", "0502_4_vocals.0082.wav": "와", "0502_4_vocals.0104.wav": "아침 밥", "0502_4_vocals.0109.wav": "자꾸와", "0502_4_vocals.0121.wav": "어", "0502_4_vocals.0162.wav": "음", "0502_4_vocals.0163.wav": "그래 이거", "0502_4_vocals.0167.wav": "하", "0502_5_vocals.0001.wav": "앞으로 1년을 또 공부 만 하면 된다네", "0502_5_vocals.0023.wav": "이 히", "0402_1_vocals.0001.wav": "돈 벌러 떠난 오빠를 기다리고 기다리던 꽃이 됐다는 도라지", "0402_1_vocals.0003.wav": "지금 찬란 하게 핀 꽃도 언젠가 질 거라는 걸 알아서 일까요", "0402_1_vocals.0004.wav": "아님 그 아름다움에 질투라도 나는 걸까", "0402_1_vocals.0008.wav": "은혜도 보고 싶었어요", "0402_1_vocals.0009.wav": "볼륨을 높여요 저는 유인나입니다", "0402_1_vocals.0011.wav": "KBS 쿨 FM 유인나의 볼륨을 높여요 4월 2일 토요일 자꾸 그러는 존박의 베이비 드렸습니다", "0402_1_vocals.0013.wav": "꽃 피는 봄이입니다 꽃에는 꽃말 전설 이런게 있죠", "0402_1_vocals.0014.wav": "근데 찾아 보거나 들어 보면 어쩜 그렇게 슬픈 게 많아요", "0402_1_vocals.0015.wav": "누구를 기다리거나 사랑이 이루어지지 않았거나 못 만나거나 그러다 죽고", "0402_1_vocals.0016.wav": "그러면 그때 거기서 꽃이 피고 대부분이 그렇더라고요", "0402_1_vocals.0017.wav": "아 물론 그냥 하는 말이겠죠 누군가 지원했을 거예요 그렇죠 근데 왜 그렇게 아름다운 꽃을 보면서 이렇게 슬픈 이야기들을 지원 했을까", "0402_1_vocals.0018.wav": "싶어요", "0402_1_vocals.0020.wav": "꽃은", "0402_1_vocals.0021.wav": "언젠가는 지니까 그 좀 슬퍼서 그랬나", "0402_1_vocals.0022.wav": "하긴 꽃을든 거 보면 참", "0402_1_vocals.0023.wav": "아 정말 아쉽긴 해요", "0402_1_vocals.0024.wav": "자 그러니까 피어 있을 때 실컷 즐깁시다 우리 예쁜 거 보면서 예쁜 생각 좋은 말들 많이 많이 나눠요 광고 듣고 와서 부른 가족들 사연 만나 볼게요 잠깐만", "0402_2_vocals.0000.wav": "검색어 하나를 정해서 부른 가족들의 사연을 꼼꼼하게 만나는 시간입니다 볼륨 검색", "0402_2_vocals.0001.wav": "오늘의 검색어는 조금으로 전에 봤어요 어떤 이야기를 보내셨는지 만나 보겠습니다", "0402_2_vocals.0002.wav": "3005 님 항상 유지 방송 들으면서 하루에 쌓인 스트레스를 조금이나마 풀고 있어요", "0402_2_vocals.0003.wav": "근데 이렇게 보내면 문자가 도착하나요 궁금합니다", "0402_2_vocals.0004.wav": "아우 그럼요", "0402_2_vocals.0005.wav": "금방 옵니다", "0402_2_vocals.0006.wav": "아주 멀리 있어도 2초면", "0402_2_vocals.0008.wav": "많이 보내주세요", "0402_2_vocals.0009.wav": "911 님 오늘 자취방 조금 남아 있던 짐을 다 빼고 왔어요 대학생을 함께한 자치권과 정말로 작별이란이 시원 섭섭하네요", "0402_2_vocals.0010.wav": "와 이거 진짜 섭섭하게 따", "0402_2_vocals.0011.wav": "대학생활 얼마나 재밌을까요", "0402_2_vocals.0012.wav": "자취방에서 친구들이랑 모여서 토요일 같이 마시는 날 디저트 되고 밤새 시험 공부하거나 뭐 과제 하느라고 막 진짜 힘든 날도 있었을 거고", "0402_2_vocals.0014.wav": "자 이거는 다시 보고 싶어도 볼 수 없는 곳이잖아요 사진 같은 거 남겨 주세요", "0402_2_vocals.0015.wav": "다 정리 하기 전에", "0402_2_vocals.0016.wav": "응 또 663 형님 아빠가 늦은 저녁으로 삼겹살을 드시는데요", "0402_2_vocals.0017.wav": "엄마가 저도 먹을 거냐고 물어 보더라고요 먹으면 몸에 죄짓는 기분이 될 거 같아서 그냥 조금만 구라고 말씀드렸는데요 205기 한점에 들어가는 순간", "0402_2_vocals.0018.wav": "저도 모르게 폭풍 젓가락질 해버리고 말았어요", "0402_2_vocals.0019.wav": "이상하게 안 먹는다고 하고 먹을 때가 더 맛있는 거 같아요", "0402_2_vocals.0021.wav": "어 맞아 라면도 한입 얻어 먹을 때가 진짜 맛있잖아요", "0402_2_vocals.0022.wav": "삼겹살 어떻게 그렇게 먹을 수 있어요 근데 조금 많은 불가능해요 삼겹살은 한두 점은 아예 안 먹거나", "0402_2_vocals.0023.wav": "배터지게 먹고 나 둘 중에 하나", "0402_2_vocals.0024.wav": "삼겹살은 무슨 일 있어도 나는 못 끊을 거 같아요", "0402_2_vocals.0027.wav": "박종 1시 가요", "0402_2_vocals.0028.wav": "요즘 봄 냄새가 많이 나네요", "0402_2_vocals.0029.wav": "기온이 올라가니 초록 잎들이 조금씩 보여요", "0402_2_vocals.0030.wav": "이 친구들 덕분에 공개도 더욱 상쾌해지는 거 같아요", "0402_2_vocals.0032.wav": "이초록 잎들이 뿜어내는이 공기는 저는 집에서 화분 키운다고 했잖아요", "0402_2_vocals.0033.wav": "베란다 문을 베란다 쪽에 화분들이 있고 아침에", "0402_2_vocals.0034.wav": "물 주려고 문을 탁 열면 식물 냄새 진짜 좋아요", "0402_2_vocals.0035.wav": "식물이 주는 그 냄새는 진짜 큰 선물이에요", "0402_2_vocals.0036.wav": "여러분들도 화분 하나씩이라도 한번 키워보세요 재밌어요", "0402_2_vocals.0038.wav": "자 그럼 노래 듣고 오겠습니다", "0402_2_vocals.0039.wav": "손은경 신청곡 프라이머리 조만간봐요 6 윈이에요", "0402_2_vocals.0040.wav": "자 검색어로 볼륨 낮춰 때 사람 만나 보고 있습니다 볼륨 검색어 오늘의 검색어는 조금이에요 조금 임영미 씨 가요 유대 요즘 살이 쪄서 걱정하고 했더니 남편이 훌라후프를 사 왔더라고", "0402_2_vocals.0041.wav": "근데 훌라후프 안쪽에 혹처럼 튀어나와서 엄청 아픈 거 있죠", "0402_2_vocals.0042.wav": "조금 돌렸는데 허리에 붉은 띠가 생겼어요", "0402_2_vocals.0043.wav": "아 이거 아파 이거 아파 이거 못 해요 훌라후프 아픈 거 진짜 얼마나 아픈데 종류가 진짜 많더라고요", "0402_2_vocals.0044.wav": "저는 원래 훌라후프 진짜 잘 들려요 돌리면서 뛰고 별지도 않은데", "0402_2_vocals.0045.wav": "아 이거 아픈 거는 한 바퀴도 못 들리게 더라고요", "0402_2_vocals.0046.wav": "진짜 허리 살빼는데 좋다고 하던데요", "0402_2_vocals.0048.wav": "조민희 씨가 제 이름은 조민이에요 한글 이름이죠 크크", "0402_2_vocals.0049.wav": "태어날 때 너무 작아서 간호사분들이 미니 같다고 했대요 그래서 아버지가 미니로 지으셨다는데요", "0402_2_vocals.0051.wav": "흔하지 않은 이름이라 사람들이 잘 못 알아 들어서 조금은 불편하지만 만족합니다", "0402_2_vocals.0053.wav": "야 간호사 언니", "0402_2_vocals.0054.wav": "본의 아니게 작명가가 됐네요", "0402_2_vocals.0055.wav": "미니 같다고 하는 표는 처음 들어봐요", "0402_2_vocals.0056.wav": "애기 너무 조금 하다 민희 같아 물어볼게", "0402_2_vocals.0057.wav": "미니미라고 하지 않았을까", "0402_2_vocals.0058.wav": "장거리 님이라고 하잖아요", "0402_2_vocals.0059.wav": "그래야 조민 이미지 좀 그동안 이름 부를때", "0402_2_vocals.0060.wav": "뭐 예를 들면 그냥 김종민 이렇게 조금 인신 줄 알았어요 그래서 조민이이라고 하시는 건 줄 알았는데", "0402_1_vocals.0000.wav": "방을 기다리다 병에 걸려 죽은 자리에서 따는 등수", "0402_2_vocals.0061.wav": "정말 설레임이 조민희", "0402_2_vocals.0063.wav": "너 102 4시 언니 저 너무 힘들어요", "0402_2_vocals.0065.wav": "대학교 왔는데 왜 이렇게 마음 무겁게 힘들까요", "0402_2_vocals.0067.wav": "목적도 꿈도", "0402_2_vocals.0068.wav": "너무 멀리 있는 거 같고 지쳐", "0402_2_vocals.0069.wav": "조금은 여유롭고 행복하게 대학생을 하고 싶은데", "0402_2_vocals.0070.wav": "언니는 대학생 때로 돌아가면 뭘 가정해 보고 싶어요", "0402_2_vocals.0072.wav": "인아 씨가 당연히 지금 뭐", "0402_2_vocals.0073.wav": "너무 멀리 있는 거하고 까마득하고", "0402_2_vocals.0074.wav": "그래서 좀 힘들긴 하겠죠 근데 모든 대학생들 마찬가지 아닐까요 같은 상황은 같은데", "0402_2_vocals.0075.wav": "받아들이는게 좀 다르잖아요 성격에 따라서", "0402_2_vocals.0076.wav": "여유를 갖고 생각해 보세요 인혜 씨 지금 어떻게 해도 올 날은 옵니다 안 올라는 안 오고요", "0402_2_vocals.0077.wav": "전대 학생 때로 돌아가면 여행 다니고 싶어", "0402_2_vocals.0079.wav": "여행을 너무 안 다녔던 거 같아", "0402_2_vocals.0080.wav": "그냥 자유롭게 여행 다니고 씨씨 해 보고 싶고", "0402_2_vocals.0082.wav": "아 너 736 님이요 유지 스스로 앞머리를 잘랐는데 삐뚤삐뚤한 술 조금만 더 조금만 더 하다가 완전 추사랑 됐어요 추상우 기억이라도 하죠 어쩌면 좋아요 바보 같아요 유유", "0402_2_vocals.0084.wav": "변발 안 된게 어디에요", "0402_2_vocals.0086.wav": "이게 미용실 가야 돼요 이렇게 전문가가 괜히 전문가가 아니라니까요", "0402_2_vocals.0087.wav": "앞머리 자르는 거 쉬워 보이지만 그게 균형을 맞춰서 일자로 자르는게 얼마나 어려운데", "0402_2_vocals.0088.wav": "근데 추석 머리는 진짜 진짜로 귀엽잖아요", "0402_2_vocals.0090.wav": "조금 일찍 길을 다 보면 자리 잡아서 제일 예쁜 순간에도 올 거예요", "0402_2_vocals.0092.wav": "자 오늘 볼륨 검색어는 여기까지입니다 내일도 이거 주니까 기대해 주시고요 보내 주신 사람들 항상 고맙게 보고 있습니까 많은 이야기 들려 주세요 그럼 저는 노래 한 곡 듣고 오겠습니다 신청 곡이에요 정키의 홀로", "0402_3_vocals.0000.wav": "이야기도 더 만나 볼게요 이야기", "0402_3_vocals.0001.wav": "강준현 씨", "0402_3_vocals.0002.wav": "5월에 제대를 앞둔 군인입니다", "0402_3_vocals.0003.wav": "휴가를 하는데 친구들이 다 군대 있어서 그냥 혼자 도서관에 와서 용 그랬어", "0402_3_vocals.0004.wav": "너무 오랜만에 하는 공부 라 그런지 적응 이 안되네", "0402_3_vocals.0005.wav": "그래도 슬슬 해야 될 것 같아요", "0402_3_vocals.0006.wav": "오 좋은데", "0402_3_vocals.0007.wav": "어 저였으면 안 와 군대를 갔다 와서 저녁을 하고 나면 몇 달은 시원하게 놀 것 같아", "0402_3_vocals.0008.wav": "시원하게 놀기", "0402_3_vocals.0009.wav": "자 이제 시작해 보자", "0402_3_vocals.0010.wav": "이럴 텐데", "0402_3_vocals.0011.wav": "그러면 또 적응도 안되고 좀 힘들 수 있겠어요 준영이처럼 이렇게", "0402_3_vocals.0012.wav": "제대로 한 두 달 앞둔 상황이", "0402_3_vocals.0013.wav": "응 먼저 미리 좀 4회 다 살려 애들을 슬슬 준비하는 거", "0402_3_vocals.0014.wav": "그게 훨씬 더 현명한 방법인 거 같네요", "0402_3_vocals.0015.wav": "날씨", "0402_3_vocals.0017.wav": "1825", "0402_3_vocals.0018.wav": "저를 좋아하는 남자애가 있는데", "0402_3_vocals.0019.wav": "어떻게 하면 될까요", "0402_3_vocals.0020.wav": "거절하면 사이가 안 좋아질까 걱정돼요", "0402_3_vocals.0023.wav": "안 좋구나 182 언니는 안타깝네요", "0402_3_vocals.0024.wav": "솔직하게 말해요 그래 가장 좋은 방법일 거예요 나는 네가 친구로서 진짜 좋은데", "0402_3_vocals.0025.wav": "연애를 하고 싶거나 그런 마음은 좀 안 된다", "0402_3_vocals.0026.wav": "그래도 너랑 친하게 지내고 싶다", "0402_3_vocals.0027.wav": "하지만 그게 너에게 고통을 주는 일이라며", "0402_3_vocals.0028.wav": "내가", "0402_3_vocals.0029.wav": "물러서기", "0402_3_vocals.0030.wav": "뭐 그렇게 얘기", "0402_3_vocals.0031.wav": "헤어져 그게 또 상대방에 대한 예의", "0402_3_vocals.0033.wav": "그리고", "0402_3_vocals.0034.wav": "7019 님", "0402_3_vocals.0035.wav": "저는 차에 카메라와 삼각대를 싣고 다녀요", "0402_3_vocals.0036.wav": "사진 찍는 걸 너무 좋아하거든요", "0402_3_vocals.0037.wav": "집에 가까운 청담대교 야경을 찍고 왔는데", "0402_3_vocals.0038.wav": "서울의 밤 풍경 매일 보지만 예쁘네요", "0402_3_vocals.0040.wav": "정말 서울의 밤은 예뻐요", "0402_3_vocals.0042.wav": "우리나라는 진짜 한강이 있잖아요 한강을 따라", "0402_3_vocals.0043.wav": "다리들이", "0402_3_vocals.0044.wav": "예쁜 거 같아 어느 나라를 가도 그 다리가 참", "0402_3_vocals.0045.wav": "아름다운 거 같아", "0402_3_vocals.0046.wav": "조명도 많고요", "0402_3_vocals.0049.wav": "주민선 씨", "0402_3_vocals.0050.wav": "유지", "0402_3_vocals.0051.wav": "학교를 왕복 5시간 걸려서 통화를 하고 있는데", "0402_3_vocals.0052.wav": "버스 안에서 긴 시간동안 뭘 해야 할 시간을 잘 활용했다고 소문이 날까요", "0402_3_vocals.0053.wav": "자는 거죠", "0402_3_vocals.0055.wav": "와우 잘 잤다 어우 꿀잠 잤네", "0402_3_vocals.0056.wav": "할 정도로 자는 거야", "0402_3_vocals.0058.wav": "조금 할 일을 해 놓고 늦게 자고 일찍 일어나기", "0402_3_vocals.0059.wav": "일찍이 일찍 일어나는 거 얼마나 좋아요", "0402_3_vocals.0060.wav": "평소보다 2시간 일찍 일어나서", "0402_3_vocals.0061.wav": "이뻐서 안 주무세요 왜냐하면", "0402_3_vocals.0062.wav": "뭐 보면 눈 나빠지니까요 버스 안에서 그 시간이 책 보면 너무 좋지만", "0402_3_vocals.0063.wav": "눈 나빠지니까", "0402_3_vocals.0064.wav": "음악도 들으시구요 아 그런 거 있잖아 만약에 공부 하는 거 있으면 그런 거", "0402_3_vocals.0065.wav": "오디오 로 공부 하는 거 많이 있으니까 들으면서 공부 다시", "0402_3_vocals.0067.wav": "자 노래 들려 드릴게요 7179 님의 신청곡 블락비의 보기드문여자이어서 엑소입니다 탄다", "0402_3_vocals.0068.wav": "볼륨 녀석들의 못다한 이야기 만나 보고 싶습니다", "0402_3_vocals.0069.wav": "산을 좀 더 소개해 드릴게요", "0402_3_vocals.0072.wav": "3938", "0402_3_vocals.0073.wav": "어제 학교에서 영어시간에 눈치 게임 했어요", "0402_3_vocals.0074.wav": "제시된 영어 단어 외우는 거였는데요", "0402_3_vocals.0075.wav": "제가 일어날 때마다 다른 친구도 동시에 일어나더라구요", "0402_3_vocals.0076.wav": "유진은 눈치 게임 잘해", "0402_3_vocals.0077.wav": "일하는 것도 비법 이 있을까", "0402_3_vocals.0079.wav": "좆 잘하는 거 같아", "0402_3_vocals.0080.wav": "되게 못 하지 나는 거 같아", "0402_3_vocals.0081.wav": "빅토리 보지 않아 어떻게 하더라", "0402_3_vocals.0083.wav": "숫자로 치면은", "0402_3_vocals.0084.wav": "시작", "0402_3_vocals.0085.wav": "이렇게 바로 하던가", "0402_3_vocals.0086.wav": "아니면 뭐 사 정도를 외친다", "0402_3_vocals.0087.wav": "그러면", "0402_3_vocals.0088.wav": "애매하게 4동 3 하기 쉽잖아요", "0402_3_vocals.0089.wav": "10월부터 내가 한다라는 걸 티를 내면서 슬슬", "0402_3_vocals.0091.wav": "이런 식으로 아주 천천히 하는 거예요", "0402_3_vocals.0092.wav": "그런데 옆 사람들", "0402_3_vocals.0093.wav": "동시에 걸리자 않게끔 있나 가사를 하네 나도 해야겠다라고 생각하도록 이렇게 좀 치려면", "0402_3_vocals.0094.wav": "해 보세요 그렇게", "0402_3_vocals.0097.wav": "한정식", "0402_3_vocals.0098.wav": "저는 오늘 옷 정리를 했어요 사이즈가 작아서 입지도 않은 새 5시", "0402_3_vocals.0099.wav": "다섯 번이나 나와서 완전 깜짝 놀랐네요", "0402_3_vocals.0100.wav": "제가 대체 옷에게 무슨 짓을 한 걸까요", "0402_3_vocals.0101.wav": "아", "0402_3_vocals.0103.wav": "저도 그런 거 많더라고요", "0402_3_vocals.0104.wav": "언니 는 뭐 이미 다 크고 나서 잘못 들인데", "0402_3_vocals.0105.wav": "보니까 3월인데 오 너무 작은 거예요", "0402_3_vocals.0106.wav": "무슨 생각으로 되나 이거 존나 와 진짜 예쁘고 귀엽긴 한데", "0402_3_vocals.0107.wav": "내가 이걸 어떻게 입 날 정도로 좀 작고 너무", "0402_3_vocals.0108.wav": "좀 너무 나쁘다고 해야 할까", "0402_3_vocals.0110.wav": "그래서 그런 거", "0402_3_vocals.0111.wav": "얼마 전에", "0402_3_vocals.0112.wav": "여덟 정리해서 아이유 양한테 줬는데", "0402_3_vocals.0113.wav": "이거 줄지 모르겠네요", "0402_3_vocals.0116.wav": "8171", "0402_3_vocals.0117.wav": "저는 향기에 되게 민감해요", "0402_3_vocals.0118.wav": "특히 좋은 향기를 맡으면 모르는 사람에게", "0402_3_vocals.0119.wav": "무슨 영수증 지불 꺼내는데", "0402_3_vocals.0120.wav": "유지도 형수 좋아요 무슨 약 있어", "0402_3_vocals.0121.wav": "아 저 진짜 연기 좋아하죠 저는", "0402_3_vocals.0122.wav": "한기", "0402_3_vocals.0123.wav": "진짜 민감해요 저 심각하게 향기 제품들의", "0402_3_vocals.0124.wav": "정말 좋아해서 아주 많이 사 모으기도 하고 아주 많은 관심 있기도 하고 저는", "0402_3_vocals.0126.wav": "그래", "0402_3_vocals.0127.wav": "메시지", "0402_3_vocals.0129.wav": "나무 타는 냄새", "0402_3_vocals.0130.wav": "같은 것도", "0402_3_vocals.0131.wav": "안 좋아하시는 분들이 많은 호불호가 갈리는 그런 냄새를 좋아하는 편이고요", "0402_3_vocals.0132.wav": "상큼한 건 어릴 땐 좋아했는데 요즘에는 그냥 그래요 꽃 냄새는 진짜 좋아하고", "0402_3_vocals.0133.wav": "그런 거 좋아하죠 흙냄새", "0402_3_vocals.0134.wav": "슬램 시흥 냄새", "0402_3_vocals.0135.wav": "그런데 향수들이 있잖아요", "0402_3_vocals.0136.wav": "그런 거 좋아해요", "0402_3_vocals.0138.wav": "늑대를 좋아하는", "0402_3_vocals.0139.wav": "향이 있다기보다는", "0402_3_vocals.0140.wav": "맡아서 좋음", "0402_3_vocals.0141.wav": "앞에서 끌림", "0402_3_vocals.0142.wav": "다 좋은 거 같아요 어 약간 이런게 얘기하고 보니까 이상형이랑 비슷한 거 같아", "0402_3_vocals.0143.wav": "왜 다 그렇지 않아", "0402_3_vocals.0144.wav": "이상형이 정확히", "0402_3_vocals.0147.wav": "키 크고 피부 색깔 이렇고 뭐가 이런 스타일의 여자 이렇게 딱 정해져 있어도 4시 만나는 사람 보면 유정유일해서 그리고 저는 전철에서 그리고 그런 거 있잖아요", "0402_3_vocals.0148.wav": "그런 것처럼 향수를 좋아하는 거 같아", "0402_3_vocals.0150.wav": "자 노래 듣고 있습니다 김난경 시신 천국이에요", "0402_3_vocals.0151.wav": "이적의 하늘을 달리다", "0402_4_vocals.0000.wav": "시간 남겨 주신 마태호 페리클레스의 작가의 창에 여자 듣고 신곡은 캐스커의 모든 토요일였습니다", "0402_4_vocals.0004.wav": "창문에 대한 책이 있군요", "0402_4_vocals.0005.wav": "작가들의 서재 나무 독서", "0402_4_vocals.0007.wav": "그런 거에 대한 책은 봤는데", "0402_4_vocals.0008.wav": "작가들이 생각하는 창문이 다니고 신선한데", "0402_4_vocals.0010.wav": "현성 씨가 남겨주신 구절은", "0402_4_vocals.0011.wav": "브라질 작가", "0402_4_vocals.0012.wav": "다니엘 갈래의 창밖 풍경에 대한 그림", "0402_4_vocals.0013.wav": "어떤 창문은 몽상을 위한 탈출구 짐은 어떤 창문은 함께 하는 친구", "0402_4_vocals.0016.wav": "그리고 그림이 좀 궁금해졌어요 여러 창문들이 궁금하기도 하고", "0402_4_vocals.0017.wav": "현정 씨 반납 고맙습니다", "0402_4_vocals.0018.wav": "자 그럼 두 번째 작품 만나서 줘", "0402_4_vocals.0019.wav": "이정희 씨가 남겨졌다", "0402_4_vocals.0020.wav": "이기호", "0402_4_vocals.0021.wav": "초간단 또띠아 토스트 레시피", "0402_4_vocals.0024.wav": "그의 어머니는 부엌 식탁에 어지럽게 널려있는 밀가루 반죽을 바라보다가", "0402_4_vocals.0025.wav": "그에게 조용히 물었다", "0402_4_vocals.0026.wav": "만두를 해 먹으려고 했던 거니", "0402_4_vocals.0027.wav": "그는 부모님의 얼굴을 본 후", "0402_4_vocals.0028.wav": "처음엔 당황스러웠습니다", "0402_4_vocals.0029.wav": "이네 부끄러워졌다", "0402_4_vocals.0030.wav": "또띠아를 해 보려고", "0402_4_vocals.0031.wav": "뭐", "0402_4_vocals.0033.wav": "도티", "0402_4_vocals.0034.wav": "여보", "0402_4_vocals.0035.wav": "얘가 지금 뭐라는 거야", "0402_4_vocals.0036.wav": "그래야 아버지가 어머니를 보면서 물었다", "0402_4_vocals.0037.wav": "뽀삐를 왜 먹어이 새벽에", "0402_4_vocals.0038.wav": "그는 거기까지 듣고", "0402_4_vocals.0039.wav": "다시 아무 말 없이 그의 방으로 달려 들어갔다", "0402_4_vocals.0041.wav": "하고 싶은 말은 많았으나", "0402_4_vocals.0042.wav": "그저 모든 것이 부끄러워 졌을 뿐이다", "0402_4_vocals.0043.wav": "나는 그저 무언가를 다시 해보려고 했을 뿐인데", "0402_4_vocals.0044.wav": "그는 괜스레 케이블 TV 속 셰프가 원망스러웠다", "0402_4_vocals.0045.wav": "누구에겐 초간단 요리가", "0402_4_vocals.0046.wav": "또 누군가에겐 그렇지 않으면", "0402_4_vocals.0047.wav": "아무도 그것을 말해 주는 사람은 없었다", "0402_4_vocals.0049.wav": "주인공은 새벽 3시에 TV를 보다가", "0402_4_vocals.0050.wav": "또띠아 토스트를 해 먹고 싶어서 방에서 나와요", "0402_4_vocals.0052.wav": "오늘은 시간 취업이 되지 않아 모든 것에 의혹을 이런 어떤 남자가 드디어", "0402_4_vocals.0053.wav": "뭔가를 해보고 싶었던 거예요", "0402_4_vocals.0054.wav": "하지만 또띠아 반죽이 없어서 밀가루로 그걸 만들기 시작하자", "0402_4_vocals.0055.wav": "누구나 만들 수 있다니까", "0402_4_vocals.0056.wav": "싫다니까", "0402_4_vocals.0057.wav": "금방 할 줄 알았는데", "0402_4_vocals.0058.wav": "반죽을 밀던 소주병을 깨뜨리면서 부모님을 깨워요", "0402_4_vocals.0059.wav": "그리고 이렇게 됩니다", "0402_4_vocals.0060.wav": "왠지 부끄러워서", "0402_4_vocals.0061.wav": "뭔가 한다는게", "0402_4_vocals.0062.wav": "남들에게는 쉬운 일이 자신에게는 버겁다는게 부끄러워서", "0402_4_vocals.0063.wav": "다시 방으로 달려들어 가고 말지", "0402_4_vocals.0065.wav": "제 이야기 같아서 마음이 너무 먹먹해져 써요 눈물이 날 정도로", "0402_4_vocals.0066.wav": "하지만 누구나 그렇다고 생각해요", "0402_4_vocals.0067.wav": "모두가", "0402_4_vocals.0068.wav": "뭐든 쉬운 건 아니라고", "0402_4_vocals.0069.wav": "그러니까 볼륨 가족에게도 위로가 되었으면 하는 마음으로 남겨요", "0402_4_vocals.0070.wav": "이기호", "0402_4_vocals.0071.wav": "초간단 또띠아 토스트 레시피", "0402_4_vocals.0072.wav": "이정희 씨가 남겨주신 이기호의", "0402_4_vocals.0073.wav": "초간단 또띠아 토스트 레시피 신곡은", "0402_4_vocals.0074.wav": "양희은의 나영이네 냉장고에 넣었습니다", "0402_4_vocals.0076.wav": "초간단 또띠아 토스트 레시피는 소설가 이기호", "0402_4_vocals.0077.wav": "웬만해선 아무렇지 않다", "0402_4_vocals.0078.wav": "소설 집에 있는 그렸어요", "0402_4_vocals.0079.wav": "아주 짧은 한선호 장쯤 되는", "0402_4_vocals.0080.wav": "소설 40여편의 몇 개 있는 책 이래", "0402_4_vocals.0081.wav": "위치지만 묵직한 통찰이 실린 글 드리려고 합니다", "0402_4_vocals.0083.wav": "이 작가의 말도 재밌어요 짧은 소절을 묶은 책이니까", "0402_4_vocals.0084.wav": "작가의 말 도시조 형식으로 적어보겠다", "0402_4_vocals.0085.wav": "짧은 글 우습다고 14리 덤볐다가", "0402_4_vocals.0086.wav": "편두통 위장장애 골고루 알았다네", "0402_4_vocals.0087.wav": "짧았던 사랑일수록 치열하게 닫혔거나", "0402_4_vocals.0090.wav": "재밌을 거 같은데", "0402_4_vocals.0091.wav": "저도 이제", "0402_4_vocals.0092.wav": "자주 바라겠어", "0402_4_vocals.0093.wav": "반납해 주신 정희씨 말처럼 읽으면서 왠지 짠해지는 그렸어요", "0402_4_vocals.0094.wav": "맞아 정희 씨 누구나 뭐든 쉬운 사람은 없어요 다들 그렇게 간단하게 이야기 하고 해 내기까지는 아주 많은 시간이 필요 없을 겁니다", "0402_4_vocals.0095.wav": "다행이에요 저 시각을 알고 있어서 힘을 낼 준비가 되어 있어서", "0402_4_vocals.0097.wav": "우리도 다들 힘냅시다", "0402_4_vocals.0098.wav": "오늘 반납해 주신 2분께는 선물 보내 드릴게요", "0402_5_vocals.0000.wav": "안녕 가겠습니다", "0402_5_vocals.0001.wav": "6305 님이야", "0402_5_vocals.0002.wav": "사소한 일로 안이랑 다투다가 큰 소리 치면서 방문을 나섰는데요 그냥 나가면 좀 모양 빠질 거 같아서 방문을 아주 세게 닫았는데", "0402_5_vocals.0003.wav": "미쳐 발가락을 문틈 사이에서 못 봤네요 아 정말", "0402_5_vocals.0004.wav": "세상에 태어나서 이렇게 살 떨리게 아픈 적은 처음이었어요", "0402_5_vocals.0005.wav": "남자분들", "0402_5_vocals.0006.wav": "아내랑 싸워도 문은 살살 닫읍시다", "0402_5_vocals.0007.wav": "창의문", "0402_5_vocals.0008.wav": "문은 살살 닫아야 돼요", "0402_5_vocals.0009.wav": "어릴 때도", "0402_5_vocals.0010.wav": "사춘기 시절에 엄마한테 혼나면", "0402_5_vocals.0011.wav": "이거 이런 거 같아 문을 세게 닫으면 때는 다", "0402_5_vocals.0012.wav": "엄마 나 아내보다 약한 입장일 때", "0402_5_vocals.0013.wav": "약자 1대 균이 큰 소리는 좀 못 냈고 좀 뭔가 반항을 하고 싶고", "0402_5_vocals.0014.wav": "그래도 세계 된다는게 문이죠", "0402_5_vocals.0015.wav": "다 끝나면", "0402_5_vocals.0016.wav": "얼마나 우리 그 민망 합니까 금방 반성하겠다 일부러", "0402_5_vocals.0017.wav": "눈은 언제나 아무리 화가 나도 문은 살살 닫는 걸로", "0402_5_vocals.0018.wav": "최미정 씨 남편이 새 차를 뽑았는데요 조수석에 앉아 다리가 아파서 신발 벗고 다리를 올렸더니 소리를 고래고래 지르면서 깨끗한 차를 목욕 했다며 화를 내네요 아니 제가 뭘 잘못한 거지", "0402_5_vocals.0019.wav": "완전 어이없음", "0402_5_vocals.0021.wav": "소리를 고래고래 지른 건 조금 너무 했고요", "0402_5_vocals.0023.wav": "그럴 수는 있어요 저도 새차 뽑았을때", "0402_5_vocals.0024.wav": "되게 신경 쓰여요 왜냐면 내가 너무 살살 살살 차를 다르고 있으니까 만약에 누군가를 태웠는데", "0402_5_vocals.0026.wav": "심지어", "0402_5_vocals.0027.wav": "모래밭을 이따 타면 그것조차 신경 쓰이고", "0402_5_vocals.0028.wav": "너무 세게 쾅 닫으면 내 마음", "0402_5_vocals.0029.wav": "이렇게 예정테니까 기다려 남자분들 얼마나 차를 아끼니까 그럴 수도 있죠 근데", "0402_5_vocals.0030.wav": "1년만 지나면 그때는 정말 멋진 신경도 안 쓰잖아요 정식아 조금만 참고", "0402_5_vocals.0031.wav": "살살 다뤄주세요 잘", "0402_5_vocals.0033.wav": "8 7 6 5 님 집에 아무도 없어서 혼자 밥해 먹고 있네요 밥은 전기밥솥이 대신 해 주지만 반찬은 제가 남자인지라 너무 허전하네요", "0402_5_vocals.0034.wav": "같이 밥 먹는 기분 좀 내 주세요", "0402_5_vocals.0036.wav": "사진을 이렇게 보내 주셨네요", "0402_5_vocals.0037.wav": "으악", "0402_5_vocals.0038.wav": "밥이랑 구운 햄만 딸랑 있어요", "0402_5_vocals.0040.wav": "근데 솔직히 맛있겠다", "0402_5_vocals.0041.wav": "충분하죠", "0402_5_vocals.0042.wav": "이거 8 7 6 님", "0402_5_vocals.0044.wav": "반찬이 많을수록 맛이 없어져요 밥은", "0402_5_vocals.0045.wav": "너무 먹을게 많으면 이거 맛이 중구난방", "0402_5_vocals.0046.wav": "제주 주제가 없다고 주제가", "0402_5_vocals.0047.wav": "오늘의 주제는 셈이에요 좋아요 8765 님 다음에는 버터랑 간장 참기름", "0402_5_vocals.0048.wav": "어 맛있다 비벼서 계란 프라이 하나 해서 그렇게만 드세요", "0402_5_vocals.0051.wav": "017 님 33살 첼로를 배우기 시작했어요", "0402_5_vocals.0052.wav": "어릴 때 막연하게 1학기에 대한 로망이 있었는데 드디어 시작을 했습니다 아직 갈 길이 멀 짐 안 그래도 언젠가는 멋지게 연주할 그날을 꿈꾸며 열심히 배울게요", "0402_5_vocals.0053.wav": "비록 누가 알아 주지 않는다고 해도 제게 기쁨으로 주거든요", "0402_5_vocals.0054.wav": "언니는 이런 제 맘 알아 주실 거죠", "0402_5_vocals.0055.wav": "그럼요 제가 얼마나 작년 재작년에 바이올린 배우고 싶다는 이야기를 많이 했으니까", "0402_5_vocals.0056.wav": "남들이 알아주는 거는", "0402_5_vocals.0057.wav": "아무짝에 쓸모가 없어요 사실 내가 즐거우면 그만인 거죠", "0402_5_vocals.0058.wav": "017", "0402_5_vocals.0059.wav": "실력이 어느 정도가 오늘 늘지는 모르겠지만 어쨌든 배우는 동안 연습하는 동안 행복하면 그걸로 충분한 겁니다", "0402_5_vocals.0061.wav": "자 그럼 노래 듣고 올게요 추석 신청곡 와플에 연애하고 싶다이어서 740 신청곡입니다 박보람의 예뻐졌다", "0402_5_vocals.0062.wav": "KBS 쿨 FM 유인나의 볼륨을 높여요 불륜 가족들의 사연 만나보겠습니다 3938", "0402_5_vocals.0063.wav": "저는 초등학교 6학년이에요 학교 수업 시간에 발표를 하고 싶은데 제 성격이 너무 소심하고 내성적인이라 손을 잘 못 들어요 타이밍도 놓치고요 유유", "0402_5_vocals.0064.wav": "인아 언니", "0402_5_vocals.0065.wav": "발표 잘하는 방법 없을까요", "0402_5_vocals.0067.wav": "어려운 질문이다 이런 건 사실 상상 달님", "0402_5_vocals.0068.wav": "저한테 하루 전날 이런 질문을 주셔야지 제가 생각해 보고 답변을 할 수 있는 거 같아요 아무려면", "0402_5_vocals.0069.wav": "DJ라고 해서 모든 것이 해결사가 아니기 때문에", "0402_5_vocals.0070.wav": "이거 제가 좀 생각해 보고 얘기해도 될까요", "0402_5_vocals.0071.wav": "내일 얘기하자", "0402_5_vocals.0072.wav": "사무장 팔리움 좋아요 내일 발표 잘하는 방법에 대해서 이야기해 보도록 하세", "0402_5_vocals.0074.wav": "2080 님 27살 직장인입니다", "0402_5_vocals.0075.wav": "주말마다 가는 PC 방에 있는데 그 곳에서 일하시는 분이 완전 제 이상형입니다", "0402_5_vocals.0076.wav": "번호 교환을 좀 하고 싶은데 쪽지로 할까요 아니면 말로 할까요", "0402_5_vocals.0080.wav": "전화는 말로 하는게 좋을 거 같아", "0402_5_vocals.0081.wav": "왜냐면 쪽지 일까지는 좋아요", "0402_5_vocals.0082.wav": "그래 쪽지로 시작된 인연이", "0402_5_vocals.0083.wav": "문감 말로 바뀌는 그 어젯밤 있잖아요", "0402_5_vocals.0084.wav": "영원히 쪽지로 해야만 할 거 같아", "0402_5_vocals.0086.wav": "편지로 시작한 사이가 실제로 만나면 좀 그걸 좀 소화하기가 어려울 수 있잖아요 나 처음부터 그냥 관심 있다는 걸 좀 치를 4시고요", "0402_5_vocals.0087.wav": "여러가지 말들 한 마디만 걸어도 조 사람", "0402_5_vocals.0088.wav": "뭐야 이거 얼마예요 이런 거 말고 나한테 사적인 걸 뭐 식사는 하셨어요 라든지 이런 것만 해도 사실 알바생 입장에서는", "0402_5_vocals.0089.wav": "뭐야 나한테 관심 있나 하잖아요", "0402_5_vocals.0090.wav": "그런 걸 좀", "0402_5_vocals.0091.wav": "엄마 집 걸다가", "0402_5_vocals.0092.wav": "실적", "0402_5_vocals.0093.wav": "이미 저 사람이 내가 이거 뭐야 내가 좋아한다는 걸 눈치 챘다 싶을 때", "0402_5_vocals.0094.wav": "너무 놀라지 않도록 연락처 한번 물어봐 보세", "0402_5_vocals.0096.wav": "양윤영 씨는요 썸타는 남자가 있는데요 만나서 밥을 먹는데 자꾸만 저를 먼저 챙겨 주는 거예요 밥 먹다가 흘리면 휴지 들고 와서 막 닦아 주고", "0402_5_vocals.0097.wav": "제 맘을 자꾸 콩닥콩닥 떨리게 만드는데", "0402_5_vocals.0098.wav": "이 남자도 저랑 같은 마음이지 궁금해요 대놓고 물어봐도 괜찮을까요", "0402_5_vocals.0099.wav": "응", "0402_5_vocals.0100.wav": "당연히 같은 마음일 것 같은데요", "0402_5_vocals.0101.wav": "챙겨 주고 막 닦아 주고 엄마 이거는 뭐", "0402_5_vocals.0102.wav": "엄청 윤영실 보다 더 관심이 많은 거 아닐까요", "0402_5_vocals.0106.wav": "강병민 씨 가요 혼자 심야 영화를 보러 가려고요 우리는 영화 보면서 뭐 먹어요 뭘 먹으면 좋을지 고민 중인데 못 고르겠어요", "0402_5_vocals.0107.wav": "점촌 영화관에서 파는 거 멈춰 팝콘 아니면 그 오징어 버터구이 오징어 있잖아요 핫도그 까진 안 먹어", "0402_5_vocals.0109.wav": "카톡으로는 뭔가", "0402_5_vocals.0110.wav": "그 안에들은 내용물들이", "0402_5_vocals.0111.wav": "삐져 나오고 너 흘리고 영화에서", "0402_5_vocals.0112.wav": "시선을 떼서 핫도그를 봐야만 할 것 같은 음식이라서", "0402_5_vocals.0113.wav": "안 먹고 그냥 영화를 보면서 손으로만 먹을 수 있니", "0402_5_vocals.0114.wav": "오징어 라든지 팝콘을 먹습니다", "0402_5_vocals.0115.wav": "영화 사실 팝콘 먹으러 가는 거 아니에요", "0402_5_vocals.0116.wav": "결혼할 때 먹는 파쿠르 진짜 맛있어", "0402_5_vocals.0117.wav": "또 생각해보면", "0402_5_vocals.0118.wav": "영화관 아닌 곳에서 먹는 팝콘은 뭔가요 저는 영화관 아닌 곳에서 난 거의 팝콘 안 먹어 본 거 같아", "0402_5_vocals.0120.wav": "자 알아듣냐 3938", "0402_5_vocals.0121.wav": "초등학교 6학년 학생이", "0402_5_vocals.0122.wav": "발표 잘하는 방법은 내일", "0402_5_vocals.0123.wav": "이야기 나눠 보도록 하고 어차피 너 내일 학교 안 가잖아 다음 주에 학교 가는 거니까 내일 얘기하고", "0409_1_vocals.0000.wav": "유인나의 볼륨을 높여요 4월 9일 토요일", "0409_1_vocals.0001.wav": "자꾸 그러네", "0409_1_vocals.0002.wav": "이적의 그대랑 드셨습니다", "0409_1_vocals.0004.wav": "어떤 스님이 그랬대", "0409_1_vocals.0005.wav": "내가 선택해야 하는 것은", "0409_1_vocals.0006.wav": "내 안에서 찾을 수 밖에 없다", "0409_1_vocals.0007.wav": "남들에 말도 결국", "0409_1_vocals.0008.wav": "그냥 남들을 경험일 뿐이니까", "0409_1_vocals.0009.wav": "너무 많은 정보가 오히려 방해가 될 때가 있다고", "0409_1_vocals.0011.wav": "충분히 고민을 하고 결정하자", "0409_1_vocals.0012.wav": "그 시간을 조금씩 줄이다 보면", "0409_1_vocals.0013.wav": "결정장애라고 해야 하나요 그 결정이 주고", "0409_1_vocals.0014.wav": "쉬워진다", "0409_1_vocals.0015.wav": "고민을 하는 것만으로도 이미 좋은 선택을 할 준비가 되어 있다는 거니까", "0409_1_vocals.0016.wav": "난 왜 결정을 못 내리고", "0409_1_vocals.0017.wav": "왜 이렇게 힘들지", "0409_1_vocals.0018.wav": "그런 것 때문에 걱정하지 말자 구요", "0409_1_vocals.0019.wav": "변 한숙 씨 가요", "0409_1_vocals.0020.wav": "유진이는 꿈이 뭐니라고 물어 봤을 때 어떤 대답을 하실 거예요", "0409_1_vocals.0021.wav": "아직 저한테는 대답하기 힘든 질문이네요 유유", "0409_1_vocals.0024.wav": "동진에요", "0409_1_vocals.0025.wav": "저도 대답하기 힘든 질문이네요", "0409_1_vocals.0026.wav": "어른이다 보니까", "0409_1_vocals.0027.wav": "10대 때는 꿈이 뭐니", "0409_1_vocals.0028.wav": "간호사요 선생님이요 연예인 이요 가수요 대통령이야 과학자", "0409_1_vocals.0029.wav": "이런게 있잖아요", "0409_1_vocals.0030.wav": "근데 커서도 분명히 꿈은 있는데 참", "0409_1_vocals.0031.wav": "옛날과 다른게 있다면", "0409_1_vocals.0032.wav": "현실성을", "0409_1_vocals.0033.wav": "짜져 보게 된다는 거 그래서 어디까지 정을 정해도 될까를 고민을 많이 하게 되는 거 같아", "0409_1_vocals.0034.wav": "그래도 분명히 마음속 깊이", "0409_1_vocals.0035.wav": "꿈이 있다는 거 그게 중요한 거죠 현숙 씨도 그래", "0409_1_vocals.0036.wav": "작은 광고 듣고 와서 불륜 가족들이 사연 만나 볼게요", "0409_1_vocals.0046.wav": "검색어 하나를 정해서 부른 가족들의 사연을 꼼꼼하게 만나는 시간입니다", "0409_1_vocals.0047.wav": "볼륨 검색", "0409_1_vocals.0048.wav": "오늘의 검색어는 약속으로 정해 봤어요 자 약속 어떤 이야기를 보내셨는지 만나겠습니다", "0409_1_vocals.0049.wav": "자궁 167", "0409_1_vocals.0050.wav": "언니 저고 3인데요 친구들하고 오늘 수능 끝나고 한강 가서 치킨 먹기로 약속하고 지금 열공 중이에요 빨리 그날이 왔으면 좋겠습니다", "0409_1_vocals.0052.wav": "얼마나 남았나요 한 6개월", "0409_1_vocals.0053.wav": "어 금방 이혜영 사실 6개월 그래요 지금 열심히 해", "0409_1_vocals.0054.wav": "그날의 치킨 마셜 신도 마실 수 있을 겁니다", "0409_1_vocals.0055.wav": "어제 열심히 하세요 파이팅 6913 님 곧 어머니 생신이라 약속을 잡고 평소 생각했던 모바일 관리와", "0409_1_vocals.0056.wav": "내일 관리를 받게 해 드렸어요 50대 후반이 드시면서 가끔 심란해 하시는 거 같아 예약했었는데", "0409_1_vocals.0057.wav": "처음에는 사용하시던 어머니께서도 만족해 하시네요 앞으로 표현을 자주하는 아들이 될까 합니다 좋은 결심이에요", "0409_1_vocals.0058.wav": "뭐 딸들도 그렇구요 너무 표현을 안 하게 되는 거 같아요 나이 들면서 특히 중고등학생 그 사춘기 시절을 지내면서", "0409_1_vocals.0059.wav": "좀 편하게 되는 거 같아", "0409_1_vocals.0060.wav": "사춘기 때는 어쩔 수 없다 저도 좀 성인이 되고 나서는", "0409_1_vocals.0061.wav": "엄마 아빠한테 애교도 좀 떨고 간지럽더라도 사랑 고백도 좀 하고", "0409_1_vocals.0062.wav": "그러자고", "0409_1_vocals.0064.wav": "유지 언니 저 좀 위로해 주세요", "0409_1_vocals.0065.wav": "얼마 전 남자친구랑 헤어졌는데요", "0409_1_vocals.0066.wav": "사귈 때 검색 번호 가지고 약속을 했는데", "0409_1_vocals.0067.wav": "왠지 벚꽃이 피면 전 남친이 생각날 것 같아요", "0409_1_vocals.0068.wav": "유유", "0409_1_vocals.0070.wav": "아 그랬구나 이렇게 좋은 봄을 앞두고 왜 헤어졌대요 그래요 작년에", "0409_1_vocals.0071.wav": "작년 이맘때 연애하고 있었나요이 남자", "0409_1_vocals.0072.wav": "아니면 1년 못 만나고 헤어진 건", "0409_1_vocals.0073.wav": "작년에 진짜 좋은 벚꽃 구경을 했다던가", "0409_1_vocals.0075.wav": "확인 그랬으면 더 마음이 아프겠네요", "0409_1_vocals.0077.wav": "추억은 최대한 안 만드는 걸로", "0409_1_vocals.0079.wav": "6932 님 한 달에 한 권 책을 읽기 일주일에 3번 운동하기", "0409_1_vocals.0080.wav": "스스로에게 약속을 했는데요", "0409_1_vocals.0081.wav": "왜 지키지도 못할 거", "0409_1_vocals.0082.wav": "매번 약속은 왜 하는 걸까요", "0409_1_vocals.0084.wav": "저희 엄마가 저한테", "0409_1_vocals.0085.wav": "작심삼일 자꾸 하라고 3일에 한 번씩 작심삼일 하라고", "0409_1_vocals.0086.wav": "이렇게 말씀하신 적이 있었거든요", "0409_1_vocals.0087.wav": "그 다음부터 진짜 조금 생각이 바뀌었어요 그래야 내가 아무리 이게 스스로의 다짐", "0409_1_vocals.0088.wav": "빨리 무너질지 현장 오늘 하루라도", "0409_1_vocals.0089.wav": "헛되이 보내지 않는다면 그걸로 됐다", "0409_1_vocals.0090.wav": "난 생각을 좀 하게 된 거 같아", "0409_1_vocals.0092.wav": "아예 이거 약속 안 하면 진짜 엉망진창 할 수 있어요 그 사람이니", "0409_1_vocals.0094.wav": "열심히 이거 약속을 지켜도록 노력해 보세요", "0409_1_vocals.0095.wav": "자 그럼 노래 듣고 오겠습니다 2차 일 시켜서 신청해 주신 이하이 원투쓰리포이어서 장범준입니다", "0409_1_vocals.0096.wav": "자 검색어로 부른 가족들 사람 만나고 있습니다 볼륨 검색", "0409_1_vocals.0097.wav": "오늘의 검색어는", "0409_1_vocals.0098.wav": "약속이 없다", "0409_1_vocals.0100.wav": "김슬기 씨 가요 언니", "0409_1_vocals.0101.wav": "날씨가 너무 좋아서", "0409_1_vocals.0102.wav": "약속도 없는데 원피스를 무려 3개나 주문했어요", "0409_1_vocals.0103.wav": "이거 원피스 예쁘게 입고 나갈 수 있는 좋은 약속 생겼으면 좋겠습니다", "0409_1_vocals.0105.wav": "잘했어요 슬기씨 약속 잡고", "0409_1_vocals.0106.wav": "옷 사려면 늦죠 옷 사주고 약속 잡는 거죠", "0409_1_vocals.0107.wav": "삼척 없어져", "0409_1_vocals.0108.wav": "이거 어디 나갈 돌아다녀 보세요", "0409_1_vocals.0109.wav": "좋은 일 생길지도 누가 알아", "0409_1_vocals.0110.wav": "정현진", "0409_1_vocals.0111.wav": "얼마 전에 가족들이 다 약속이 있는 바람에 아빠랑 단 둘이서 밥 먹으러 식당에 갔는데요 계속 핸드폰 만지고 바쁜 척하고 그랬어", "0409_1_vocals.0112.wav": "이상하게 아빠가 좋은데 표현을 잘 못 하겠어요", "0409_1_vocals.0114.wav": "유리는 애교 많은 딸이죠 어때요", "0409_1_vocals.0116.wav": "전화 원래는 진짜 애교가 많은 날이었는데 이상하게", "0409_1_vocals.0117.wav": "일을 시작하고 나서", "0409_1_vocals.0118.wav": "몇 년 차가 되면서", "0409_1_vocals.0119.wav": "좀", "0409_1_vocals.0120.wav": "안 그렇게 된 거 같아 집에서는", "0409_1_vocals.0121.wav": "말을 잘 안 하고", "0409_1_vocals.0123.wav": "힘들어", "0409_1_vocals.0124.wav": "꺼져 있고 사실 그렇게 안 힘든데도 괜히 집에 가면 힘들어", "0409_1_vocals.0125.wav": "어디 간 그리고", "0409_1_vocals.0126.wav": "크루즈 나옴", "0409_1_vocals.0127.wav": "와 너 왜 그러니", "0409_1_vocals.0128.wav": "내가 이래서 얼마나 내가 나가고 나면 엄마 아빠", "0409_1_vocals.0129.wav": "아유 제가 얼마나 힘들까", "0409_1_vocals.0130.wav": "라는 생각이", "0409_1_vocals.0131.wav": "뭐 바지가 왜 그랬지 나는 생각을 하는데", "0409_1_vocals.0132.wav": "반성하고 또 그렇고 반성하고 또 그러네", "0409_1_vocals.0134.wav": "엄마 아빠 저는 사실", "0409_1_vocals.0135.wav": "별로 안 힘듭니다", "0409_1_vocals.0138.wav": "7042", "0409_1_vocals.0139.wav": "저 같은 동에 사는 다섯 살 꼬마에게 청원 받았어요 전화 나중에 나랑 결혼해요", "0409_1_vocals.0140.wav": "일하는데 왜 가슴이 떨리는 걸까요 크크크", "0409_1_vocals.0143.wav": "안돼요 그렇게 가슴 이 떨려서 어떻게 해요 진짜", "0409_1_vocals.0144.wav": "요즘 많이", "0409_1_vocals.0145.wav": "궁 하셨나봐요", "0409_1_vocals.0146.wav": "응 빨리 연애 좋은 연애", "0409_1_vocals.0147.wav": "하시기 바랍니다", "0409_1_vocals.0148.wav": "너 7593 님 친구들이 남자 친구 만난다고 저를 버리고가 버렸어요 나중에 헤어지고 만나 달라고 약속 잡기 만 해 봐", "0409_1_vocals.0149.wav": "해 보라고", "0409_1_vocals.0150.wav": "줄래", "0409_1_vocals.0151.wav": "나 진짜 심심하다고", "0409_1_vocals.0154.wav": "그래요 저도 친구가 결혼하고 나니까", "0409_1_vocals.0155.wav": "진짜 맛나게가 잘 안 되더라고요", "0409_1_vocals.0157.wav": "그렇다고 진짜 뭐 헤어지기 바라고 그런 건 아니지만", "0409_1_vocals.0158.wav": "만나 줬으면 좋겠어요", "0409_1_vocals.0159.wav": "중국어", "0409_1_vocals.0160.wav": "결혼한다고 그렇게 안 만나 주나", "0409_1_vocals.0163.wav": "볼륨 검색한 결과집니다 내일도 이어지니까 기대해 주시고요 보내주신 사람들 꼼꼼하게 보고 있으니까 많은 이야기 들려 주십시오", "0409_1_vocals.0164.wav": "저는 노래 한 곡 듣고 올게요 5236 님의 신청곡입니다 메이트의 너에게", "0409_2_vocals.0000.wav": "이브가 시작되었습니다", "0409_2_vocals.0001.wav": "평소에 미쳐 다 나누지 못한 이야기 더 만나 볼게요 못다한 이야기", "0409_2_vocals.0002.wav": "9851 님 언니 저는고 3 학생이에요", "0409_2_vocals.0003.wav": "조회 시간에 담임선생님께서 시험 볼 때 나 공부할 때 항상 부적처럼 지니고 있으면", "0409_2_vocals.0004.wav": "의미 있는 물건을 갖고 다녀 보라고 하셨는데요", "0409_2_vocals.0005.wav": "뭘 갖고 다니면 좋을까요 추천 좀 해 주세요", "0409_2_vocals.0008.wav": "선생님 왜 그런 걸 시켜 가지고 그렇게 고민하게", "0409_2_vocals.0009.wav": "무거운 거 말고요 무조건 가벼운 거 사진 한 장", "0409_2_vocals.0010.wav": "아니면 진짜 엄마 여기다 좀 부족을 좀 써 주세요", "0409_2_vocals.0011.wav": "엄마의 좋은 이야기라든지 소중한 것은 종류를 코팅을 잘 해서 들고 다니면 좋을 것 같네요", "0409_2_vocals.0012.wav": "안상진 씨 이제 제가 아네 옷을 사서 선물해 줬는데요", "0409_2_vocals.0013.wav": "저는 나름 괜찮다고 생각 하는 디자인 인데 항상 마음 에 안 들어요", "0409_2_vocals.0014.wav": "왜 그런 걸까요", "0409_2_vocals.0016.wav": "안성시 안 먹을 거 아닌 거죠", "0409_2_vocals.0019.wav": "그래도 절하면 만약에 제가 장재식 부인 입장 이라며", "0409_2_vocals.0020.wav": "아무리 마음에 안 들어도 이걸 자꾸만 옷을 자기가 직접 골라서 선물해 주는 남편이 진짜 고맙고 사랑스러운 것 같아", "0409_2_vocals.0022.wav": "응 그렇네요 상진 씨 부인", "0409_2_vocals.0023.wav": "너 박종서 씨", "0409_2_vocals.0024.wav": "옛날 살았던 동네를 걸었는데 그냥 아무 데나 다 추억이 있네요 놀이터도 그냥 길", "0409_2_vocals.0025.wav": "초등학교", "0409_2_vocals.0026.wav": "괜히 약간 울적해 줬어요", "0409_2_vocals.0027.wav": "왜요 왜 시작했어요", "0409_2_vocals.0028.wav": "응 좀 그런게 있죠 어릴 때 샀던데 가면은 괜히 뭔가 내가 너무 많이 변한 거 같고 나만 너무 좀 나빠진 거 같고", "0409_2_vocals.0029.wav": "너 문과를 소홀히 했다는 죄책감도 좀 들고", "0409_2_vocals.0030.wav": "괜히 그런 감정이 되는 거 같아요 아마 가장 말고 순수했던 시절이 남아 있는 곳이라서 그런 거 아닐까", "0409_2_vocals.0032.wav": "노래 뛰어 드리겠습니다", "0409_2_vocals.0033.wav": "한우정", "0409_2_vocals.0034.wav": "권우정 씨께서 신청하신 우쿨렐레 피크닉 좋아 좋아 6 김신자 시에 신청곡에요 서인국의 봄 타나", "0409_2_vocals.0035.wav": "불륜 가족들의 못 다한 이야기 만나 보고 있습니다", "0409_2_vocals.0036.wav": "박준홍 씨 가요 무대 요즘 자꾸만 졸음이 쏟아져요 여기서도", "0409_2_vocals.0037.wav": "저 소득", "0409_2_vocals.0038.wav": "유진은 졸릴 때 어떻게 하나요", "0409_2_vocals.0039.wav": "잠 깨는 방법 좀 공유해 주세요", "0409_2_vocals.0042.wav": "씻는다", "0409_2_vocals.0043.wav": "먹는다", "0409_2_vocals.0048.wav": "사진을 찍어 본", "0409_2_vocals.0049.wav": "셀카를 찍어 보는 거예요 진짜 졸릴 때", "0409_2_vocals.0050.wav": "와 진짜 충격적이다 내 얼굴", "0409_2_vocals.0051.wav": "일하면서", "0409_2_vocals.0052.wav": "장학회 할 겁니다", "0409_2_vocals.0055.wav": "0188 님 통역기를 하나 가지고 다닐까 봐요 저희 집 근처 외국인들이 가는데 길 물어볼때마다 무슨 소린지 몰라서 소리만 하고 많아요 유유", "0409_2_vocals.0058.wav": "길 물어보는 건 주는 아시나 봐", "0409_2_vocals.0059.wav": "나는 길이면 알려 줄 수 있잖아요 얼마든지", "0409_2_vocals.0060.wav": "그 정도는 뭐 이거 영어는", "0409_2_vocals.0061.wav": "용기가 중요한 거 같아", "0409_2_vocals.0062.wav": "불도 4532 님 언니", "0409_2_vocals.0063.wav": "최근에 새로운 업무를 배우게 됐는데 선배가 엉터리로 알려 주니까 답답하고 답답하네요", "0409_2_vocals.0064.wav": "어떻게 해야 하나 일단 토닥토닥 좀 해 주세요", "0409_2_vocals.0066.wav": "아 진짜 안 됐다 아이고 가르쳐 주는 사람이 엉망으로 가르쳐 주면 진짜 얼마나 답답한데 이거 그렇다고 선배니까", "0409_2_vocals.0067.wav": "아 뭔 무슨 말이에요 아 좀 제대로 말 좀 해 줘요라고 할 수도 없고", "0409_2_vocals.0068.wav": "아우", "0409_2_vocals.0069.wav": "어쩔 수 없어요 진짜 이거는", "0409_2_vocals.0070.wav": "꾹 참고", "0409_2_vocals.0071.wav": "살짝 다른 사람들한테 좀 물어 볼 수 있으면 좋은데", "0409_2_vocals.0072.wav": "하지 못하면", "0409_2_vocals.0074.wav": "뭐 열심히 한번 독학을 좀 해 보세요", "0409_2_vocals.0076.wav": "신지선 시가", "0409_2_vocals.0077.wav": "언니가 이번에 새로 산 제가 새로 산 원피스를 빌려 주면 망고를 사 준다고 해서 원피스를 빌려줬는데", "0409_2_vocals.0078.wav": "언니가 많이 망고맛 사탕을 사 주는 거 있지", "0409_2_vocals.0079.wav": "그 사탕 맛있게 먹기 냈는데", "0409_2_vocals.0080.wav": "다시는 옷 안 빌려 줄 거예요", "0409_2_vocals.0082.wav": "야 이거 뭐라고 할 거면 사탕을 먹으면 안 됐죠 지선 씨 그것도 유혹을 이기지 못했군요", "0409_2_vocals.0083.wav": "방탄의 언니가 이번엔 진짜 망고 사 주겠다고 그럼 또 빌려주게 될 걸", "0409_2_vocals.0084.wav": "아 내가 언니한테 속는 거는", "0409_2_vocals.0085.wav": "성인이 될 때까지입니다", "0409_2_vocals.0086.wav": "아 물론 언니도 섞어야 속고 속이는 관계", "0409_2_vocals.0087.wav": "노래 듣고 올게요 2374 님의 신청곡 비투비 봄날의 기억 그리고 6 god입니다", "0409_2_vocals.0088.wav": "보통리", "0409_3_vocals.0001.wav": "야 너 내가 사 준 거야 그거", "0409_3_vocals.0002.wav": "이런 말 안 말해 봤자 못 알아듣니", "0409_3_vocals.0003.wav": "괜히 사 줬어요 저 지금 후회하고 있거든요 그거다 내가 사준 건", "0409_3_vocals.0004.wav": "정신이 들고 좀 사 줘야 돼요 이래서 한 일곱 살 넘어", "0409_3_vocals.0005.wav": "고마움을 알고 누가 실제인지 파악할 수 있을까", "0409_3_vocals.0006.wav": "사 줘야 되는데", "0409_3_vocals.0008.wav": "저는 이모란 말도 아직 정확히 못 들었어 이거라고 해요 지금", "0409_3_vocals.0009.wav": "하도 주변에서 이모 이모 하니까 이모 왔다 이모야 이모야 해주니까", "0409_3_vocals.0011.wav": "누구야 누구야", "0409_3_vocals.0012.wav": "이거", "0409_3_vocals.0014.wav": "이걸 헤어져 부부 Bounce 더 자", "0409_3_vocals.0015.wav": "저희 아빠한테 듣고 싶은 말이에요 토요일에도 일요일에도 학교 가는 날처럼 아침 7시에 깨우세요 유유", "0409_3_vocals.0016.wav": "끔찍하다 진짜", "0409_3_vocals.0017.wav": "안 돼 안 돼 야 진짜 이거는 가족 회의를 한번 여세요", "0409_3_vocals.0018.wav": "토요일 일요일에는", "0409_3_vocals.0019.wav": "9시", "0409_3_vocals.0020.wav": "내가 문제 정해 줘 9시로 하세", "0409_3_vocals.0021.wav": "양건을 들고 가족 회의를 신청을 해 보십시오", "0409_3_vocals.0022.wav": "자 듣고 싶은 말 속에 되신 분들 선물 드릴 거고요 2분은 여기까지입니다 토요일 3분은 반납예정일 준비되어 있어요 노래 한 곡 듣고 첨부로 돌아오겠습니다", "0409_3_vocals.0023.wav": "김선영 씨 신청", "0409_3_vocals.0024.wav": "안녕하신가 영애 솜과사탕", "0409_3_vocals.0025.wav": "주방에서 뭐 하고 있나요", "0409_3_vocals.0029.wav": "KBS 쿨 FM 유인나의 볼륨을 높여요 3부 첫곡은 아울 시티의 굿 타임 드셨습니다 신청 해주셨어요 719 님한테는 스탠드 선물로 보내 드릴게요", "0409_3_vocals.0030.wav": "볼륨 가족들을 위한 선물 소개해 드리겠습니다", "0409_3_vocals.0031.wav": "헤어 기계 명가 유닉스전자에서 스타일", "0409_3_vocals.0032.wav": "한우 곱창 요리 전문점 매장 이용권", "0409_3_vocals.0033.wav": "블라인드 테스트 라벨영 화장품에서 무공패", "0409_3_vocals.0034.wav": "호텔에서 여성의류 쇼핑몰 이용권", "0409_3_vocals.0035.wav": "민감한 피부에 퍼퓸 오일 세트", "0409_3_vocals.0036.wav": "긴 생머리에 비밀 시크릿 길에서 소패스트샴푸", "0409_3_vocals.0037.wav": "아이파크에서 새우젓으로 갈 수련회", "0409_3_vocals.0038.wav": "화장품 전문 기업 유리 그래서 화장품 세트", "0409_3_vocals.0039.wav": "주식회사 브링크 10시에서 속눈썹증모 좋아 눈썹 세트", "0409_3_vocals.0040.wav": "밥미키 안경에서 밥미키 선글라스", "0409_3_vocals.0041.wav": "빙그레 꽃게랑 불짬뽕", "0409_3_vocals.0042.wav": "감성조명 원룸에서 인디언 스탠드", "0409_3_vocals.0043.wav": "완료 그대로 화장품에서 파운데이션", "0409_3_vocals.0044.wav": "여대생의 가방 맥끌라니에서 현재", "0409_3_vocals.0045.wav": "다양한 디자인 모모 케이스에서 나만의 핸드폰 케이스", "0409_3_vocals.0046.wav": "주식회사 사라젠에서 폴리스 6 중 스페셜 홈 케어 브랜드 세렌디피티에서 쇼핑몰 이용권", "0409_3_vocals.0047.wav": "내일 전문 브랜드 톡 젤에서 셀프 젤네일 패키지", "0409_3_vocals.0048.wav": "황정음의 영캐주얼 무자크에서 의류상품권", "0409_3_vocals.0049.wav": "주식회사 커피에 반하다 해서 커피교환권 스타일리쉬 SPA 브랜드 아클림 콜렉트 의류상품권", "0409_3_vocals.0050.wav": "천연 프리미엄 아이스크림 나뚜루팝에서 아이스크림 케이크", "0409_3_vocals.0051.wav": "후불제 여행 전문 기업 아멘 2에서 여행상품권을 드립니다", "0409_3_vocals.0053.wav": "그림 홈페이지 선곡표 확인해 주시고요 당첨되신 분들", "0409_3_vocals.0054.wav": "연락처 알려주세요", "0409_3_vocals.0055.wav": "볼륨을 높여요 토요일 3도는 반납 예정 1시간이 준비되어 있습니다 시작하기 전에 광고 듣고 올게요 잠깐만요", "0409_3_vocals.0056.wav": "잘못된 길이 지도를 만든다", "0409_3_vocals.0057.wav": "낸시 구절이 있어요", "0409_3_vocals.0059.wav": "우연한 만남 조금은 낯선 풍경도 나쁘지 않은 계절입니다", "0409_3_vocals.0061.wav": "내가 맛난 것들이 누군가에게는 삶의 지표가 되어 주기도 하고 마음에 안식처가 되어주고 찾아 줘", "0409_3_vocals.0062.wav": "나눠 주세요", "0409_3_vocals.0063.wav": "만나서 좋았던 반가웠다", "0409_3_vocals.0064.wav": "나쁘지 않았다", "0409_3_vocals.0066.wav": "우리 함께 해", "0409_3_vocals.0067.wav": "반납 예정일", "0409_3_vocals.0068.wav": "볼륨 가족들이 남겨 주신 좋은 구절들을 함께 만나보는 시간이죠 영화 만화 소피", "0409_3_vocals.0069.wav": "감동적인 이야기 나누는 시간을", "0409_3_vocals.0070.wav": "반납 예정", "0409_3_vocals.0071.wav": "김이 날씨가 이런 사람 주셨어요", "0409_3_vocals.0072.wav": "언니 반납 예정일에 제가 읽었던 책 나왔어요", "0409_3_vocals.0073.wav": "오 대박 신기해", "0409_3_vocals.0074.wav": "아 이거 너무 좋다 나미 짜리짜리", "0409_3_vocals.0075.wav": "언니가 읽어 주니까 더 좋아요", "0409_3_vocals.0076.wav": "아이고 고맙습니다 느낌표가 몇 개예요 이게 도대체 그래 오늘도 뭔가에 짜릿하게 통하는 그 기분 맛보는 분들 많았으면 좋겠네", "0409_3_vocals.0077.wav": "퇴근 반납예정일 오늘 첫 번째로 만나도 제품은요 김민희씨가 남겨 주었어요", "0409_3_vocals.0078.wav": "미움받을 용기", "0409_3_vocals.0080.wav": "함께 있으면", "0409_3_vocals.0081.wav": "왠지 숨이 막히고", "0409_3_vocals.0082.wav": "긴장으로 몸이 뻣뻣해지는 관계는", "0409_3_vocals.0083.wav": "연애 는 가능해", "0409_3_vocals.0084.wav": "사랑이라고는 할 수는 없니", "0409_3_vocals.0085.wav": "인간은", "0409_3_vocals.0086.wav": "이 사람과 함께 있으면", "0409_3_vocals.0087.wav": "자유롭게 행동할 수 있다", "0409_3_vocals.0088.wav": "넌 생각이 들었을 때", "0409_3_vocals.0089.wav": "사랑을 실감할 수 있는", "0409_3_vocals.0090.wav": "열등감을 느끼지도 않고", "0409_3_vocals.0091.wav": "우월함을 과시할 필요도 없네", "0409_3_vocals.0092.wav": "평온한", "0409_3_vocals.0093.wav": "지극히 자연스러운 상태라고 할 수 있지", "0409_3_vocals.0094.wav": "진정한 사랑이란 그런 걸세", "0409_3_vocals.0095.wav": "반면에", "0409_3_vocals.0096.wav": "구속이란", "0409_3_vocals.0097.wav": "상대를 지배하려는 마음의 표준이며", "0409_3_vocals.0098.wav": "글쓴이 바닥에 깔린 생각이기도 하지", "0409_3_vocals.0099.wav": "내게 불신감을 품은 상대와", "0409_3_vocals.0100.wav": "한 공간에 있으면", "0409_3_vocals.0101.wav": "자연스러운 상태로 있을 수 없겠지", "0409_3_vocals.0102.wav": "아들 너는 말했네", "0409_3_vocals.0103.wav": "함께 사이좋게 살고 싶다면", "0409_3_vocals.0104.wav": "설훈", "0409_3_vocals.0105.wav": "대등한 인격체로 대하지 않으면 안 된다", "0409_3_vocals.0109.wav": "싫은 책을 자주 안 봐서 베스트셀러 목록에만 의지하는데", "0409_3_vocals.0110.wav": "굉장히 오랫동안 베스트셀러에 올라와 있던 책이라 궁금했어요", "0409_3_vocals.0111.wav": "한 철학자와 청년의 대화체로 이루어진 책입니다", "0409_3_vocals.0112.wav": "철학자는 계속해서", "0409_3_vocals.0113.wav": "알프레드 아들러라는 심리학자 이론을 설명하는데", "0409_3_vocals.0114.wav": "성령과 함께 수업을 받는 기분이 들어요", "0409_3_vocals.0115.wav": "사랑해는 긴장감이 필요하다고 생각해서", "0409_3_vocals.0116.wav": "말을 돌리게 하는 사람이 나타나지 않는다고", "0409_3_vocals.0117.wav": "박 2년 했는데", "0409_3_vocals.0119.wav": "이 대목을 읽고 마음을 고쳐 먹기로 했어요", "0409_3_vocals.0120.wav": "상대방 앞에서도 그냥", "0409_3_vocals.0121.wav": "나 자신으로 있을 수 있는 사람을 찾아야겠다", "0409_3_vocals.0123.wav": "나타날지는 모르겠지만요", "0409_3_vocals.0125.wav": "아무튼 보고 나면 뭔가 느끼는게 많은 책입니다", "0409_3_vocals.0126.wav": "미움받을 용기", "0409_3_vocals.0127.wav": "김민희씨가 남겨주신 미움받을 용기이어서 듣고 신곡", "0409_3_vocals.0128.wav": "브라운 아이드 소울의 아름다운 날들이었습니다", "0409_3_vocals.0130.wav": "와", "0409_3_vocals.0131.wav": "너무 좋다 감동 받았어", "0409_3_vocals.0132.wav": "정말 공감했고", "0409_3_vocals.0133.wav": "사랑은 참 아무리 해도", "0409_3_vocals.0134.wav": "사랑이 뭔지 잘 모르겠잖아요 어떤 사람이 좋은 사람인지 좀 헤매게 되고", "0409_3_vocals.0135.wav": "정말 맞는 거 같아", "0409_3_vocals.0138.wav": "받을 용기", "0409_3_vocals.0139.wav": "이게 이렇게 좋은 책이 없군요 저희 집 책장에 그냥 고쳐져 있으니 참 오래 됐거든요 미움받을 용기내는 기시미 이치로라는 철학자와 고가 후미타케라는 프리랜서 작가가 함께 쓴 책", "0409_3_vocals.0140.wav": "미니 시간 있으면 내려 아주 오랜 시간 그 셀러에 올랐던 책이고", "0409_4_vocals.0001.wav": "김민희씨가 남겨주신 미움받을 용기이어서 듣고 신곡", "0409_4_vocals.0002.wav": "브라운 아이드 소울의 아름다운 날들이었습니다", "0409_4_vocals.0004.wav": "와", "0409_4_vocals.0005.wav": "너무 좋다 감동 받았어", "0409_4_vocals.0006.wav": "정말 공감했고", "0409_4_vocals.0007.wav": "사랑은 참 아무리 해도", "0409_4_vocals.0008.wav": "사랑이 뭔지 잘 모르겠잖아요 어떤 사람이 좋은 사람인지 좀 헤매게 되고", "0409_4_vocals.0009.wav": "정말 맞는 거 같아", "0409_4_vocals.0012.wav": "받을 용기", "0409_4_vocals.0013.wav": "이게 이렇게 좋은 책이 없군요 저희 집 책장에 그냥 고쳐져 있으니 참 오래 됐거든요 미움받을 용기는 기시미 이치로라는 철학자와 고가 후미타케라는 프리랜서 작가가 함께 쓴 책 이래", "0409_4_vocals.0014.wav": "민희 씨 말씀 대로 아주 오랜 시간 베스트셀러에 올랐던 책이고", "0409_4_vocals.0015.wav": "심리학자 알프레드 아들러 이론을 이야기하고 있는 색이라고 합니다", "0409_4_vocals.0016.wav": "사람과의 관계 열쇠는 결국 마음가짐이 잖아요", "0409_4_vocals.0017.wav": "알면서도 잘 풀리지 않는 숙제지만", "0409_4_vocals.0018.wav": "그 마음 가지면 어떻게 먹어야 하는가에 대해 알기 쉽게", "0409_4_vocals.0019.wav": "굉장히 설득력있게 당겨 있대요", "0409_4_vocals.0020.wav": "미니 시간 날 때 주신 대목 보면서도 막 어 맞아 맞아 그러면서 왔는데 참이 말 좋네요", "0409_4_vocals.0022.wav": "함께 사이좋게 살고 싶다면 서로를 대등한 인격체로 대하지 않으면 안 된다", "0409_4_vocals.0025.wav": "어디서도 작아지지 말고 또 누구에게도 커지지 말고 서로가 똑같은 크기로 대등하게 마음 주고 받으면 참 좋겠다라는 생각 들었어요", "0409_4_vocals.0026.wav": "반납 고맙습니다 미니 씨", "0409_4_vocals.0027.wav": "자 그럼 두번째 작품도 만나보죠", "0409_4_vocals.0028.wav": "지현주 씨가 남겨주셨어요", "0409_4_vocals.0029.wav": "백석", "0409_4_vocals.0030.wav": "내가 생각하는 것은", "0409_4_vocals.0031.wav": "박근", "0409_4_vocals.0032.wav": "범철 날 따지기에 느긋한이 푸석한 밤이다", "0409_4_vocals.0033.wav": "거리에는 사람도 많이 나서", "0409_4_vocals.0034.wav": "항상 음성 할 것이다", "0409_4_vocals.0035.wav": "어쩐지이 사람들과", "0409_4_vocals.0036.wav": "신한 인싸 다니고 싶은 밤이다", "0409_4_vocals.0037.wav": "그렇구나", "0409_4_vocals.0038.wav": "나는", "0409_4_vocals.0039.wav": "하얀 자리 위해서", "0409_4_vocals.0040.wav": "마른 팔뚝", "0409_4_vocals.0041.wav": "새파란 핏대를 바라보며", "0409_4_vocals.0042.wav": "나는 가난한 아버지를 가진 것과", "0409_4_vocals.0043.wav": "내가 오래 그려 오든 처녀가 시집을 가는 것과", "0409_4_vocals.0044.wav": "그렇게도 탈출 하든 동무가", "0409_4_vocals.0045.wav": "나를 버린 일을 생각한다", "0409_4_vocals.0046.wav": "너 내가 아는", "0409_4_vocals.0047.wav": "금호 미성 하고 돈도 있는 사람들이", "0409_4_vocals.0048.wav": "즐거이 술을 먹으려 다닐 것과", "0409_4_vocals.0049.wav": "내 손에는", "0409_4_vocals.0050.wav": "진단서 하나도 없는 것과", "0409_4_vocals.0051.wav": "그리고 그", "0409_4_vocals.0052.wav": "아들아 세상 살아 도드리", "0409_4_vocals.0053.wav": "유 전기도 없는 것을 생각한다", "0409_4_vocals.0054.wav": "그리고", "0409_4_vocals.0055.wav": "이런 생각이", "0409_4_vocals.0056.wav": "내 눈 깔아", "0409_4_vocals.0057.wav": "내 가슴 가루", "0409_4_vocals.0058.wav": "뜨겁게 하는 것도 생각한다", "0409_4_vocals.0060.wav": "취업 준비생입니다", "0409_4_vocals.0061.wav": "아 날은 추운데 나는 뭐 하나 싶은 요즘", "0409_4_vocals.0062.wav": "집에서 뒹굴거리다가 발견 1시 집에 있던 시에요", "0409_4_vocals.0063.wav": "시 읽으면서 어머 나 보는 중", "0409_4_vocals.0064.wav": "했던 건 처음이네", "0409_4_vocals.0065.wav": "나만 빼고 모두가 행복한 것 같은 봄날이라", "0409_4_vocals.0066.wav": "저는 왠지 서글픈 기분이", "0409_4_vocals.0067.wav": "잔뜩 드는 볼륨 가족들도 있을 것 같아서요", "0409_4_vocals.0068.wav": "함께 나누고 싶어졌어요", "0409_4_vocals.0069.wav": "백석", "0409_4_vocals.0070.wav": "내가 생각하는 것은", "0409_4_vocals.0071.wav": "지현주 씨가 남겨 주신 백석에 내가 생각하는 것은 여자들 것인고 근 뉴욕 물고기에 2 - 왔습니다", "0409_4_vocals.0073.wav": "백석은", "0409_4_vocals.0074.wav": "굉장히 유명하죠 교과서에도 나오는 분 있잖아요", "0409_4_vocals.0076.wav": "시험 문제 나왔었나", "0409_4_vocals.0077.wav": "문제집 회사도 봤던 거 같기도 하고", "0409_4_vocals.0078.wav": "잘 생긴 걸로 유명도 하죠", "0409_4_vocals.0080.wav": "순우리말을 사랑은 시인으로 평가 되기도 하고 모더니즘을 발전시킨 신으로 평가 되기도 하는 시간이라고 합니다", "0409_4_vocals.0081.wav": "어 이게 뭔가 옛날", "0409_4_vocals.0082.wav": "예 말이라고 하나", "0409_4_vocals.0083.wav": "지금 일하는 살짝 다른 말 줄 싫어서", "0409_4_vocals.0084.wav": "사실 읽으면서 굉장히 긴장했어요", "0409_4_vocals.0085.wav": "어 이게 틀릴까봐", "0409_4_vocals.0086.wav": "감사하네요 이렇게 뭔가 좀 멋진 시를 반납해 준 현주 씨한테 고맙고요 어 여기 우리 작가가 저 참고하라고 써 주는게 있어요 여러분도 궁금하실까봐 축구 읽어 줄게요 시에서", "0409_4_vocals.0087.wav": "우리가 모를만한 표현들이 좀 있었잖아요 가끔 봄철 날 따지기에 누구 다니 박석한 밤이다 따지기가", "0409_4_vocals.0088.wav": "나직이 이른 봄 얼었던 흙이 불리려는 무렵 1회", "0409_4_vocals.0089.wav": "누구 다니는 여주에 있는 사 다니고에게 사다리꼴의", "0409_4_vocals.0090.wav": "갑니다가", "0409_4_vocals.0091.wav": "아니 나 지금에 다니다 연말이라", "0409_4_vocals.0092.wav": "너 이렇게 싸 다니니 참 어려운 발음 이었네", "0409_4_vocals.0093.wav": "너도 살 틀 하든", "0409_4_vocals.0094.wav": "너무나 다정스러운 허물없이 위해 주고 보살펴 주던지 금액은 살뜰하다", "0409_4_vocals.0095.wav": "요 건가 봐요", "0409_4_vocals.0096.wav": "류성기 거는 레코드플레이어 옛날 유성기구", "0409_4_vocals.0097.wav": "이러니까 진짜 우리 엄청 막 교육방송 갖다 열어 놓은 자는 거 아니죠 여러분", "0409_4_vocals.0098.wav": "암튼 이게 중 이런 말들 때문인지 뭔가 좀 영화같아요 그림도 그래 가지고요 방 안에 추리닝 입고 누워 가지고 헤어진 애인 생각하다가 날 버린 친구들 떠올렸다가 노래나 들을까 했는데 뭐들을 것도 없고 그래서 당신을", "0409_4_vocals.0099.wav": "영화가 아니라 오늘 우리 볼은 가족들을 볼 수 있나요", "0409_4_vocals.0100.wav": "그랬다면 잘 하셨어요 오늘 하루 되게 실적으로 보내셨네", "0409_4_vocals.0102.wav": "자 좋은 글 남겨 주신 현주 씨 고마워요 오늘 반납해 주신 두 분께 선물 보내 드릴게요 영화소설 CSM 안 와 모든 좋습니다 유인나의 볼륨을 높여요 홈페이지 반납예정일 게시판 이용하시면 됩니다", "0409_4_vocals.0103.wav": "광고 제거하지 여러분 사람 만나 볼게요 잠깐만", "0409_4_vocals.0104.wav": "너의 목소리", "0409_4_vocals.0105.wav": "너의", "0409_4_vocals.0106.wav": "것 같아", "0409_4_vocals.0108.wav": "같아", "0409_4_vocals.0109.wav": "유인나의 볼륨을 높여", "0409_4_vocals.0110.wav": "KBS 쿨 FM University 사연 만나 볼게요", "0409_4_vocals.0111.wav": "0586", "0409_4_vocals.0112.wav": "UD 재생 129 다가 오거든요", "0409_4_vocals.0113.wav": "여자 친구가", "0409_4_vocals.0114.wav": "가지고 싶은 거 있어 하길래", "0409_4_vocals.0115.wav": "없다고 했더니 응 알겠어", "0409_4_vocals.0116.wav": "한 겁니다 유유 아니 두 번은 물어 봐야 하는 거 아닌가요 유유", "0409_4_vocals.0117.wav": "아이 그래도 이거", "0409_4_vocals.0118.wav": "잘 하셨어요 가지고 싶은 거 있어 나 요즘에", "0409_4_vocals.0119.wav": "갖고 싶었어 이렇게 얘기하는 것도 좀 너무 좀 괜찮아요 속 보이고", "0409_4_vocals.0120.wav": "생일 선물은 그게 좋은 거 같아요 그 내가 바라는 걸 못 받는 것보다 물론 친구들끼리 모여서 실용적인 거 선물해 주는 것도 아주 좋은 방법이고 그게 아닌 뭐 그냥 연인들끼리라며", "0409_4_vocals.0121.wav": "이 사람이 내 생일", "0409_4_vocals.0122.wav": "날 위해서 뭘 준비를 해 줄까", "0409_4_vocals.0123.wav": "나한테 뭐가 필요하다고 생각을 했을까", "0409_4_vocals.0124.wav": "이런 교회에서 좀 받아 보는 거 아주 재밌는 거 같아", "0409_4_vocals.0125.wav": "남자친구의 취향도 알아 볼 수가 있고", "0409_4_vocals.0126.wav": "여자 친구가 뭐 선물 해주는지 한번 잘 봐 봐요", "0409_4_vocals.0127.wav": "안경 모자 이런 거 사 주는 거면 얼굴 좀 가려고 다니란 뜻이구나 이런 거 좀 파악할 수가 있잖아요", "0409_4_vocals.0129.wav": "어도 하루에 혹시 아빠한테 잡혀서 배드민턴 캐치볼 탁구 헬스를 하루에 다 했어", "0409_4_vocals.0130.wav": "그랬더니 몸이 제 말을 안 들어요", "0409_4_vocals.0131.wav": "저희 아빠 왜 적당히 하는 걸 모르시는 걸까요", "0409_4_vocals.0132.wav": "다음엔 도망가야 겠어요 유유", "0409_4_vocals.0133.wav": "아", "0409_4_vocals.0134.wav": "이철혁 짱이다 체력장 다음에 체력장 하고 며칠 동안 알잖아요", "0409_4_vocals.0135.wav": "평소에 운동을 해야 되는데", "0409_4_vocals.0136.wav": "둘 중에 하나 내 18시가 좀 평소에 운동을 하시던가", "0409_4_vocals.0137.wav": "아빠가 좀", "0409_4_vocals.0138.wav": "고루고루 일주일에 한두 번", "0409_4_vocals.0139.wav": "청도 좀 강조를 다쳐서 운동을 계속 시키던가", "0409_5_vocals.0000.wav": "헬스장 가지 마시고요 또 헬스장을 도와준다고 거기 돈 갖다 내고 안 가지 마시고요", "0409_5_vocals.0001.wav": "어떠신지요 씨 화장실에서 미끄러졌어요 깨어나보니 화장실에 제가 널부러져 있더라구요 우와 살다 살다 계절이라는 걸 처음 해 봤네", "0409_5_vocals.0002.wav": "다친 데는 없고요 살아있으면 감사했습니다", "0409_5_vocals.0003.wav": "볼륨 가족들 조심해요", "0409_5_vocals.0004.wav": "그래 아저씨 이후의 삶은 덤으로 얻은 삶이다 생각하시고 좋은 일 많이 하시고 당장에 하고 싶은 일 많이 하시고요", "0409_5_vocals.0005.wav": "조금 더 좋지만 저금하기 보다는 오늘 내가 먹고 싶은 거 드시고요 갖고 싶은 것도 어느 정도 사시고요", "0409_5_vocals.0006.wav": "그렇게 사세요", "0409_5_vocals.0007.wav": "YT", "0409_5_vocals.0008.wav": "소변 경순 씨", "0409_5_vocals.0009.wav": "옛날 씨는 겨울 옷 넣어두고 여름 옷 꺼내 졌나요", "0409_5_vocals.0010.wav": "겨울 옷 정리하면서 방송 듣고 있는데 먼지 때문에 콧물이 자꾸만 나와요", "0409_5_vocals.0011.wav": "경순 씨 마스크 하세요 저는 옷 정리할 때 꼭 마스크에요", "0409_5_vocals.0012.wav": "처음부터는 잘 안 한다 또이 생각이 안나요 그래서 처음에는 하다가", "0409_5_vocals.0013.wav": "10분 20분 지나면", "0409_5_vocals.0014.wav": "아니나 다를까", "0409_5_vocals.0015.wav": "에취 에취 재채기하고 콧물 줄줄 나면서 슬슬 눈이 부어오르면서 머리가 아파지기 시작하거든요 그때 급하게 마스크를 꺼내는데", "0409_5_vocals.0016.wav": "확실히 마스크 하고 정리 하면은 훨씬 괜찮습니다", "0409_5_vocals.0017.wav": "자 그럼 노래 띄워 드릴게요 3414 님의 신청곡", "0409_5_vocals.0018.wav": "버벌진트 굿모닝 6a 브레이크입니다", "0409_5_vocals.0019.wav": "KBS 쿨 FM 유인나의 볼륨을 높여요 불량 가족들의 사연 만나 보겠습니다", "0409_5_vocals.0020.wav": "각종서식 13년지기 친구의 여동생한테 과외를 해주고 있는데요 무표정으로 채점하고 감시하면서", "0409_5_vocals.0021.wav": "이어폰으로는 볼륨 듣고 있어", "0409_5_vocals.0022.wav": "창민아 미안해", "0409_5_vocals.0025.wav": "아중사 씨", "0409_5_vocals.0026.wav": "책정 하다", "0409_5_vocals.0028.wav": "어째 빵 터지거나", "0409_5_vocals.0029.wav": "수정하다가 뭐 안타까워 하거나", "0409_5_vocals.0030.wav": "이런 리액션 절로 나오는 거 아니에요", "0409_5_vocals.0031.wav": "아 과외 할 때는 잠시 꺼 두셔도 됩니다 정섭 씨 너무 고맙긴 해요", "0409_5_vocals.0032.wav": "이륙 이웃님 UD 제가 중 이때 약도 없다는 중 2병에 걸리면서 일본 로맨스 학원드라마 엄청 빠지면서 방송을 하고 살았어요", "0409_5_vocals.0034.wav": "자기 자신을 잘 파악하고 계신 분이네요 근데 구이가 된 지금 잠시 그런 드라마를 봤는데 주체를 못 하겠습니다 유유 어떡하죠 나 시험 기간인데 또 망상하고 살면 이제 진짜 안 되는데", "0409_5_vocals.0035.wav": "근데 시험기간만 되면 하고 싶은 일이 더 많아지는 걸까요 왜", "0409_5_vocals.0038.wav": "어 다 그래요 저기 막 드라마 촬영할 때 하고 싶은 일이 너무 많아서 제 핸드폰 메모장에 보면 오늘", "0409_5_vocals.0039.wav": "드라마 끝난 뒤 할 일 해서 온갖 거 다 적어 놨어요", "0409_5_vocals.0040.wav": "하나도 못 해요 하나 더 끝나고 나면은 우리가 하기가 싫어 그리고 그 메모장 보면 쓸데없는 거 보다 적어 놨어", "0409_5_vocals.0041.wav": "오늘 그 구체적 으로 적어 놨다 고 하면서 그때 이게 그렇게 하고 싶어", "0409_5_vocals.0042.wav": "억지로 뭐 하나씩 하나씩 좀 해 보려고 하고 있긴 한데", "0409_5_vocals.0043.wav": "오늘 할 일 있을 때", "0409_5_vocals.0044.wav": "하고 싶은게 많아지는 거예요 근데 일요일에 언니 진짜 참으셔야 돼요 지금 그런 뭐", "0409_5_vocals.0045.wav": "드라마 보다가 인생 마음 싶어", "0409_5_vocals.0047.wav": "뻥 갑자기 너무 죄송 세게 말했죠", "0409_5_vocals.0048.wav": "미안해요 하지만 지금 공부 할 때입니다", "0409_5_vocals.0050.wav": "너 추 이미지 저요", "0409_5_vocals.0052.wav": "초보 농부가 되어 열심히 수확해 보겠습니다", "0409_5_vocals.0053.wav": "우와 축하드려요", "0409_5_vocals.0054.wav": "나도 이거 이런 거 있는 줄 알았으면 나도 응모 좀 해 볼 걸", "0409_5_vocals.0056.wav": "오늘 동네인가요 이제 터 필요하거든요 상추랑 깻잎을 좀", "0409_5_vocals.0057.wav": "제가 유기농으로 나가 심어서 좀 키워 보고 싶어요", "0409_5_vocals.0058.wav": "그냥 해 본 씨 몸 건강하게 많이 키워서 그거 많이 드시고 건강하세요", "0409_5_vocals.0060.wav": "야 그럼 노래 띄워 드릴게요 톡으로 삼형제 아빠 님께서 신청해 주신 제이레빗 선잠 6인실 신청곡", "0409_5_vocals.0061.wav": "스탠딩 에그 리틀 스타", "0409_5_vocals.0062.wav": "사랑스런 나의", "0409_5_vocals.0069.wav": "유인나", "0409_5_vocals.0070.wav": "볼륨을 높여요", "0409_5_vocals.0071.wav": "자 그럼 우리는 끝까지 행복해야지", "0409_5_vocals.0072.wav": "주말마다 동네 도서관에가 보네요", "0409_5_vocals.0073.wav": "클럽 작은 지역에선 아저씨께서 핫도그를 팔고 계십니다", "0409_5_vocals.0074.wav": "그런데 얼마 전부터 장사를 하지 않으시더라구요", "0409_5_vocals.0075.wav": "그 자리엔 다리 부상으로 휴업합니다 곧 돌아올게요", "0409_5_vocals.0076.wav": "라는 문구만 붙어 있고요 그런가 보다 하고 넘어갔는데요", "0409_5_vocals.0077.wav": "휴업 안내문에 낙서가 가득하더라구요", "0409_5_vocals.0078.wav": "아저씨 얼른 나와서 빨리 돌아오세요", "0409_5_vocals.0079.wav": "벌써부터 핫도그가 먹고 싶어요 언제 오시나요", "0409_5_vocals.0080.wav": "날아라 획득을 등등등 아저씨가 만들어 준 핫도그가 도서관을 찾는 사람들의 마음까지 채워주던 것 같습니다", "0409_5_vocals.0081.wav": "우리를 위로하는 건 대단한 무언가가 아니라", "0409_5_vocals.0082.wav": "이렇게 사소한 마음의 소통이라는 사실에 뭉클해지는 하루였어요", "0409_5_vocals.0084.wav": "하트가 아저씨가 그거 빨리 봤으면 좋겠다 그거 보면 좀 더 빨리 나올 수 있지 않을까요", "0409_5_vocals.0085.wav": "나 투데이 오늘은 박윤정 씨께서 보내주셨습니다 선물 보내 드릴게요", "0409_5_vocals.0086.wav": "유인나의 볼륨을 높여요 오늘은 여기까지입니다 끝 곡으로 아일랜드의 사랑은 너라서 들으면서 저는 인사 드릴게요", "0409_5_vocals.0087.wav": "불륜가족", "0409_5_vocals.0088.wav": "우리는", "0501_1.0000.wav": "어른이 된 걸까요", "0501_1.0001.wav": "지워 드렸습니다", "0501_1.0004.wav": "이 아름다운 5월의 첫날을 여러분들과 함께 하게 돼서", "0501_1.0005.wav": "정말 정말 너무너무 기뻐요", "0501_1.0006.wav": "안녕하세요 저는", "0501_1.0007.wav": "유디 꿀돼지 유 딸기 유치 자유투로 시카 부쳐 주신 별명이 진짜 많아요 별명부자", "0501_1.0008.wav": "유인나입니다", "0501_1.0010.wav": "오늘 정식 공휴일은 아닌데요이 근로자의 날이 어서", "0501_1.0011.wav": "많은 분들이 아마 씻었을 거 같아요", "0501_1.0012.wav": "오면서 보니까", "0501_1.0013.wav": "한강에도 여의도공원에서", "0501_1.0014.wav": "다른 날 보다 사람들이 좀 많더라고요", "0501_1.0016.wav": "여러분들은 뭐 하셨나요", "0501_1.0018.wav": "우지 날씨가", "0501_1.0019.wav": "일이 오늘이 증상인지 아닌지만 알려 주세요 알려 주세요 하셨는데", "0501_1.0020.wav": "글쎄", "0501_1.0021.wav": "뭘 거 같아", "0501_1.0022.wav": "이따 이따가 알려 드릴게요", "0501_1.0024.wav": "가지 마시고 g 날씨", "0501_1.0025.wav": "어 제가 이따가 생방인지 아닌지 말씀 드릴 때까지 꼭 함께 해 주셔야 됩니다", "0501_1.0026.wav": "주혜정 씨", "0501_1.0027.wav": "어제 푹 쉬었어요", "0501_1.0028.wav": "네 저 푹 쉬었어요", "0501_1.0029.wav": "너무 꿀 같아요 출근길도 끓이고", "0501_1.0030.wav": "막 퇴근 하는 것도 너무 뿌리고", "0501_1.0033.wav": "행복해 요즘에 이거", "0501_1.0034.wav": "오늘 벌써 3일째 내요", "0501_1.0036.wav": "왜 이렇게 3일이 빨리 가냐 짚었다 가도 그래도 내일이 또 남았다라는 생각이 기쁘기도", "0501_1.0038.wav": "정은 씨가 설정을 시도 오랜만이에요", "0501_1.0039.wav": "오늘 제게 뭐래 존나 생각하니", "0501_1.0040.wav": "부추전 이요 잘 하셨어요 부추전", "0501_1.0041.wav": "전 요즘에 감자전이 그렇게 좋더라고요 옛날엔 파전을 좋아했는데", "0501_1.0042.wav": "요즘에 감자전이 그렇게 좋아", "0501_1.0043.wav": "봤어", "0501_1.0044.wav": "이시원 씨 어른답게 맥주 한 캔 사왔어요 들으면서 마시라고요", "0501_1.0045.wav": "주량이 어떻게 되세요", "0501_1.0046.wav": "맥주 한 캔 사 왔는데 거기 막 만 치자가 생기는 거 아니겠지", "0501_1.0047.wav": "김성연 씨가 그래서 저는 어른이 된 뒤로 계속 탄산음료를 마시나 봐", "0501_1.0048.wav": "학생 때는 부모님께서 못 마시게 하셨거든요", "0501_1.0049.wav": "아 이거 근데 저 진짜 오프닝 글 읽으면서", "0501_1.0050.wav": "어 그런가", "0501_1.0051.wav": "아 나도 그런 거 생각해 봤는데 진짜 그런 거 같아", "0501_1.0053.wav": "초등학생도 있잖아요 그때 책을 진짜 좋아했어요 책 읽는 걸 근데", "0501_1.0054.wav": "어제가", "0501_1.0055.wav": "시험을 봐서", "0501_1.0056.wav": "1등 2등 이런 거 말고 올백 올백을 맞아야만 엄마가 책을 세 권을 사 주시는 거야", "0501_1.0057.wav": "얼마나 귀한 책이에요 그게", "0501_1.0058.wav": "그래서 진짜 책 3권", "0501_1.0059.wav": "받는게 어려운 일이 없어 가지고", "0501_1.0060.wav": "다커", "0501_1.0061.wav": "정말", "0501_1.0062.wav": "이거 너무했다 싶을 정도로 책을 사야지", "0501_1.0064.wav": "아닐 거 그거 다 읽으려면 나 집에서 나올 수 없어 책을 읽어야 돼요", "0501_1.0065.wav": "근데 꼭 그렇게 책을 사고", "0501_1.0066.wav": "옷을 그렇게 안 사 주셔서 옷을 한참 한동안 정말", "0501_1.0067.wav": "많이 사 먹어 그랬거든요", "0501_1.0069.wav": "일리가 있는 말 같아", "0501_1.0071.wav": "떠들다 보면", "0501_1.0072.wav": "다음 코너가 있다는 걸 하나 주세요 이렇게", "0501_1.0073.wav": "이렇게 된다", "0501_1.0074.wav": "여러분들이 중간 중간 저한테 문자 좀 보내주세요 정신차리라", "0501_1.0075.wav": "어 또 많은 분들이 보라 기대했는데 아니라고", "0501_1.0076.wav": "1시 뭐 하시는데", "0501_1.0077.wav": "아쉬워 하지 마세요", "0501_1.0078.wav": "듣는 라디오를 함께 해요 그리고", "0501_1.0079.wav": "3부 정도에는 보라로 같이 할까요", "0501_1.0080.wav": "반응", "0501_1.0081.wav": "반은", "0501_1.0083.wav": "그냥 하고 반은 뭐라 하고 아닌가 그게 기술적으로 조금 어려운 일이에요 아미리 미리 그게 돼야 되는 거구나", "0501_1.0085.wav": "아 중간에 되는게 아니구나 나는 내 핸드폰 카메라 12시 그런 건 줄 알았지 그렇군요 여러분 그럼 오늘은 제가 특별히 더 여러분들 길음동에 시켜 드리도록", "0501_1.0086.wav": "TV 보도록 하겠습니다 드릴 듣는 라디오로 함께 해", "0501_1.0088.wav": "그리고", "0501_1.0090.wav": "아 또 또 많은 분들이 제로 오늘 비가 오면 하고 싶은게 있다 소곤소곤 귓속말 해서", "0501_1.0091.wav": "비가 안 온다", "0501_1.0092.wav": "좋아지는데", "0501_1.0094.wav": "비오는", "0501_1.0096.wav": "비 오잖아 여러분 그죠", "0501_1.0097.wav": "착한 사람 눈에만 보이는 비가 와요", "0501_1.0098.wav": "비가 오니까 이따가", "0501_1.0099.wav": "하고 싶은 걸로 하도록 할게요", "0501_1.0100.wav": "수요일은 tmi 퀴즈가 있습니다", "0501_1.0101.wav": "tmi 란 말 아시죠 설마 이거 이거 모르지 않겠죠 저도 하는", "0501_1.0102.wav": "투머치인포메이션", "0501_1.0103.wav": "요즘 어디 가서 자기 이야기 하면 엄마 tmi 라면 서면 밥 주잖아", "0501_1.0104.wav": "볼륨에서는 tmi 관심가져", "0501_1.0105.wav": "심지어 다른 사람 이 나의 tmi 궁금해 하면서 맞추려고 한다", "0501_1.0107.wav": "어떤 거는인지 저도 조금 궁금하네", "0501_1.0108.wav": "얘기하다 보면", "0501_1.0109.wav": "참여하는 사람이 입장에서는 누가 왜 이런 걸 맞추고 있지 싶다는데", "0501_1.0110.wav": "상금을 들으면서 모두가 훈훈하게 마무리하는 그럼 따뜻한 코너라고", "0501_1.0111.wav": "이따가 정자 보여서 오승원 아나운서랑 같이 해 볼게요", "0501_1.0112.wav": "신청", "0501_1.0113.wav": "소소한 일상 이야기로 저한테 말 많이 많이 걸어 주시고요", "0501_1.0114.wav": "오늘 자랑할", "0501_1.0115.wav": "웃었더니", "0501_1.0116.wav": "위로가 필요한 일 다 좋습니다", "0501_1.0117.wav": "문자 #8910 단문 50원 작문과 사진은 105", "0501_1.0118.wav": "카톡 마이크에는 무료입니다 많이 보내주세요", "0501_2.0000.wav": "1시", "0501_2.0001.wav": "생방송이에요", "0501_2.0002.wav": "아니", "0501_2.0003.wav": "집중 안 할 거예요 월요일 날 분명히 월 화 수 목은 생방송이다라고 말씀을 드렸는데", "0501_2.0004.wav": "아마 월화 중에는 못 들었나 봐", "0501_2.0005.wav": "10시가 누나", "0501_2.0006.wav": "누나 오늘 오늘 대본보고 tmi 배운 거 맞죠 하셨는", "0501_2.0008.wav": "뭐야 오늘 알았어요", "0501_2.0010.wav": "아 저는 사실 안 찍으면 안 됐어요 작년에", "0501_2.0011.wav": "그 때 거기서 배웠어요 이런 날 있다는", "0501_2.0012.wav": "백윤식", "0501_2.0013.wav": "비 지금 걸 호우특보처럼 내리고 있어", "0501_2.0014.wav": "고양이야 멍멍해봐 아니면 내 마음에 비가 와요", "0501_2.0015.wav": "뭐지 마음에 비가 와요 무슨 일 있어요", "0501_2.0017.wav": "고양이야 멍멍해봐 님 혹시 무슨 일이 있는 거라면 무슨 일이 있다", "0501_2.0018.wav": "무슨 일인지도 보내주세요", "0501_2.0019.wav": "문상준 씨가 비가 아니라", "0501_2.0020.wav": "내 눈에서 물이 흐르는 겁니다 여러분 눈물 닦아", "0501_2.0021.wav": "왜 왜 왜", "0501_2.0022.wav": "왜요 삼촌 씨 무슨 일 있으세요", "0501_2.0023.wav": "한예진 씨가 선우정아의 비 온다 불러 줘요 맨날 불렀잖아요", "0501_2.0024.wav": "와 진짜 기억력", "0501_2.0025.wav": "장난 아니다 진짜로", "0501_2.0026.wav": "맞아 내가 그랬지 내가이 노래를 비가 오는 날마다 불렀잖아요", "0501_2.0027.wav": "비 온다", "0501_2.0028.wav": "비 온다", "0501_2.0031.wav": "리틀미미", "0501_2.0032.wav": "제가 보기에 굉장히 재빠른 분은 아니거든요 며칠 보니까", "0501_2.0035.wav": "근데 왜 그럴까 만큼은 정말 재빠르게 최고인 거 같아", "0501_2.0036.wav": "125 님 오늘 날 tmi 원래 밀크티 안 좋아하는데 엄마랑 같이 먹으려고 펄들어간 밀크티 도전해봤다요", "0501_2.0037.wav": "아주 좋아요 밀크티", "0501_2.0038.wav": "이거 왠지", "0501_2.0039.wav": "먹을 때마다 죄책감 들지 않아요 너무 달 잖아요 이거", "0501_2.0040.wav": "달리기를 조절할 수가 있다", "0501_2.0041.wav": "저는", "0501_2.0042.wav": "아주 많이 루고 달걀", "0501_2.0043.wav": "아마 니로 달걀로 딸기로 마니로", "0501_2.0044.wav": "아주머니 달걀로 먹는데", "0501_2.0045.wav": "그렇지 않으면 이거 먹는 의미가 없다라는 생각이 들고", "0501_2.0046.wav": "너무 달아서 쭉", "0501_2.0047.wav": "그렇다고", "0501_2.0048.wav": "이상 오산 님 미루고 미루던 겨울 패딩 세탁 했어요 이제서야 숙제를 끝낼게", "0501_2.0049.wav": "사랑합니다", "0501_2.0050.wav": "잘 하셨어요", "0501_2.0051.wav": "응", "0501_2.0052.wav": "미루던 일을 하기에 좋은 계절인 거 같아요 너무 덥지도 너무 춥지", "0501_2.0053.wav": "나는 기다리니까", "0501_2.0054.wav": "뭐래 나 4시 깨겠다 뭔지 아냐", "0501_2.0055.wav": "이거", "0501_2.0056.wav": "쪼꼬 잖아요", "0501_2.0057.wav": "낄 때 끼고 빠질 때 빠져라", "0501_2.0058.wav": "봤지", "0501_2.0059.wav": "설마", "0501_2.0063.wav": "tmt 박하연 지금 언니 tmt 알아요 궁금증", "0501_2.0065.wav": "이거 내가 웃는 얼굴 안", "0501_2.0068.wav": "의미가 있어", "0501_2.0069.wav": "아 투머치토커", "0501_2.0071.wav": "그렇군", "0501_2.0072.wav": "여기 오니까 역시나 놀랐던 거 많이 알아봐야 돼", "0501_2.0074.wav": "이제 우리 그거 해야 돼요 소곤소곤 귓속말 맛", "0501_2.0076.wav": "이제 내가 좀 감을 잡았나 봐", "0501_2.0077.wav": "만두가 이렇게", "0501_2.0078.wav": "알려주지", "0501_2.0080.wav": "이미 떠 있었어 이제 코다로 가라고 여기 모니터에 떠 있었어", "0501_2.0081.wav": "하지만 저 이거 못 봤으니까", "0501_4.0000.wav": "근로자의 다리도 쉬지 못하는 사람", "0501_4.0001.wav": "이것이 우리의 현주소입니다", "0501_4.0002.wav": "대박 대박 나 진짜 다시 틀 거야 드라마", "0501_4.0003.wav": "가만있어 봐요 좀 싫어요 여기 리모컨 높이 들었어요 나 잡아봐라", "0501_4.0007.wav": "키가 커", "0501_4.0008.wav": "아 너무 높아", "0501_4.0011.wav": "알 자꾸 채널 돌리니까 버퍼링 걸려 잖아요 저것 좀 봐요", "0501_4.0012.wav": "내가 뭐라고 그 그 그 그 그", "0501_4.0013.wav": "김고은 대박 대박", "0501_4.0015.wav": "전 왼쪽으로", "0501_4.0016.wav": "전보다 더 자기를 좋아하니까", "0501_4.0019.wav": "불교 보는 것도 이렇게 귀엽냐", "0501_4.0020.wav": "뭐야 갑자기 뭐 뭐 이러면 내가 막 좋아하고 그럴 줄 어떻게 알았지 내가 이겼어 내가 이겼어 대박 내 마음에 막걸리나 보여", "0501_4.0021.wav": "이 과장", "0501_4.0022.wav": "내 마음속에 저장", "0501_4.0023.wav": "표정 대박", "0501_4.0024.wav": "내 허밍어반스테레오 하와이안커플 듣고 왔습니다", "0501_4.0025.wav": "힘들어 어쨌거나 청춘", "0501_4.0026.wav": "제가 옛날에 이사제를 어떻게 했었더랬다", "0501_4.0027.wav": "와이", "0501_4.0028.wav": "몇 명이에요", "0501_4.0029.wav": "이렇게 진짜 힘들다 힘들다", "0501_4.0031.wav": "권현정 씨", "0501_4.0032.wav": "와 김분도 그대로야 대박", "0501_4.0033.wav": "기분이 정말 인기가 많았어", "0501_4.0034.wav": "제가 생각하는이 멋진 남자애 톤으로 그 당시에 김구", "0501_4.0035.wav": "연기를 열심히 했었는데", "0501_4.0036.wav": "50분 진짜 인기 많았잖아요 그래서 김 분이랑 과장 결혼한다고 했을 때만", "0501_4.0037.wav": "저희 엄마가 그 결혼식 엄마 갈 거라고", "0501_4.0042.wav": "그 진심을 몰라 둘이 아직도 잘 살고 있는 거였어 김군 진짜 부처 보살님이네", "0501_4.0044.wav": "왜요 매력 있죠", "0501_4.0045.wav": "이 과장 매력 있죠", "0501_4.0046.wav": "박영민 씨가 아 크크크 완전 공감 채널 전쟁이다", "0501_4.0047.wav": "이런 거야 정말", "0501_4.0048.wav": "채널 아 맞아요 저도 어렸을 때 언니랑 진짜 채널 가지고만 있었던 거 같아", "0501_4.0049.wav": "김효정 씨 흔한 부부싸움 남편은 9번 안에는 7번 크크크", "0501_4.0050.wav": "059 님 ig 누군 발령 걔도 재능이 있다니", "0501_4.0051.wav": "못 보셨어요 진심이 닿다 해돋이 발연기", "0501_4.0052.wav": "관련 귀신이 게임 많이 있었는데", "0501_4.0053.wav": "어떤 신 보다도 제가 고민을 많이 했던 싱글이에요 도대체 이거", "0501_4.0054.wav": "어떻게 해야지 진짜가 될까", "0501_4.0055.wav": "어떻게 해야 진짜 드라마 쓰기", "0501_4.0056.wav": "여주인공이", "0501_4.0057.wav": "발 연기를 하는 것 좀 볼 수 있을까", "0501_4.0058.wav": "아주 고민을 많이 해서 했던게", "0501_4.0059.wav": "왜 갔어요 왜", "0501_4.0060.wav": "연기가 보통 할 때", "0501_4.0061.wav": "응 내가 그랬잖아 아니야 그래 그래서", "0501_4.0062.wav": "암 알았어 먹을게 이렇게 하면 약간 단련기 다 되는 경우가 많이 있거든", "0501_4.0063.wav": "그거 여기다 넣어", "0501_4.0064.wav": "그러세요 왜 내가 뭐라고 이렇게 당신 목숨 이럴 수도 있다고", "0501_4.0065.wav": "선생님이 진짜 진짜 좋아 하셨어요", "0501_4.0066.wav": "현장에서 자주 다", "0501_4.0067.wav": "너무 웃겨서 쓰러지게 웃었는데", "0501_4.0069.wav": "아 특별출연 해주신 장기용 씨가 정말 고생을 많이 하셨죠", "0501_4.0070.wav": "제가 그런 발령 길을 하는데도", "0501_4.0071.wav": "나 앞에서 감정을 잡고 눈물을 흘리신 연기를 아주 아주 열심히 집중해서", "0501_4.0073.wav": "다시 한", "0501_4.0074.wav": "귀요미에게 박수를 보냅니다", "0501_4.0075.wav": "정지훈 씨가 구구구구구구구", "0501_4.0077.wav": "이거 작가님 이랑 피디님 이 방송국 에 하고 싶으신 말씀 맞는 거죠", "0501_4.0079.wav": "박수를 치고 계세요", "0501_4.0080.wav": "다 때려 치우고 계시고", "0501_4.0081.wav": "애들이 물도 맞추려고 저 저장 애들이", "0501_4.0082.wav": "어떻게 알았지 이거는 나 애들이랑 고치지도 않았는데", "0501_4.0083.wav": "저장을 나눠 하려니까", "0501_4.0085.wav": "김근희", "0501_4.0086.wav": "그거 유 과장님 짱 화면을 저장 이게", "0501_4.0087.wav": "잘 모를 거 같은 거야 그래서 저 짱짜장 이렇게 내가", "0501_4.0088.wav": "알아들으라고", "0501_4.0090.wav": "우리 좀 쉬어들은 너무 똑똑해서 그런 사람 없겠지", "0501_4.0091.wav": "간혹 가다", "0501_4.0093.wav": "그래도 오늘 애들이 맞춰 주신 분들이 계세요", "0501_4.0094.wav": "김원빈 씨", "0501_4.0096.wav": "종사자 씨", "0501_4.0097.wav": "맞았어", "0501_4.0098.wav": "종사자 씨", "0501_4.0099.wav": "어 그래 자고 오늘 모든 직장인들이 쉬는 건 아닙니다 오늘도 일하고 있는 방송국 종사자 한 분을 만나 갔음", "0501_4.0100.wav": "종사자수", "0501_4.0101.wav": "정사 다시 이게 애들이 할 거야 또", "0501_4.0102.wav": "유: 아나운서 어딨어 이거 정답이에요 8시 반: 아나운서", "0501_4.0103.wav": "두 가지 애들이 맞춰 주신 분들", "0501_4.0104.wav": "이미를 골라서 선물 보내 드리도록 하겠습니다", "0501_4.0105.wav": "지금은 43분 35초네", "0501_4.0106.wav": "아직 나에겐 1분 30초", "0501_4.0109.wav": "316 님은 이게 아닌데 리모컨 싸움 하다가 막 각방쓰고 해야하는", "0501_4.0110.wav": "왜 갑자기 닭살든", "0501_4.0111.wav": "그러게 말이에요 천생연분인가 봐요 유 과장이랑 김 군이랑", "0501_4.0113.wav": "진짜 잘 맞나 봐요", "0501_4.0114.wav": "어디 김분 같은 사람이 어디 없을까", "0501_4.0115.wav": "없을 거 없을 거야 없어요", "0501_4.0116.wav": "정유정 씨는 솔직히 말해 봐요 음성 변조 목소리도 효과 아니고 언니가 직접 한 것", "0501_4.0117.wav": "정말 못내는 목소리가 없어 크크 크크", "0501_4.0118.wav": "왜 아니야 혜정 씨", "0501_4.0119.wav": "아까 나한테 이거 음성 문자 하지 말고 그냥 하라는 거예요", "0501_4.0121.wav": "아 진짜 부탁드립니다 요거 하나만 음성 전주로 좀 해 주십시오", "0501_4.0122.wav": "다른 건 열심히 해 보겠습니다", "0501_4.0123.wav": "음성 변조까지는 제가 아직 못 했어", "0501_4.0124.wav": "언젠가 언젠가 돌아올 때는 제가 이것도 완벽하게 해서 돌아오도록 하겠습니다", "0501_4.0125.wav": "여러분들이 원하신다면 말이에요", "0501_4.0126.wav": "자 그럼 우리 노래를 한 곡 듣고 와야죠", "0501_4.0127.wav": "와서", "0501_4.0129.wav": "오승원 아나운서 님이랑 함께 돌아온", "0501_4.0130.wav": "그건 조금 있다", "0501_4.0133.wav": "노래를", "0501_4.0134.wav": "뛰어들게", "0502_1_vocals.0001.wav": "본인", "0502_1_vocals.0002.wav": "여름 같은데도 있고", "0502_1_vocals.0003.wav": "그런 날이 바로 길을 잃은 하루인 것", "0502_1_vocals.0004.wav": "계절만 그런가", "0502_1_vocals.0005.wav": "오늘을 살면서 과거에 어떤 날로 되돌아간 것 같은 날도 있어요", "0502_1_vocals.0006.wav": "그게 언제든 어떤 계절이 돼", "0502_1_vocals.0007.wav": "반가운 하루라면 참 좋겠네", "0502_1_vocals.0008.wav": "네", "0502_1_vocals.0009.wav": "나는 일단 우리가 보낸 시간이", "0502_1_vocals.0010.wav": "기분 좋은 다른 계절이 없길 바라요", "0502_1_vocals.0011.wav": "보고 싶었어요", "0502_1_vocals.0012.wav": "여기는 악동뮤지션 수현의 볼륨을 높여요 있어요 저는 스페셜 DJ 유인나", "0502_1_vocals.0013.wav": "매일 웃고 있으면 너무", "0502_1_vocals.0016.wav": "5월 2일 목요일 자꾸 그러는", "0502_1_vocals.0017.wav": "엔젤스", "0502_1_vocals.0018.wav": "드라마 한 번 더 해피엔딩 OST 였어요 엔젤 California 드렸습니다", "0502_1_vocals.0020.wav": "안녕하세요 저는 스페셜 DJ 유인나 유두입니다", "0502_1_vocals.0021.wav": "아 정말 너무 감동이에요 우리 선곡이 전 정말 이런 노래가 있다는 걸 잊고 살고 있었거든", "0502_1_vocals.0022.wav": "제가 불렀어요 여러분이 노래를", "0502_1_vocals.0023.wav": "잘됐다 그러면은 갑자기 노래를 조금 못 하면서 갑자기 뭔가 부족한", "0502_1_vocals.0024.wav": "그런 소리가 나는데 그게 바로 더 파티", "0502_1_vocals.0025.wav": "잠시 걸그룹으로 활동을 했었죠 제가", "0502_1_vocals.0026.wav": "최고의 사랑 때도 제 걸 그룹 활동 했던 거 날씨", "0502_1_vocals.0027.wav": "아무튼 이렇게 저를 위한 성 구글 막", "0502_1_vocals.0029.wav": "해 주셔서 정말 감사해요", "0502_1_vocals.0031.wav": "시간이 진짜 빨리 가네요 오늘이 벌써 나올게요 마지막 날이네요", "0502_1_vocals.0032.wav": "응", "0502_1_vocals.0035.wav": "그런 건 나중에 얘기 하지 뭐 지금", "0502_1_vocals.0036.wav": "지금부터 같이 오라고", "0502_1_vocals.0038.wav": "마음이 너무 따뜻해요 정말 아플 치료에 굉장히 많은 분들이 오셨어요 쓸 수 없어", "0502_1_vocals.0039.wav": "하기 여의도공원까지 계속해 줘 가지고 안녕하세요 들리시죠 오늘은 정말", "0502_1_vocals.0040.wav": "진짜 많은 분들이 오셨어요 저랑 이렇게 마주 보고 계시는데", "0502_1_vocals.0041.wav": "제가 손을 흔들면 다 같이 손 흔들고 제가 이렇게 하트를 하면 또 다 같이 어떻게 해요 그래서 정말 지금 마음이 너무너무 따뜻하네", "0502_1_vocals.0042.wav": "여러분", "0502_1_vocals.0043.wav": "배고파요", "0502_1_vocals.0044.wav": "샌드위치 받았어요 샌드위치 드시고 그래도 배가 고프다 하면 또 말씀해주세요", "0502_1_vocals.0045.wav": "어 그래 그래 알았어 더 배고프다고 할 사람들이야", "0502_1_vocals.0046.wav": "문자를 좀 볼게요", "0502_1_vocals.0047.wav": "많은 분들이 지금", "0502_1_vocals.0049.wav": "김성연 씨가 유리와 함께 이번 주가 그랬어요 같이 그때로 돌아가던 것 같아", "0502_1_vocals.0050.wav": "맞아 우리 오프닝 있었던 얘기처럼", "0502_1_vocals.0051.wav": "오늘을 살고 있는데 과거의 언젠가를 돌아간 것 같아", "0502_1_vocals.0052.wav": "그런 생각이 드는 날 있잖아요", "0502_1_vocals.0053.wav": "요번 주에 우리가 함께했던 월 화 수 목이 좀 그랬던 거 같아요", "0502_1_vocals.0054.wav": "와 어떡해", "0502_1_vocals.0055.wav": "그 자리로 오니까 어떻게 그 느낌 그대로", "0502_1_vocals.0056.wav": "모두가 모든 것이", "0502_1_vocals.0057.wav": "이렇게 같은 느낌으로 있을 수 있지", "0502_1_vocals.0058.wav": "아주 신비로운 경험 한 거 같아", "0502_1_vocals.0059.wav": "김현숙 시간", "0502_1_vocals.0060.wav": "당신은 사계절의 매력을 가진 사람이", "0502_1_vocals.0062.wav": "뭔지 모르겠는데 너무 과한 칭찬인 거 같아서 기분이 좋습니다", "0502_1_vocals.0064.wav": "이렇게 와 주셔서 고맙고 감사합니다 언제나 응원할게요 하셨어", "0502_1_vocals.0065.wav": "아 물론 저도 언제나", "0502_1_vocals.0066.wav": "언제나 제가 있는 곳에서 항상 여러분 생각을 하고 있고요", "0502_1_vocals.0067.wav": "심지어 저는 여러분 이야기를 정말 많이 해요 평소에", "0502_1_vocals.0069.wav": "10분 항상 응원하고 있어", "0502_1_vocals.0070.wav": "하고 싶은 얘기도 진짜 많은데 그때 그때", "0502_1_vocals.0071.wav": "방법이", "0502_1_vocals.0072.wav": "SNS g", "0502_1_vocals.0073.wav": "못 해 가지고", "0502_1_vocals.0074.wav": "방금 한번 찾아볼게", "0502_1_vocals.0075.wav": "아이 문자 뭔지 모르게 되게 좋다 4990 님이 있나 짜증 나 짜증 나 짜증 나 짜증 나 짜증 나 짜증 나", "0502_1_vocals.0076.wav": "하트도", "0502_1_vocals.0077.wav": "4 5 6 7 9 개 찍어 주셨어요", "0502_1_vocals.0078.wav": "응 세금 언니 나 지금 이렇게 불렀어요", "0502_1_vocals.0079.wav": "알겠습니다", "0502_1_vocals.0080.wav": "이주리 씨가 있는 언니가 오픈 스튜디오 있는 팬들에게 샌드위치를 돌렸어요 유인나 당신은 핏 당신은 사랑 당신은 빵", "0502_1_vocals.0081.wav": "물방울 끝났냐", "0502_1_vocals.0082.wav": "맞아요", "0502_1_vocals.0083.wav": "먹고 싶은 거는 제가 다 사 드릴 수 있어", "0502_1_vocals.0084.wav": "그러니까 저 편도 편도 앞에 있죠 팬티 도둑 편", "0502_1_vocals.0085.wav": "분들 아시는분들은 아시겠지만", "0502_1_vocals.0086.wav": "저희 매니저", "0502_1_vocals.0087.wav": "김견 성분의 별명이 팬티도둑 팬도 있잖아요", "0502_1_vocals.0088.wav": "언젠가는 자기가 그 설명이 잊혀질거라 기대를 하고 있지만 이렇게 또 한번 자기가 분명하게 말씀을 드린 겁니다 왜 팬티도둑 있냐 군대 일지 당시의", "0502_1_vocals.0089.wav": "팬티를 막 훔쳐 이거 어때요", "0502_1_vocals.0090.wav": "서로 서로 엄청 있고 그래 딴 얘기를 듣고 제가 너무 변태 도둑아", "0502_1_vocals.0091.wav": "어 저기 있다 저기 있다 팬더", "0502_1_vocals.0092.wav": "현정에게 뭐가 먹고 싶은지 여러분 다 얘기하세요 사 줄게", "0502_1_vocals.0093.wav": "8시 10분 이네요 33초", "0502_1_vocals.0094.wav": "지금 그러면 퀴즈입니다 지금 뭐 하는지", "0502_2_vocals.0000.wav": "수현의 볼륨을 높여요 함께 하고 계십니다", "0502_2_vocals.0001.wav": "월요일부터 오늘까지 함께 하고 있는 스페셜 DJ 유인나입니다", "0502_2_vocals.0002.wav": "어 아까 코너 소개를 안 해 드렸어요 목요일 3시 이후에는", "0502_2_vocals.0003.wav": "200%라는 코너 함께해요 주제를 하나 두고", "0502_2_vocals.0004.wav": "그런 거 대화하는 시간이에요 아니 시간 일해요", "0502_2_vocals.0005.wav": "너무나 능수능란한 존나 이런 거는 좀 안 할게", "0502_2_vocals.0006.wav": "그런 시간이 일해요 여러분 우리 그거 잘 하잖아요 하나 주제를 두고 우리끼리 떠드는 거", "0502_2_vocals.0007.wav": "제주 재미있다 3부에서 알려 드릴 거고요 지금은 그냥 하고 싶은 이야기 마구마구 보내주세요", "0502_2_vocals.0008.wav": "오늘 어땠어요 편한 친구한테 말하듯이 아무 얘기나 해 주시면 됩니다", "0502_2_vocals.0009.wav": "문자 번호 #8910 안중에 50원 김건 100원", "0502_2_vocals.0010.wav": "카톡 마이 K는 무료입니다", "0502_2_vocals.0011.wav": "신청곡도 보내주시면 틈틈이 챙겨 드릴게요", "0502_2_vocals.0016.wav": "우리가 어디 보자 너무 친근해 줘 엄마 엄마가 잘내는 소리", "0502_2_vocals.0017.wav": "어디 보자", "0502_2_vocals.0019.wav": "사람들을 지금 좀 볼까요 소근소근 귓속말로 갈까요", "0502_2_vocals.0020.wav": "소곤소곤 귓속말 시간이 긴 아저씨", "0502_2_vocals.0022.wav": "설정을 씨가 어제 유지가 감자전 만나자고 해서 오늘 감자전 했어요", "0502_2_vocals.0023.wav": "남편이 이틀 연속 부침개 특집이냐고", "0502_2_vocals.0025.wav": "아 이틀 연속 저는 조금 그렇다", "0502_2_vocals.0026.wav": "하지만 감자전 이라면 저는 매일 매일 먹을 수 있을 거 같아", "0502_2_vocals.0028.wav": "2017년 중학교 땐 집에서 만들었는", "0502_2_vocals.0029.wav": "드디어 성인이 되어서 오픈 스튜디오 부스 앞에서 라디오 들어요 중학생때부터 소원이었는데 소원 성취 했어", "0502_2_vocals.0031.wav": "쑥스러울 수 있으니까 마이크는 안 10월 10일테니까", "0502_2_vocals.0032.wav": "누군지 선반 들어가 줘", "0502_2_vocals.0033.wav": "어디", "0502_2_vocals.0034.wav": "하여도 주무시고 너무너무 귀여운", "0502_2_vocals.0035.wav": "50대 같은데 서류 맞아요 20", "0502_2_vocals.0038.wav": "정말 너무 너무 귀여워", "0502_2_vocals.0039.wav": "귀여운 우리", "0502_2_vocals.0040.wav": "... 땡", "0502_2_vocals.0041.wav": "진짜 너무너무 제가 이런게 내가 키운 거잖아 사실 그죠 꼭 맞대 맞대", "0502_2_vocals.0043.wav": "아 기분이 정말 좋네요 뿌듯해", "0502_2_vocals.0045.wav": "우리 붉은 가족들이", "0502_2_vocals.0047.wav": "우와 이거 사진인가 봐 어머 대박 유진이가 진짜 내일", "0502_2_vocals.0048.wav": "유리 유리 방송 함께 들으면 데이트하던 남자가 나랑 결혼해 해서 지금 아기 재우고 있어요 시간이 이렇게 결정 훌쩍 지났네요", "0502_2_vocals.0049.wav": "아기 잠들면 다시 듣기로 함께 들으면 말랑한 기분 느껴야 겠어요", "0502_2_vocals.0051.wav": "아기가", "0502_2_vocals.0052.wav": "정말 천사 같다", "0502_2_vocals.0053.wav": "너 정말 방울방울 하구나", "0502_2_vocals.0054.wav": "정말 예쁘다", "0502_2_vocals.0056.wav": "천사처럼 자고 있어요", "0502_2_vocals.0057.wav": "진짜 감개무량하다 축하해요", "0502_2_vocals.0058.wav": "891 님 선물 좋은 거 챙겨서 보내 드릴게요", "0502_2_vocals.0059.wav": "당신 6시가 누나 그래서 미역국은 어떻게 만든다고요 알려 준다면서", "0502_2_vocals.0060.wav": "알려 줘", "0502_2_vocals.0061.wav": "아니 근데 이거 제가 해보니까 스스로 터득하는게 나아", "0502_2_vocals.0062.wav": "내가 뭐 알려 주고", "0502_2_vocals.0063.wav": "다 이제 자기가 해봐야겠어 뜨게 되는 거지", "0502_2_vocals.0064.wav": "여러분들을 강하게 키우고 싶기 때문에", "0502_2_vocals.0066.wav": "너무 촉박하니까 이번 주까지 미역국 만들어 보고", "0502_2_vocals.0067.wav": "어떻게 어떻게 만들었는지 구체적으로 해서 여기다", "0502_2_vocals.0068.wav": "#8910 한 1050원 김건백 1인 여기에다가 문자를 주세요", "0502_2_vocals.0069.wav": "그러면 아마 수요일 시간 확인 하실 거예요", "0502_2_vocals.0072.wav": "175 산이 뮤지컬입니다 지하철에서 퇴근길에 듣고 있는데요 큰일이에요 배터리가 얼마 안 남았어요 아직 20분 더 가야 하는데 유유", "0502_2_vocals.0073.wav": "보조 배터리 없어요", "0502_2_vocals.0074.wav": "우리 여기 보조 배터리 없나 선물", "0502_2_vocals.0075.wav": "없다", "0502_2_vocals.0076.wav": "있어 있어", "0502_2_vocals.0077.wav": "있어 있어 웬일이니", "0502_2_vocals.0078.wav": "아니 이거 어디로 산님 보조배터리 선물로 보내 드릴게요 모바일 교환권으로 드릴 수 있는 거 같아", "0502_2_vocals.0079.wav": "다음 이거 드릴테니까 꼭 챙겨서 다니세요", "0502_2_vocals.0080.wav": "그러면 이제 그 시간이에요 어제처럼 그럼 무서운 시간 아니라고요 전세", "0502_2_vocals.0082.wav": "어제 제가 배웠잖아요 어떤 톤으로 해야 하는 건지", "0502_2_vocals.0085.wav": "이렇게 하면 무서운", "0502_2_vocals.0086.wav": "소근소근 개통", "0502_2_vocals.0087.wav": "이렇게 하면 안 무서운", "0502_2_vocals.0088.wav": "두 번째 걸로", "0502_2_vocals.0089.wav": "할 건데요", "0502_2_vocals.0090.wav": "제가 너무 관여를 하나 싶기도 한데 리코더를", "0502_2_vocals.0091.wav": "오늘도 제가 하고 싶은 걸로", "0502_2_vocals.0092.wav": "구경하시고 이걸 좀 나왔어요 여러분 드리라", "0502_2_vocals.0093.wav": "공유하고 싶어서 제가 요즘에", "0502_2_vocals.0094.wav": "한강을 걸어요 4시 5시 6시로 때찌", "0502_2_vocals.0095.wav": "한강 걸으면서", "0502_2_vocals.0096.wav": "이 노래를 반복 재생을 해서", "0502_2_vocals.0097.wav": "들으면서 모든", "0502_2_vocals.0098.wav": "나무와 강과 노을과 아름다운 모든 걸 눈에 담고 있거든", "0502_2_vocals.0099.wav": "여러분들도 내일부터는 그 시간에 저와 함께", "0502_2_vocals.0100.wav": "그런 시간으로 빠졌으면 좋겠다", "0502_2_vocals.0101.wav": "바람에", "0502_2_vocals.0102.wav": "이걸 가져갔습니다", "0502_2_vocals.0105.wav": "볼까요", "0502_3_vocals.0001.wav": "네 여러분 어땠나요", "0502_3_vocals.0002.wav": "좋았어요", "0502_3_vocals.0003.wav": "응", "0502_3_vocals.0004.wav": "제가 요즘에", "0502_3_vocals.0005.wav": "고쳐 가지고", "0502_3_vocals.0006.wav": "이 노래를 이럴 거미 노래를 계속 듣고 있었어요 그래서", "0502_3_vocals.0007.wav": "어 여기 저기", "0502_3_vocals.0008.wav": "이것 좀 들어 봐라 이것 좀 들어 봐라", "0502_3_vocals.0010.wav": "듣는데 꼭 한강을 봐라 귀찮게 언니한테 전화해서 언니 언니 내가 지금이 노래를 들으면서 내가 보내는 사진 한강을 봐봐", "0502_3_vocals.0011.wav": "이런 그랬거든", "0502_3_vocals.0012.wav": "이 노래 들으면서 한강 사진 내가 찍은 한강 사진 보면 진짜 좋은", "0502_3_vocals.0013.wav": "그거 어떻게 보여 주고 싶", "0502_3_vocals.0014.wav": "그거 우리 팬도 팬도 SNS 있는데", "0502_3_vocals.0015.wav": "올려 주라", "0502_3_vocals.0016.wav": "올려 주십시오", "0502_3_vocals.0017.wav": "그러면 부탁드리겠습니다", "0502_3_vocals.0018.wav": "한강사", "0502_3_vocals.0019.wav": "이따가 제가 보내 드릴테니까", "0502_3_vocals.0020.wav": "해준대요 자기 있다고 그걸로 되겠제 어", "0502_3_vocals.0021.wav": "좋은 거 내가 올려 달라고 하면", "0502_3_vocals.0023.wav": "나한테 그걸 통해서 제가", "0502_3_vocals.0024.wav": "화면 사진으로 올릴테니까", "0502_3_vocals.0025.wav": "분위기 뭐야 이거 왜 그러냐고", "0502_3_vocals.0026.wav": "무섭다", "0502_3_vocals.0027.wav": "나 오늘은 안 무섭게 한다고 했는데 무섭대", "0502_3_vocals.0028.wav": "0 2 3 5 7 1", "0502_3_vocals.0029.wav": "빨간 휴지 줄까 파란 휴지 줄까", "0502_3_vocals.0031.wav": "희진 씨드 아닌 거 맞죠", "0502_3_vocals.0033.wav": "오늘부로 소곤소곤 귓속말 코너는 폐지됐습니다 여러분 끝", "0502_3_vocals.0034.wav": "몰라 몰라 나는 오늘까지고 반응이 계속 이러네", "0502_3_vocals.0035.wav": "시흥시 갈 때는 무서워 안 무서워 안 무서워 써요", "0502_3_vocals.0036.wav": "아 그래요 그럼 다행이네", "0502_3_vocals.0037.wav": "원래 무서운 크는가 봐요 이거 여름에 좋지 뭐", "0502_3_vocals.0039.wav": "빵 빠라 빠라 빠라 빵빵 뭐야", "0502_3_vocals.0040.wav": "이게 뭐냐", "0502_3_vocals.0041.wav": "맞아요 hec 가나인 언니 노래 너무 좋은 거 많아요 맞아요 저도 요즘에 계속 찾아서 들어 보고 있는데", "0502_3_vocals.0042.wav": "정말 좋더라고요", "0502_3_vocals.0043.wav": "봉봉 씨가 아닌 것 같아 왜냐하면 소곤소곤 하기 전에 이미 뿌리 완성된 목소리는 걸요", "0502_3_vocals.0044.wav": "이런 과한 칭찬하는 아주 좋아하지", "0502_3_vocals.0046.wav": "겸손이란 없어요 칭찬 가지는 너무 너무 신나요 칭찬 많이 많이 해 주세요", "0502_3_vocals.0048.wav": "사연을 이거 하나", "0502_3_vocals.0049.wav": "2064", "0502_3_vocals.0050.wav": "저한테는 잘생긴 남자 친구 한 명 있어요 그런데 일주일 전에", "0502_3_vocals.0051.wav": "저한테 돈을 빌려 갔는데 연락이", "0502_3_vocals.0052.wav": "돈은 잃어도 되는데 잘생긴남사친 읽기 싫어요", "0502_3_vocals.0054.wav": "얼마 빌려 갔대", "0502_3_vocals.0055.wav": "그래도 안 받을 생각으로 빌려주 적겠죠 보통 요즘에는 알고 계시잖아요 도는 분이라는 거", "0502_3_vocals.0056.wav": "돈 많이 주는게 아니라 친구까지 있는 거니까 내가 줄 수 있다 선 안에서만 빌려 주자", "0502_3_vocals.0057.wav": "많은 걸 우리가 이제 다들 알고 있기 때문에", "0502_3_vocals.0058.wav": "문자 한번 해 보세요 야 돈 안 갚아도 된다 연락해라 너까지 입고 싶지 않다", "0502_3_vocals.0059.wav": "그러면서", "0502_3_vocals.0060.wav": "엄청 엄청 이걸 계기로 좋은 친구 생일 가도 될 수도 있지", "0502_3_vocals.0061.wav": "아닌가 좋은 연인사이가 되고 싶은 건가", "0502_3_vocals.0062.wav": "야", "0502_3_vocals.0063.wav": "그렇죠 그렇죠", "0502_3_vocals.0064.wav": "이 소리는", "0502_3_vocals.0065.wav": "광고로 넘어가는 소리겠지 아무래도", "0502_3_vocals.0066.wav": "아이 노래 진짜 좋다", "0502_4_vocals.0000.wav": "초등학생 여자 아이가 커서 발레리가 됐다 현서 만들어진 캐릭터인데", "0502_4_vocals.0001.wav": "아직도 목소리 힘이 좀 없죠", "0502_4_vocals.0003.wav": "어디세요 이런 유 과장한테지지 않잖아요", "0502_4_vocals.0004.wav": "발레리 여전히 너무 다소곳하게 잘 먹어", "0502_4_vocals.0005.wav": "아니 미안 잘 먹여", "0502_4_vocals.0006.wav": "내가 보고 싶은 대로 많나 봐요 잘 먹여 한 방을 잘 먹인다고", "0502_4_vocals.0008.wav": "강경모 씨 아 대박", "0502_4_vocals.0009.wav": "이거 해 보면 재밌어요 저기 처음에는 이게 저도 유 과장", "0502_4_vocals.0010.wav": "그 당시에 작가 언니가 과장이란 캐릭터는 좀 일을 했으면 좋겠다 해서 제가 어 그럼 말투를 좀 우와 대박 이런 거겠네 응 알겠어 하고 만들어 준 건데 아주 중독성 있어요 하다 보면 재밌다고 열기", "0502_4_vocals.0011.wav": "이준엽 씨가 끊어 끊어 끊어 끊어 애들이 드렸죠 맞아요 저는 이런 건 안 하지만 이런 거는 이런 전화는 거리가 멀지만", "0502_4_vocals.0012.wav": "애들이 되었고요", "0502_4_vocals.0014.wav": "척추 진주", "0502_4_vocals.0015.wav": "다른 존재감에 드리고 정말 귀신이다", "0502_4_vocals.0016.wav": "뭘 하면 다 알아 이제", "0502_4_vocals.0017.wav": "송영진 집 저런 걸 몇 년째 애들이", "0502_4_vocals.0018.wav": "맞는 거 같아", "0502_4_vocals.0019.wav": "웬만한 건 다 제 애들이 그랬던 거 같아요 쓸데없는 말이다 이거는 이런 거", "0502_4_vocals.0022.wav": "5892 님이요 저 요즘 PT 받는거든요 근데 트레이너쌤 메세지 덮어도 놓고", "0502_4_vocals.0023.wav": "감자튀김이랑 오징어랑 칠리소스에 가득 찍어서 먹었어요 칭찬해 주세요", "0502_4_vocals.0024.wav": "어디가 칭찬할 일이", "0502_4_vocals.0025.wav": "칭찬 할 일이 없어", "0502_4_vocals.0026.wav": "이게 뭐야", "0502_4_vocals.0027.wav": "피티지 얼마나 비싼데", "0502_4_vocals.0028.wav": "이거 하는 동안만이라도", "0502_4_vocals.0029.wav": "먹지 말라는 것은 먹지 마시고", "0502_4_vocals.0030.wav": "소수라도 인간적으로 빼자 진짜", "0502_4_vocals.0031.wav": "감자튀김", "0502_4_vocals.0032.wav": "빼야 되는", "0502_4_vocals.0033.wav": "튀김지 소스를 빼요 그러면", "0502_4_vocals.0034.wav": "너무 이거 돈이 아깝잖아 노력도 아깝고 이때만큼은 진짜 집중해서 열심히 하셔야 돼", "0502_4_vocals.0035.wav": "파이팅입니다 내일부터 지은 씨한테 589 님한테", "0502_4_vocals.0036.wav": "선물을 보내 드려야 되겠어 피자 싣고", "0502_4_vocals.0037.wav": "피자가 있으면서 내가 다이어트를 한다 이정도 위즈는 있어 줘야 되는 거거든요", "0502_4_vocals.0038.wav": "피자 시즌권 보내드리겠습니다", "0502_4_vocals.0039.wav": "이성연 cud 누나 내일 부대 복귀 해야 돼서 뜨겁게 볼륨 듣고 있어", "0502_4_vocals.0041.wav": "상윤 씨", "0502_4_vocals.0042.wav": "기분이 어때요", "0502_4_vocals.0043.wav": "아니 좀 걱정이 야 진짜 왜 이렇게", "0502_4_vocals.0044.wav": "걱정이 되는지 모르겠어요 마음을 막 그래요 요즘 애들 저는 기회 다니다 보면", "0502_4_vocals.0045.wav": "카드 우리 학생 친구들이랑 5년 간 여기서 정이 많이 쌓이고", "0502_4_vocals.0046.wav": "군인 친구들이랑도 너무 얘기를 많이 나눠서", "0502_4_vocals.0047.wav": "지나가다 보면", "0502_4_vocals.0049.wav": "재도 내가 키워내고 재도 내가 기분이 되네", "0502_4_vocals.0050.wav": "아이고 지금 내 새끼 군단 내 새낀데 이런 생각을 말 들으면서 다 안 하시고 싶더라고요", "0502_4_vocals.0051.wav": "그래요 열어 줘", "0502_4_vocals.0052.wav": "송영진 씨 단호박 이나 유디 원래 칭찬 잘해주는데", "0502_4_vocals.0053.wav": "그치 나는 진짜 칭찬해 주는 거 칭찬 받는 거 좋아하는데", "0502_4_vocals.0054.wav": "PT가 아니 저희 언니가 요즘에 운동을 그렇게 열심히 하더라고", "0502_4_vocals.0055.wav": "근데", "0502_4_vocals.0056.wav": "이게 효과를 많이 봤어요 저희 언니가 그걸 보고 나니까 PT 할 때 진짜", "0502_4_vocals.0057.wav": "받을 때", "0502_4_vocals.0058.wav": "일단 제대로 하고 열심히 하면 효과가 이정도가 있는 거고 나를 제가 눈으로 확인 했기 때문에", "0502_4_vocals.0059.wav": "입금하게 열심히 하시라고 응원을 해 드린 거예요", "0502_4_vocals.0060.wav": "야 그러면", "0502_4_vocals.0062.wav": "8시 43분 21초에 노래를 한 곡 들을게요", "0502_4_vocals.0063.wav": "페퍼톤스 노래 들을까", "0502_4_vocals.0064.wav": "몰라요", "0502_4_vocals.0066.wav": "몰라요", "0502_4_vocals.0068.wav": "근데 다 춤추 졌죠", "0502_4_vocals.0069.wav": "언제든 어디서든이 노래가 흘러나오면 유인나가 어디서든 멈추고 춤을 추고 있겠구나 생각해 주세요", "0502_4_vocals.0070.wav": "정말 좋네요", "0502_4_vocals.0071.wav": "자 지금 부터", "0502_4_vocals.0072.wav": "여러분들의 사연을", "0502_4_vocals.0075.wav": "재밌는게 진짜 많아", "0502_4_vocals.0076.wav": "깜짝이야 박정아 씨", "0502_4_vocals.0077.wav": "유진은 오이 먹을 때 고추장 찍어 먹었잖아요", "0502_4_vocals.0080.wav": "먹었다", "0502_4_vocals.0083.wav": "제가 다이어트 할 때 정말 다이어트에 대해서", "0502_4_vocals.0084.wav": "아무런 지식과 상식이 없었던 시절에", "0502_4_vocals.0085.wav": "오이를 먹자", "0502_4_vocals.0086.wav": "오이 다이어트 식품으로 나는 우유를 정했다", "0502_4_vocals.0087.wav": "아주 고추장을 듬뿍 찍어서 먹었어요 그냥 너무 심심하니까", "0502_4_vocals.0088.wav": "그러면 안 되는 건데", "0502_4_vocals.0089.wav": "아시죠 이런 거 소스 고추장 쌈장 이런 거 빼고 드세요 다이어트 하실 때", "0502_4_vocals.0090.wav": "장미수 씨가 직장인 친구들도 언니가 키웠어요 유유", "0502_4_vocals.0091.wav": "저 내가 키운 아가들 진짜 많지", "0502_4_vocals.0092.wav": "우리 뒷통수도 내가 키웠는데", "0502_4_vocals.0093.wav": "우리 집 벌써 두 군데를 맞잖아요 다 다 저녁을 했어요 여러분들은 다 알고 계시겠지요", "0502_4_vocals.0094.wav": "저녁을 했는데 아직도 못 봐 가지고", "0502_4_vocals.0095.wav": "아 참", "0502_4_vocals.0096.wav": "제가 키우는 또 누가 있어", "0502_4_vocals.0097.wav": "주자 씨도 잘 키웠는데 하차 수지 수지", "0502_4_vocals.0098.wav": "우리 수지는 제가 키워서 이렇게 디제이를 또", "0502_4_vocals.0099.wav": "그렇게 제가 만들어 줬잖아요", "0502_4_vocals.0100.wav": "아 정말 근데 수지씨가 너무 너무 잘하죠 너무 너무 잘 하고 있어서", "0502_4_vocals.0101.wav": "제가 많이 배우고 있습니다 지금", "0502_4_vocals.0102.wav": "시나몬의 진실", "0502_4_vocals.0103.wav": "언니 선택해 주세요 아침밥 아침밥 아침", "0502_4_vocals.0105.wav": "전화 아침밥", "0502_4_vocals.0106.wav": "아침마다 똑같이 먹어요 저는", "0502_4_vocals.0107.wav": "정말 오래 됐어 몇 년이 되는지 모르겠네", "0502_4_vocals.0108.wav": "오래됐는데", "0502_4_vocals.0110.wav": "고구마", "0502_4_vocals.0111.wav": "그리고", "0502_4_vocals.0112.wav": "그릭요거트", "0502_4_vocals.0113.wav": "그리고 커피", "0502_4_vocals.0114.wav": "어 뭐 그 정도 올 거 같아요 가끔 정말 배고픈 아침에는 식빵을 먹기도 하고", "0502_4_vocals.0115.wav": "근데", "0502_4_vocals.0116.wav": "저는 정말로 튼튼해요 단 사과의 덕이라 생각합니다 여러분들도 아침", "0502_4_vocals.0117.wav": "아침에 좀 제가 수리 다 하시는 분들은 조금 지나서 물론 물 한 잔 아침에 맛있게 드시고요", "0502_4_vocals.0118.wav": "아침에 물한잔만 미지근하게 마시고 시작해도", "0502_4_vocals.0119.wav": "위장이 다 편안해져요", "0502_4_vocals.0120.wav": "자", "0502_4_vocals.0122.wav": "50분이다 8시 50분", "0502_4_vocals.0123.wav": "자 그러면 광고를 듣고 와서 제가 광고를 소개해 드리도록 하겠습니다", "0502_4_vocals.0124.wav": "네 5월 협찬품 소개해 드리겠습니다", "0502_4_vocals.0125.wav": "수현이의 볼륨을 높여요에서 드리는 선물이에요", "0502_4_vocals.0126.wav": "헤어기기 전문 브랜드 JMW 전문가용 헤어 드라이어", "0502_4_vocals.0127.wav": "반려동물", "0502_4_vocals.0128.wav": "함박웃음 울지마 마이펫에서 멀티남 이종", "0502_4_vocals.0129.wav": "피부만 봅니다 리더스코스메틱에서 마스크 팩", "0502_4_vocals.0130.wav": "로맨틱 브랜드 로미스토리", "0502_4_vocals.0131.wav": "에서 로미스토리에서 온라인 상품권", "0502_4_vocals.0132.wav": "자라나는 자신감 잘 확인해서 자라 화를 드립니다", "0502_4_vocals.0133.wav": "홍대 메이크업 브러쉬 플라리아에서", "0502_4_vocals.0134.wav": "블랙 에디션 세트", "0502_4_vocals.0135.wav": "예쁘구나 면이면서 카멜리아 오일 폼 클렌저", "0502_4_vocals.0136.wav": "살에 선인장을 담은", "0502_4_vocals.0137.wav": "헉슬리 위에서 오일 에센스", "0502_4_vocals.0138.wav": "자연주의 맞춤 솔루션 티엔에서 극세사 마스크팩 세트", "0502_4_vocals.0139.wav": "피부 전문가 가 만든 바노바기 에서 호일 마스크", "0502_4_vocals.0140.wav": "남성뷰티 키핀터치에서 mlbb 벨벳 틴트", "0502_4_vocals.0141.wav": "한 번 먹고 빠져드는 청년피자에서 피자 시식권", "0502_4_vocals.0142.wav": "천연 화장품 봉프레에서 모바일 상품권", "0502_4_vocals.0143.wav": "퍼플 엑스에서 clitic", "0502_4_vocals.0144.wav": "메디컬스파 코스메틱 하우스에서 비비큐", "0502_4_vocals.0145.wav": "물만으로 클렌징 크롬에서 클렌징퍼프 트러블", "0502_4_vocals.0146.wav": "트래블팩 트래블팩", "0502_4_vocals.0147.wav": "아름다운 계절 아비쥬 뷰티 상품권", "0502_4_vocals.0148.wav": "바디허벌즈에서 천연허브 화장품 세트", "0502_4_vocals.0149.wav": "수연이가 선택한 향기 브랜드 W 드레스룸에서", "0502_4_vocals.0150.wav": "핸드크림 5종 세트 드립니다", "0502_4_vocals.0152.wav": "내 노래 한 곡 듣고 저는 3부에 먼저 가서", "0502_4_vocals.0153.wav": "나 기다릴 건데요 그 전에 잠깐 같이 놀아요", "0502_4_vocals.0154.wav": "선물이 진짜 많네요", "0502_4_vocals.0155.wav": "제가 오늘 한 번도 안 틀렸으니까 20분께 선물을 보내 드리도록 하겠습니다 어깨동무네 응", "0502_4_vocals.0156.wav": "넌 왜 이렇게 안 틀리고 안 되냐 어려운 선물이 정말 많아졌네요", "0502_4_vocals.0157.wav": "그치만 제가 이걸 정말 꼭 성공하고 싶었는데", "0502_4_vocals.0159.wav": "지금부터 제가", "0502_4_vocals.0160.wav": "여러분 많이 많이 뽑아서 선물 드릴게요", "0502_4_vocals.0161.wav": "여자가 잘못했으니까", "0502_4_vocals.0164.wav": "2 해제 보내 주신 거 이거 제가 안 틀리고 읽으면 이거 10분 갑니다", "0502_4_vocals.0165.wav": "간장 공장 공장장 경찰청 창살 쇠창살", "0502_4_vocals.0166.wav": "2시 왔어", "0502_4_vocals.0168.wav": "그래도", "0502_4_vocals.0169.wav": "더 어려운 거 있으면 보내 줘 보세요 여러분 제가 그 미션 클리어 하면", "0502_4_vocals.0170.wav": "10분 선물 드리는 걸로 합시다", "0502_4_vocals.0171.wav": "김천시 가정 커피소년 생각나요 커피소년 특히 오셨잖아요", "0502_5_vocals.0000.wav": "검색으로 살았는데", "0502_5_vocals.0002.wav": "이걸로 바뀌는 게 외롭고 힘들어도", "0502_5_vocals.0003.wav": "그때 우연히 켠 라디오에서 내 귀에 흘러들어온 다정하고 예쁜 목소리", "0502_5_vocals.0004.wav": "유리는", "0502_5_vocals.0005.wav": "내 수험생활에 봄이 왔다", "0502_5_vocals.0006.wav": "우리는 더 행복해질 거예요", "0502_5_vocals.0007.wav": "나는 그린 사는 행복의 주문 같았다", "0502_5_vocals.0008.wav": "오 년 전 오늘의 나는", "0502_5_vocals.0009.wav": "유리에게 위로를 받으면서", "0502_5_vocals.0010.wav": "힘든 시절을 버티던 수험생이 어때", "0502_5_vocals.0011.wav": "어느새 대학 졸업반", "0502_5_vocals.0012.wav": "취업 문제로 또 힘든 시간을 보내고 있는 요즘이었는데", "0502_5_vocals.0013.wav": "잠시나마 돌아와", "0502_5_vocals.0014.wav": "행복해질 거라고 주문을 걸어 주는 얘기가 반가웠다", "0502_5_vocals.0015.wav": "오늘의", "0502_5_vocals.0016.wav": "악동뮤지션 수현의 볼륨을 높여요 잘 간 스페셜 DJ 유인나", "0502_5_vocals.0017.wav": "유리 골디와 함께 하셨습니다", "0502_5_vocals.0018.wav": "어떠셨어요 여러분", "0502_5_vocals.0019.wav": "616 님이 이런 선물 같은 시간 감사했습니다", "0502_5_vocals.0020.wav": "제가 훨씬 훨씬 더 감사했습니다 여러분은 여러분이 저를 너무 많이", "0502_5_vocals.0021.wav": "반겨 주셔서", "0502_5_vocals.0022.wav": "정말 너무 따뜻한 만 가득 느끼고 가요", "0502_5_vocals.0024.wav": "메이즈 님 너무 행복한 생일이었어요", "0502_5_vocals.0025.wav": "저도요", "0502_5_vocals.0026.wav": "300 유리가 한강 일하기 하기에 퇴근길 걷고 있던 한강 사진 찍어 보내요", "0502_5_vocals.0027.wav": "유리 목소리 들으면 걷는 한강은 완성 갬성 폭발 이네요 노래가 따로 필요 없어", "0502_5_vocals.0028.wav": "선물같은 4일 고마웠어요 잘 지내고 잘 살다가 우연이라도 꼭 만나요 하셨습니다", "0502_5_vocals.0029.wav": "물론이죠", "0502_5_vocals.0030.wav": "한강 사진 너무 예쁘다 잘 봤습니다 감사해요", "0502_5_vocals.0031.wav": "어 당연하지 저는 언제나 여기 있으니까요이 자리는 아니더라도", "0502_5_vocals.0032.wav": "어떻게 여러분들한테 인사를 해야 할까요", "0502_5_vocals.0033.wav": "좋은 방법으로 가까운 방법으로 여러분들한테 자주 찾아 볼 수 있는 방법을 연구하고 있어요 꼭 다시 돌아오도록 하겠습니다", "0502_5_vocals.0034.wav": "내일부터 돌아오는 수연 씨가 함께 아주 즐거운 시간 보내시구요 저도", "0502_5_vocals.0035.wav": "즐겁게 즐겁게 언제나 청취하고 있겠습니다 신나는 노래 나오면 유인나가 춤을 추고", "160501_01.0000.wav": "kbs cool fm 유인나의 볼륨을 높여요 4월 10일 일요일 첫 곡으로는 뜨거운 감자의 고백들이셨습니다", "160501_01.0001.wav": "이어 헌터 말장난 고베 은근 많네요 참 진짜 머리좋아요", "160501_01.0003.wav": "난 라면 이야 너와 함께 라면", "160501_01.0005.wav": "난 푸딩이야 오늘도 내가 보고 시프 딩야", "160501_01.0009.wav": "미안합니다", "160501_01.0011.wav": "이런 말 언제 들어 보겠어요", "160501_01.0012.wav": "자하", "160501_01.0013.wav": "이제 박살 좀 털어 내고 광고 듣고 와서 분류는 가족의 사연 만나 보죠 잠깐만요", "160501_01.0014.wav": "어디야", "160501_01.0015.wav": "검색어 하나를 정해서 볼륨 가족들의 사연을 꼼꼼하게 만나는 시간입니다", "160501_01.0016.wav": "볼륨 검색아", "160501_01.0017.wav": "오늘 검색어는 요 넘으로 정해 봤어요 너무", "160501_01.0018.wav": "어떤 이야기들 보내 주셨는지 만나볼게요", "160501_01.0019.wav": "음 6911 임", "160501_01.0020.wav": "친구가 결혼을 해요 그것두 무려 광주에서", "160501_01.0021.wav": "충남 태안에서 전라도 광주까지 너무 멀어요 유유", "160501_01.0022.wav": "22살에 결혼하는 내 친구", "160501_01.0023.wav": "과속인지 아닌지 절대 말 안 해줄 거라는 데", "160501_01.0024.wav": "왠지 과속 맞는 거 같아요 크크크", "160501_01.0028.wav": "너무 과속이 맞는 거 아닌가요", "160501_01.0029.wav": "보통 가속이지 아니면 영", "160501_01.0030.wav": "원이가 나오지", "160501_01.0031.wav": "맞는지 아닌지 말 안 해줄 건데", "160501_01.0032.wav": "이렇게 안 하잖아요", "160501_01.0034.wav": "뭐 그래요", "160501_01.0035.wav": "요즘은 혼수라고도 하더라고요", "160501_01.0036.wav": "아우 뭐 이렇게 된 거 어쩌겠어요 그냥 축하 드립니다", "160501_01.0037.wav": "그런데 얼마나 걸리는데요 태안에서 전라도 광주까지", "160501_01.0039.wav": "잘 다녀오세요 이 기회에 뭔가", "160501_01.0040.wav": "ktx 이런 것 하는 것도 귀찮다고 이런 것도 운치 있잖아요 여행 가는 것처럼 한 번 다녀와 보세요", "160501_01.0042.wav": "너 6783님", "160501_01.0043.wav": "자전거 타고 공원 가다가 타이어가 터져서 걸어서 집에 왔어요 유유", "160501_01.0044.wav": "오랜만에 힐링 하려고 했는데", "160501_01.0045.wav": "힐링은커녕 너무 힘들어서 인상 쓰면서 집에 왔네요", "160501_01.0046.wav": "어쩌다 타이어가 터졌어요", "160501_01.0047.wav": "잘 안 터지는 거잖아요 원래 자전거 타요", "160501_01.0048.wav": "물론 저는 자전거를 몇 번 안타 봐서 잘 모르긴 합니다만", "160501_01.0050.wav": "이것도 자동차처럼 타이어 점검 같은 거 받아야 돼요", "160501_01.0051.wav": "자전거 너무좋은 운동이 긴 하지만", "160501_01.0052.wav": "위험하게 타면 너무 위험한 운동일 수 있으니까", "160501_01.0053.wav": "이것저것 잘 점검하시 9", "160501_01.0054.wav": "헬멧 이런 거 있어야 하는 거 아닌가요", "160501_01.0055.wav": "아무튼 언제나 안전하게 운동하십시오", "160501_01.0056.wav": "7138 님 음", "160501_01.0057.wav": "썸녀와 같은 직장에서 일하고 있어요", "160501_01.0058.wav": "사적으로는 너무 좋은 데", "160501_01.0059.wav": "공적으로는 너무 많이 부딪히네요", "160501_01.0060.wav": "지금 일단 제 마음을 접자라고 다짐했는데", "160501_01.0061.wav": "왜 잘 안 되네요 유유", "160501_01.0062.wav": "음", "160501_01.0063.wav": "이거는 뭐 결판은 지워야지 썸 탈 때가 가장 좋을 텐데 어떻게 썸을 타다 뭡니까", "160501_01.0064.wav": "썸 타는 감정은", "160501_01.0065.wav": "포기할래 포기가 안 되는 감정 아닌가요", "160501_01.0066.wav": "야 뭐", "160501_01.0067.wav": "결판을 지어서 어떻게 되는지 또 문자 주십시오", "160501_01.0068.wav": "응 두산 알아볼까요", "160501_01.0069.wav": "7455 님", "160501_01.0070.wav": "회사 동료에게 볼륨을 높여요 홍보했어요", "160501_01.0071.wav": "완전 꿀 잼이라고 들어 보라고 했는데", "160501_01.0072.wav": "눈 하나 깜짝 안 하길래 바로 다시 듣기로 들려줬어요", "160501_01.0073.wav": "그랬더니 목소리도 너무 좋고 방송도 잘 한다고 하네요 저 잘 했죠", "160501_01.0076.wav": "아 싫다는데 왜 굳이 그랬어", "160501_02.0000.wav": "검색어로 불륜 가족들 사연 만나 보겠습니다 볼륨 검색어", "160501_02.0001.wav": "오늘의 검색어는", "160501_02.0002.wav": "너무였어요 너무 계속해서 사연 소개해드릴게요", "160501_02.0003.wav": "7526 님", "160501_02.0004.wav": "소개팅 날 만나기로 했는데 김치를 못 먹는다고 하네요", "160501_02.0005.wav": "저는 김치를 너무 좋아하고 매운 것도 잘 먹는데", "160501_02.0006.wav": "아직 만나지도 않았는데 이 사람과는 인연이 아닌가 봐요 유유", "160501_02.0007.wav": "당분간도 율이랑 함께 솔로 생활을 즐겨야겠어요", "160501_02.0008.wav": "맞아요 그게 맞는 거예요 솔로 생활 즐기세요", "160501_02.0009.wav": "왜냐하면", "160501_02.0010.wav": "맘에 안 그렇게 썩 들지 않았던 거지 뭐 김치 가지고", "160501_02.0011.wav": "우리 인연이 아닌가 보다 하겠어요 마음에 든다면", "160501_02.0012.wav": "이거는 다 고치지않은 거 같아요", "160501_02.0014.wav": "두 8134님", "160501_02.0015.wav": "일주일 내내 야근하고 있어요 유유", "160501_02.0016.wav": "그래도 사무실에서 라디오 들을 수 있게 허락해 주셔서 듣고 있어요", "160501_02.0017.wav": "오늘도 유지 너무 기다렸어요", "160501_02.0018.wav": "잉 고마워요 이렇게 날 기다려 주다니", "160501_02.0019.wav": "그런데 사무실에서 라디오 듣는 것도 허락받고 들어야 합니까", "160501_02.0020.wav": "군대가 그렇잖아요 군대 가모", "160501_02.0021.wav": "어느 때가 되면 라디오를 들을 수 있고 뭐 허락해 주고 그런 게 있다고 하더라고요", "160501_02.0022.wav": "그래서 제가 진짜 시작 모든 우리나라 군대에서 다라 즈를 들을 수 있으면 좋겠다라는 생각 많이 했었는데", "160501_02.0024.wav": "그래요 아무튼 저랑 함께 합시다 두 시간 동안", "160501_02.0025.wav": "do you", "160501_02.0026.wav": "이종권 씨 오고 기숙사 생활 중인 고등학생입니다", "160501_02.0027.wav": "집이", "160501_02.0028.wav": "엄마가 너무 그리워요", "160501_02.0029.wav": "그래도 볼륨 들으니 힐링이 되네요", "160501_02.0030.wav": "휴대폰으로 라디오 들을 수 있다는 걸 이제 알았을까요", "160501_02.0031.wav": "아 그랬어요 하긴 중권 씨 제가 만약 디제이가 아니었으면 저도 몰랐을 거예요 저두", "160501_02.0032.wav": "뭐 이런 문명의 혜택을 받고 사는 사람이 아니라서", "160501_02.0033.wav": "뭐가 얼마나 편리한 게 그런 거 잘 모르거든요", "160501_02.0034.wav": "그런데", "160501_02.0035.wav": "웬만한 건 휴대폰 어플로 다 있는 거 같아 요즘에는", "160501_02.0036.wav": "요즘 세상엔 진짜", "160501_02.0037.wav": "음 또 한 다정 씨", "160501_02.0038.wav": "저랑 일 년 내내 붙어 다니던 오빠가 있었는데", "160501_02.0039.wav": "오빠가 다른 여자를 만났는데요", "160501_02.0040.wav": "뭔가 팽 당한 느낌이에요 너무 슬퍼요 유유", "160501_02.0042.wav": "사랑은 타이밍이 저 대성 씨", "160501_02.0043.wav": "이렇게 옆에 누가 없을 때 그때 딱 내 것 만들어 써야죠", "160501_02.0044.wav": "이 어떻게 마음 너무 아프겠어요", "160501_02.0045.wav": "근데 좀", "160501_02.0046.wav": "궁금하다 국 자세하게 궁금한데요 다정 씨", "160501_02.0047.wav": "왜 내 남자 친구도 아닌", "160501_02.0048.wav": "오빠랑 남자랑 일 년 내내 붙어 다녔을까", "160501_02.0050.wav": "a 아무튼 그 오빠가 나쁜 남자가", "160501_02.0051.wav": "아니길 바라고요", "160501_02.0052.wav": "다정 씨가 상처받지 않아 길 바래요", "160501_02.0053.wav": "자 이번주 볼륨 검색어는 여기까지입니다 어떤 이야기든지 잘 챙 겨 듣고 있으니까요", "160501_02.0054.wav": "많은 얘기 나눠 주세요", "160501_02.0055.wav": "남자는 노래 한 곡 듣고 올게요 검입니다 you are my everything", "160501_03.0000.wav": "검색어로 불륜 가족들 사연 만나 보겠습니다 볼륨 검색어", "160501_03.0001.wav": "오늘의 검색어는", "160501_03.0002.wav": "너무였어요 너무 계속해서 사연 소개해드릴게요", "160501_03.0003.wav": "7526 님", "160501_03.0004.wav": "소개팅 날 만나기로 했는데 김치를 못 먹는다고 하네요", "160501_03.0005.wav": "저는 김치를 너무 좋아하고 매운 것도 잘 먹는데", "160501_03.0006.wav": "아직 만나지도 않았는데 이 사람과는 인연이 아닌가 봐요 유유", "160501_03.0007.wav": "당분간도 율이랑 함께 솔로 생활을 즐겨야겠어요", "160501_03.0008.wav": "맞아요 그게 맞는 거예요 솔로 생활 즐기세요", "160501_03.0009.wav": "왜냐하면", "160501_03.0010.wav": "맘에 안 그렇게 썩 들지 않았던 거지 뭐 김치 가지고", "160501_03.0011.wav": "우리 인연이 아닌가 보다 하겠어요 마음에 든다면", "160501_03.0012.wav": "이거는 다 고치지않은 거 같아요", "160501_03.0014.wav": "두 8134님", "160501_03.0015.wav": "일주일 내내 야근하고 있어요 유유", "160501_03.0016.wav": "그래도 사무실에서 라디오 들을 수 있게 허락해 주셔서 듣고 있어요", "160501_03.0017.wav": "오늘도 유지 너무 기다렸어요", "160501_03.0018.wav": "잉 고마워요 이렇게 날 기다려 주다니", "160501_03.0019.wav": "그런데 사무실에서 라디오 듣는 것도 허락받고 들어야 합니까", "160501_03.0020.wav": "군대가 그렇잖아요 군대 가모", "160501_03.0021.wav": "어느 때가 되면 라디오를 들을 수 있고 뭐 허락해 주고 그런 게 있다고 하더라고요", "160501_03.0022.wav": "그래서 제가 진짜 시작 모든 우리나라 군대에서 다라 즈를 들을 수 있으면 좋겠다라는 생각 많이 했었는데", "160501_03.0024.wav": "그래요 아무튼 저랑 함께 합시다 두 시간 동안", "160501_03.0025.wav": "do you", "160501_03.0026.wav": "이종권 씨 오고 기숙사 생활 중인 고등학생입니다", "160501_03.0027.wav": "집이", "160501_03.0028.wav": "엄마가 너무 그리워요", "160501_03.0029.wav": "그래도 볼륨 들으니 힐링이 되네요", "160501_03.0030.wav": "휴대폰으로 라디오 들을 수 있다는 걸 이제 알았을까요", "160501_03.0031.wav": "아 그랬어요 하긴 중권 씨 제가 만약 디제이가 아니었으면 저도 몰랐을 거예요 저두", "160501_03.0032.wav": "뭐 이런 문명의 혜택을 받고 사는 사람이 아니라서", "160501_03.0033.wav": "뭐가 얼마나 편리한 게 그런 거 잘 모르거든요", "160501_03.0034.wav": "그런데", "160501_03.0035.wav": "웬만한 건 휴대폰 어플로 다 있는 거 같아 요즘에는", "160501_03.0036.wav": "요즘 세상엔 진짜", "160501_03.0037.wav": "음 또 한 다정 씨", "160501_03.0038.wav": "저랑 일 년 내내 붙어 다니던 오빠가 있었는데", "160501_03.0039.wav": "오빠가 다른 여자를 만났는데요", "160501_03.0040.wav": "뭔가 팽 당한 느낌이에요 너무 슬퍼요 유유", "160501_03.0042.wav": "사랑은 타이밍이 저 대성 씨", "160501_03.0043.wav": "이렇게 옆에 누가 없을 때 그때 딱 내 것 만들어 써야죠", "160501_03.0044.wav": "이 어떻게 마음 너무 아프겠어요", "160501_03.0045.wav": "근데 좀", "160501_03.0046.wav": "궁금하다 국 자세하게 궁금한데요 다정 씨", "160501_03.0047.wav": "왜 내 남자 친구도 아닌", "160501_03.0048.wav": "오빠랑 남자랑 일 년 내내 붙어 다녔을까", "160501_03.0050.wav": "a 아무튼 그 오빠가 나쁜 남자가", "160501_03.0051.wav": "아니길 바라고요", "160501_03.0052.wav": "다정 씨가 상처받지 않아 길 바래요", "160501_03.0053.wav": "자 이번주 볼륨 검색어는 여기까지입니다 어떤 이야기든지 잘 챙 겨 듣고 있으니까요", "160501_03.0054.wav": "많은 얘기 나눠 주세요", "160501_03.0055.wav": "남자는 노래 한 곡 듣고 올게요 검입니다 you are my everything", "160501_04.0000.wav": "kbs cool fm 유인나의 볼륨을 높여요 2부가 시작됐습니다", "160501_04.0001.wav": "평소에 미처 다 나누지 못한 이야기들 더 만나 볼게요", "160501_04.0002.wav": "못한 이야기", "160501_04.0003.wav": "음 공이 이공님", "160501_04.0004.wav": "결혼을 앞두고 있는 예비 부부예요 음 어쩌다 보니 5주 전", "160501_04.0005.wav": "아기 천사가 찾아왔네요", "160501_04.0006.wav": "기쁘면서 걱정도 되고 기분이 묘하네요 아 그리고 태명 짓기가 이렇게 힘든 거였나요", "160501_04.0007.wav": "기발한 3명이 도저히 생각나지 않아요 좀 도와주세요", "160501_04.0008.wav": "일단 축하 드립니다", "160501_04.0010.wav": "어", "160501_04.0011.wav": "좀 일찍왔네요 아기 천사가 뭐야", "160501_04.0012.wav": "그것도 축하 드려요 결혼하시는 것도 축하 드리고요", "160501_04.0014.wav": "응 태명 태명엇 저희 언니가 결혼해서 아기를 둘 다왔어요 근데", "160501_04.0015.wav": "어 태명 되게 고민하더라고요", "160501_04.0016.wav": "근데", "160501_04.0017.wav": "어", "160501_04.0018.wav": "저희 그", "160501_04.0019.wav": "흥부 성이 호시예요 그래서", "160501_04.0020.wav": "호두로 제가 첫째는 호드로", "160501_04.0021.wav": "사실 저는 태명이 아니라 이름으로 강력히 추천을 했는데 어의 언니가 호두로 살게 할 수 없다", "160501_04.0022.wav": "아 전 지금도 언니는 후회할 거라고 말 하지만", "160501_04.0023.wav": "암튼 호두가 태명이었어요", "160501_04.0024.wav": "정말 단단하고", "160501_04.0025.wav": "깨지지 않는 아이가 나왔고요", "160501_04.0026.wav": "그리고 둘째는 순 둥이었어요 왜냐 하면 첫째가 너무 단단하고 깨지지 않은 나이 라서", "160501_04.0027.wav": "언니가 좀 순한 아이가 나왔으면 좋겠다 해서 둘째 순둥이라고 했는데", "160501_04.0028.wav": "와", "160501_04.0029.wav": "진짜 순해요", "160501_04.0030.wav": "아 진짜 첫째야 이기는", "160501_04.0031.wav": "엄청 떼쓰고 울고 그랬거든요 2기 때부터", "160501_04.0032.wav": "지금 대기 지만", "160501_04.0033.wav": "근데 둘째는 짐 태어난 지 1달 됐는데", "160501_04.0034.wav": "어우 울지도 않고", "160501_04.0035.wav": "응 한마디 울고 밥 주면 뚝 응용해서 기저귀 갈아주면 좀 그러더라고요", "160501_04.0036.wav": "그냥 그 미국님이", "160501_04.0037.wav": "응 바라는 거 있잖아 씩씩했으면 좋겠다 하면 씩씩이", "160501_04.0038.wav": "튼튼했으면 좋겠담 튼튼이", "160501_04.0039.wav": "뭐 그런 걸로 쉬운 걸로 귀엽게 한 번 지어 보세요", "160501_04.0041.wav": "응 케어", "160501_04.0042.wav": "영어를 소개해드릴까 김수경 씨", "160501_04.0043.wav": "고등학생인 동생이 과외를 받고 있어요 그런데 옆모습 만 보고는가 선생님인 줄 모르고", "160501_04.0044.wav": "어 친구 않네", "160501_04.0045.wav": "뭐라고 말한 거예요", "160501_04.0046.wav": "얼굴을 돌린 순간 너무 당황해서 서로 헛웃음 만 나왔네요", "160501_04.0047.wav": "동생 잘 가르쳐 주시겠죠 유유 유", "160501_04.0048.wav": "오도 잘 가르쳐 주지 않을까요", "160501_04.0049.wav": "과외 선생님이 아무리 어려도 대학생일 거 아니에요", "160501_04.0050.wav": "또 모르지 애교 말 수도 있고 모", "160501_04.0051.wav": "그런데 친구 하냐면은", "160501_04.0052.wav": "고등학생으로 본 거니까 우리 그런 것도 다 기분좋잖아요", "160501_04.0054.wav": "음 178님", "160501_04.0055.wav": "슈퍼에 갔다 왔는데 집에 들어가려고 하는 순간", "160501_04.0056.wav": "아무리 생각해도 비밀번호가 안 떠오르는 거예요", "160501_04.0057.wav": "결국 엄마 전에 비밀번호 알았는데요", "160501_04.0058.wav": "또 까먹을까 봐 바로 휴대폰 메모장에 적어놨어요", "160501_04.0059.wav": "유유", "160501_04.0060.wav": "저도 가끔 그래요", "160501_04.0061.wav": "그리구", "160501_04.0062.wav": "이형관 요즘은 아파트마다 들어 있고 1층 입구에서부터 비밀번호가 있잖아요 아니면 카드키 있거나 막 그래야 하잖아요", "160501_04.0063.wav": "그런데 왜", "160501_04.0064.wav": "카드키를 안 가져온 날", "160501_04.0065.wav": "비밀번호를 누르는 너는 되게 복잡해요 무슨", "160501_04.0066.wav": "별표를 누르고 뭐 흡수를 누르고 모터 별을 별을 모두 한 번 누나 두 분으로 나하고", "160501_04.0067.wav": "물도 못 누르고", "160501_04.0068.wav": "마지막에 돔을 누르고 막 그래요", "160501_04.0069.wav": "cruel 사람들이 더 귀엽더라고요 도 나쁘고", "160501_04.0070.wav": "저는 그거 못 외워서 그럴 때는 그냥", "160501_04.0071.wav": "er 호출 눌러서 열어 주십시오 하거든요", "160501_04.0072.wav": "그럴 수도 잇점", "160501_04.0074.wav": "자랑 노래를 들을까요", "160501_04.0075.wav": "이 노래 띄워드릴게요 7367님이 신청곡", "160501_04.0076.wav": "에피톤 프로젝트 선인장", "160501_04.0077.wav": "예스코는 혜민 씨의 신청곡입니다 마골피의 비행 소녀", "160501_04.0078.wav": "자 볼륨 가족들의 못다 한 이야기 만나 보겠습니다", "160501_04.0079.wav": "다 들 사연 소개해드릴게요", "160501_04.0080.wav": "큰일이네요 장진원 씨 진짜 애가 타겠어요", "160501_04.0081.wav": "유지", "160501_04.0082.wav": "여자친구가 헤어지자고 하면서 연락 수단을 다 차단했는데", "160501_04.0083.wav": "제가 어떻게 행동해야 하나요", "160501_04.0084.wav": "깔끔하게 헤어지는 게 답인 가요", "160501_04.0085.wav": "응", "160501_04.0086.wav": "어떻게 해요 이거 진짜", "160501_04.0087.wav": "화가 단단히 난 거 아닐까 지난씩 써서 하셨어요 아니면", "160501_04.0088.wav": "싸우지도 않았는데 그냥 차분하게 그만 만나는 게 좋겠어 라고 하던가요", "160501_04.0089.wav": "전자면 화를 내는 거고요 후자면 정말 다시 만날 마음이 없는 겁니다", "160501_04.0090.wav": "왜냐하면 조금 미련이 있구나 다시 만날 마음이 있으면", "160501_04.0091.wav": "내가 연락을 안 해도 연락이 올 수는 있도록 열어 준단 말이죠", "160501_04.0092.wav": "이렇게 연락 수단다 차단했다는 건", "160501_04.0094.wav": "진원 씨임", "160501_04.0095.wav": "깨끗하게 포기를", "160501_04.0096.wav": "하시는게 좋을 거 같은데요", "160501_04.0097.wav": "연락 안 하면 궁금해져서 저쪽에서 연락이 안 좋을 수도 있다", "160501_04.0098.wav": "근데 만약에 싸운 거다", "160501_04.0099.wav": "내가 뭘 단단히 잘못 했다", "160501_04.0100.wav": "그러면 진짜 어떻게 든", "160501_04.0101.wav": "a 좀 찾아 내든지 아님 경비실 같은데 편지를 하던지", "160501_04.0102.wav": "어우 너무 작은 선물이라든지 바뀌어 보세요", "160501_04.0104.wav": "3647 님", "160501_04.0105.wav": "남편이랑 면세점 가서 돈 쓰고 왔는데 돈을 벌고 오니 기분은 뭐죠", "160501_04.0106.wav": "연세정 들 이래요 크크크", "160501_04.0107.wav": "와 나랑 반대다", "160501_04.0108.wav": "정확히 안 돼요 면세점에서 살 계획이 없었는데", "160501_04.0109.wav": "뭐야", "160501_04.0110.wav": "모르겠어", "160501_04.0111.wav": "타요 막 이것저것 화장품도 사고 뭐 말도 안되는 쓰지도 않는 썬글라스도 사고 그래요", "160501_04.0112.wav": "그때는 돈을와 돈 벌었다 돈 벌었어", "160501_04.0114.wav": "내가 이걸 왜 사치", "160501_04.0115.wav": "어쨌든 지출이잖아요", "160501_04.0116.wav": "그래서 존 3647 님이랑 정확히 딱 반대의 기분을 느껴요", "160501_04.0118.wav": "사실 나 은거 같네요", "160501_04.0120.wav": "응 두 3607님", "160501_04.0121.wav": "남편한테 빨래가 안 마른다고 외출하면서 제습기 좀 들려나 했는데", "160501_04.0122.wav": "가습기를 틀어서 완전 열대 지방을 만들어 놨어요", "160501_04.0123.wav": "여름도 아닌데 너무 스페어", "160501_04.0124.wav": "응", "160501_04.0125.wav": "또 빨래가 있으면 톡톡히 가습기 역할을 하는데 그렇군요", "160501_04.0126.wav": "근데 제습기가 원래 집집마다 있어", "160501_04.0127.wav": "좋은 주석이 없는데 저희 집에", "160501_04.0128.wav": "계속이 좋은가", "160501_04.0129.wav": "아진 듯", "160501_04.0130.wav": "이런 거 무드 오 제습기 오늘부터 관심 갖기 시작하고 수일 안에 사드릴 겁니다 야", "160501_04.0132.wav": "그럼 노래 듣고 올게요 다이나믹 듀오 featuring lydia 베게", "160501_05.0000.wav": "선생님 동생이 과외를 받고 있어요 그런데 옆모습 만 보고는 선생님인 줄 모르고", "160501_05.0001.wav": "어 친구 않네", "160501_05.0002.wav": "뭐라고 말한 거예요", "160501_05.0003.wav": "얼굴을 돌린 순간 너무 당황해서 서로 헛웃음 만 나왔네요", "160501_05.0004.wav": "동생 잘 가르쳐 주시겠죠 유유", "160501_05.0005.wav": "오도 잘 가르쳐 주지 않을까요", "160501_05.0006.wav": "과외 선생님이 아무리 어려도 대학생일 거 아니에요", "160501_05.0007.wav": "또 모 르 씨 애교 말 수도 있고 모", "160501_05.0008.wav": "근데 친구 하냐면은", "160501_05.0009.wav": "고등학생으로 본 거니까 우리 그런 것도 다 기분좋잖아요", "160501_05.0012.wav": "음 178님", "160501_05.0013.wav": "슈퍼에 갔다 왔는데 집에 들어가려고 하는 순간", "160501_05.0014.wav": "아무리 생각해도 비밀번호가 안 떠오르는 거예요", "160501_05.0015.wav": "결국 엄마 전에 비밀번호 알았는데요", "160501_05.0016.wav": "또 까먹을까 봐 바로 휴대폰 메모장에 적어놨어요", "160501_05.0017.wav": "유유", "160501_05.0018.wav": "저도 가끔 그래요", "160501_05.0019.wav": "그리구", "160501_05.0020.wav": "이형관 요즘은 아파트마다 들어 있고 1층 입구에서부터 비밀번호가 있잖아요 아니면 카드키 있거나 막 그래야 하잖아요", "160501_05.0021.wav": "네에", "160501_05.0022.wav": "카드키를 안 가져온 날", "160501_05.0023.wav": "비밀번호를 누르는 너는 되게 복잡해요 무슨", "160501_05.0024.wav": "별표를 누르고 뭐 홋수를 누르고 뭐 또 별 별을 모두 한 번 누나 두 분 누나하고", "160501_05.0025.wav": "물도 못 누르고", "160501_05.0026.wav": "마지막에 돔을 누르고 막 그래요", "160501_05.0027.wav": "cruel 사람들이 웃더라고요 더 나쁘고", "160501_05.0028.wav": "저는 그거 못 외워서 그럴 때는 그냥", "160501_05.0029.wav": "er 호출 눌러서 열어 주십시오 하거든요", "160501_05.0030.wav": "그럴 수도 잇점", "160501_05.0032.wav": "자랑 노래를 들을까요", "160501_05.0033.wav": "이 노래 띄워 드릴게요 7367님이 신청곡", "160501_05.0034.wav": "에피톤 프로젝트 선인장", "160501_05.0035.wav": "예스코는 혜민 씨의 신청곡입니다 마골피의 비행 소녀", "160501_05.0036.wav": "자 볼륨 가족들의 못다 한 이야기 만나 보겠습니다", "160501_05.0037.wav": "하트 사연 소개해드릴게요", "160501_05.0038.wav": "큰일이네요 장진원 씨 진짜 애가 타겠어요", "160501_05.0039.wav": "유지", "160501_05.0040.wav": "여자친구가 헤어지자고 하면서 연락 수단을 다 차단했는데", "160501_05.0041.wav": "제가 어떻게 행동해야 하나요", "160501_05.0042.wav": "깔끔하게 헤어지는 게 답인 가요", "160501_05.0043.wav": "응", "160501_05.0044.wav": "어떻게 해요 이거 진짜", "160501_05.0045.wav": "화가 단단히 난 거 아닐까 지난씩 싸웠어요 하셨어요 아니면", "160501_05.0046.wav": "싸우지도 않았는데 그냥 차분하게 그만 만나는 게 좋겠어 라고 하던가요", "160501_05.0047.wav": "전자면 화를 내는 거고요 후자면 정말 다시 만날 마음이 없는 겁니다", "160501_05.0048.wav": "왜냐하면 조금 미련이 있구나 다시 만날 마음이 있으면", "160501_05.0049.wav": "내가 연락을 안 해도 연락이 올 수는 있도록 열어 준단 말이죠", "160501_05.0050.wav": "이렇게 연락 수단다 차단했다는 건", "160501_05.0052.wav": "진원 씨임", "160501_05.0053.wav": "깨끗하게 포기를", "160501_05.0054.wav": "하시는게 좋을 거 같은데요", "160501_05.0055.wav": "연락 안 하면 궁금해져서 저쪽에서 연락이 안 좋을 수도 있다", "160501_05.0056.wav": "근데 만약에 싸운 거다", "160501_05.0057.wav": "내 갑옷 단단히 잘못했다", "160501_05.0058.wav": "그러면 진짜 어떻게 든", "160501_05.0059.wav": "a 좀 찾아 내든지 아님 경비실 같은데 편지를 하던지", "160501_05.0060.wav": "어우 너무 작은 선물이라든지 바뀌어 보세요", "160501_05.0062.wav": "3647 님", "160501_05.0063.wav": "남편이랑 면세점 가서 돈 쓰고 왔는데 돈을 벌고 오니 기분은 뭐죠", "160501_05.0064.wav": "연세정 들 이래요 크크크", "160501_05.0065.wav": "와 나랑 반대다", "160501_05.0066.wav": "정확히 안 돼요 면세점에서 살 계획이 없었는데", "160501_05.0067.wav": "뭐야", "160501_05.0068.wav": "모르겠어", "160501_05.0069.wav": "차마 이것저것 화장품도 사고 뭐 말도 안되는 쓰지도 않는 썬글라스도 사고 그래요", "160501_05.0070.wav": "그때는 돈을와 돈 벌었다 돈 벌었어", "160501_05.0071.wav": "이게 훨씬 싸네 이러고 사는데", "160501_05.0072.wav": "내가 이걸 왜 샀지", "160501_05.0073.wav": "어쨌든 지출이잖아요", "160501_05.0074.wav": "그래서 존 3647 님이랑 정확히 딱 반대의 기분을 느껴요", "160501_05.0076.wav": "사실 나 은거 같네요 흑", "160501_05.0078.wav": "음두 3607님", "160501_05.0079.wav": "남편한테 빨래가 안 마른다고 외출하면서 제습기 좀 들려나 했는데", "160501_05.0080.wav": "가습기를 틀어서 완전 열대 지방을 만들어 놨어요", "160501_05.0081.wav": "여름도 아닌데 너무 스페어", "160501_05.0082.wav": "응", "160501_05.0083.wav": "또 빨래가 있으면 톡톡히 가습기 역할을 하는데 그렇군요", "160501_05.0084.wav": "근데 제습기가 원래 집집마다 있어요", "160501_05.0085.wav": "좋은 제습기 없는데 저희 집에", "160501_05.0086.wav": "계속이 좋은가", "160501_05.0087.wav": "어제인 듯", "160501_05.0088.wav": "이런 것으 오 제습기 오늘부터 관심 갖기 시작하고 수일 안에 사드릴 겁니다 야", "160501_05.0090.wav": "그럼 노래 듣고 올게요 다이나믹 듀오 featuring lydia paek에 있어줘", "160501_05.0091.wav": "음 조금 식혀줍니다 와아 여요아아 와아", "160501_05.0093.wav": "1551 임", "160501_05.0094.wav": "다름", "160501_05.0095.wav": "아침저녁으로 수분 크림도 바르고 잘 관리해준다고 생각했는데", "160501_05.0096.wav": "어느 순간 피부가 너무 칙칙해 보이는 거 있죠", "160501_05.0097.wav": "지금부터라도 1일 팩 해서 피부미인이 되고 싶어요", "160501_05.0098.wav": "음", "160501_05.0099.wav": "피부 톤은 근데 좀 타고나는 거 같아요", "160501_05.0100.wav": "피부관리 하나도 안 해도 정말 하얗고 분홍색이고 막 그런 피부들도 있잖아요", "160501_05.0101.wav": "아무튼 관리하는 만큼도 좋아지긴 하니까", "160501_05.0102.wav": "열심히 관리하세요 중요한 건", "160501_05.0103.wav": "이 건조하지 않게 수분 크림 그렇게 수준이 발라 주시는 거랑 자외선 차단 관리 잘 해 주시고요", "160501_05.0104.wav": "무엇보다 이너 뷰티가 중요하다는 거 잘 먹어야 합니다", "160501_05.0105.wav": "으", "160501_05.0106.wav": "아 김미영 씨", "160501_05.0107.wav": "우리 딸 시집 가지마", "160501_05.0108.wav": "정말이지 날마다 시집가라고 잔소리하는 엄마 때문에 너무 힘들어요", "160501_05.0109.wav": "응", "160501_05.0110.wav": "근데 이", "160501_05.0111.wav": "말도 스트레스일 거 같은데요 안 그래도 날씨가 생각하고 남자 친구도 없는데 우리 딸 시집 가지마", "160501_05.0112.wav": "똑같이 될 거야 시집가 이거랑 똑같이 드릴 겁니다", "160501_05.0114.wav": "듣고 싶은 말 소개된 분들 모두 모두 선물 드릴게요", "160501_05.0115.wav": "이번에는 여기까지 구요 일요일 삼 대는 이지영 씨 그리고 권정열 씨와 함께", "160501_05.0116.wav": "볼륨 가족들의 일기장 만나는 시간이죠", "160501_05.0117.wav": "마이 올드 다이어리가 준비되어 있습니다", "160501_05.0118.wav": "자 그럼 노래 한 곡 듣고 저는 3부로 돌아올게요 입어 극입니다", "160501_05.0119.wav": "이 적의", "160501_05.0120.wav": "20년이 지난 뒤", "160501_05.0121.wav": "ncs 찾아요 만에 수술 방에서 나오느냐 라고 있네요", "160501_05.0122.wav": "우리 볼륨 가족들을 위한 선물을 소개해 드리겠습니다", "160501_05.0123.wav": "여기에 명가 유닉스전자에서 스타일러", "160501_05.0124.wav": "한우 곱창 요리 전문점 곱창 고에서 매장 이용권", "160501_05.0125.wav": "블라인드 테스트 라벨 영 화장품에서 모공 팩", "160501_05.0126.wav": "콩스타일에서 여성 의류 쇼핑몰 이용권", "160501_05.0127.wav": "민감한 피부엔 이너 페이스에서 퍼퓸 헤어 오일 세트", "160501_05.0128.wav": "개인 생활의 비밀 시크릿 키에서 소 패스트 샴푸", "160501_05.0129.wav": "하이 포크에서 새우젓으로 맛을 낸 햄", "160501_05.0130.wav": "화장품 전문 기업 유리 그래서 화장품 세트", "160501_05.0131.wav": "주식회사 블링크 래쉬에서 속눈썹 증모제와 눈썹 세트", "160501_05.0132.wav": "밥 미 키 안경에서 반 미키 선글라스", "160501_05.0133.wav": "빙그레 꽃게랑 불 짬뽕", "160501_05.0134.wav": "감성 조명의 원룸에서 인디언 스탠드", "160501_05.0135.wav": "프랑스 원료 그대로 뒤로 화장품에서 파운데이션", "160501_05.0136.wav": "여대생 가방 맥 끌라니에서 핸드백", "160501_05.0137.wav": "다양한 디자인 모모 케이스에서 나만의 핸드폰 케이스", "160501_05.0138.wav": "주식회사 사라센에서 폴리쉬 6종", "160501_05.0139.wav": "스페셜 홈케어 브랜드 세렌디피티에서 쇼핑몰 이용권", "160501_05.0140.wav": "내일 전문 브랜드 특 질에서 셀프 젤 네 일 패키지", "160501_05.0141.wav": "황정음의 영 캐주얼 무자크에서 의료 상품권", "160501_05.0142.wav": "1000원 아메리카노 성공 신화 커피에 반하 다에서 커피 교환권", "160501_05.0143.wav": "stylish spa브랜드 아 클림 콜렉트에서 의류 상품권", "160501_05.0144.wav": "내츄럴 프리미엄 아이스크림 나뚜루 팝에서 아이스크림 케이크", "160501_05.0145.wav": "후불제 여행 전문 기업 아만 투어에서 여행 상품권을 드립니다", "160501_05.0147.wav": "볼륨 홈페이지 선곡표 확인하시구요 당첨되신 분들은 당첨 글 게시판에 주소랑 연락처 알려 주세요", "160501_05.0148.wav": "1 볼륨을 높여요 일요일 코너 마이 올드 다이어리 광고도 있고 이시영 씨의 권정열 씨와 함께 불륜 가족들이 보내 주신 일기장 대 만나볼게요", "190429_01.0001.wav": "이 자리에서 여러분들과 함께 하게 됐어요", "190429_01.0002.wav": "월화수목 사일 간", "190429_01.0003.wav": "어이 저거 해요 생방송", "190429_01.0004.wav": "생방송으로 여러분과 함께 할 거니까요", "190429_01.0005.wav": "기뻐해 주세요", "190429_01.0006.wav": "아니 이게 원래 디제이가 자리를 비우면 청취자 분들은 굉장히 불안해 하세요 막 불안해 하시고", "190429_01.0007.wav": "우리 우리 수연이는 어디로 간겨 보고 싶다 그러는데", "190429_01.0008.wav": "그래도 저는", "190429_01.0009.wav": "남 보다는 남이 아니잖아요 그저 무슨 말인지 알겠죠 되게 남 보다는 제가 좀 덜 아니니까", "190429_01.0010.wav": "이제라도 가족이지 그러니까 여러분도 낯설게 생각하지 마시고", "190429_01.0011.wav": "우리 오랜만에 재밌는 얘기 많이 많이 나눠요", "190429_01.0012.wav": "아", "190429_01.0013.wav": "떨려요", "190429_01.0015.wav": "뭐냐 이 자리 떨린다 그리고 바뀌었어요", "190429_01.0016.wav": "원래 저쪽 옆에 스튜디오였는데 바뀌어 가지고 전에 논에 와서", "190429_01.0017.wav": "엘레베이터부터 잘못 해서 눌렀는데 이게 조청에 안 가서 내려서 반대쪽으로 타고 그리고 내가 가기로 했던 10번 방으로 갔는데 거기는 안 계셔 가지고", "190429_01.0018.wav": "10이요 라고 하신 것", "190429_01.0019.wav": "12일을 말씀하신 식으로 들어서", "190429_01.0020.wav": "막 매니저들이랑 우르르와 해면서 다니다가", "190429_01.0021.wav": "지금도 이렇게 왔는데 옆방으로 들어갔어요 제가", "190429_01.0023.wav": "이게 많이 바뀌었네요", "190429_01.0024.wav": "그래 수지 씨의 가요 광장 제가 게스트로 나왔을 때도 여기 앉아서 들었거든요 수지랑", "190429_01.0025.wav": "여기네요 여기 아 이 차", "190429_01.0026.wav": "엄청 떨면서왔는데", "190429_01.0027.wav": "어", "190429_01.0028.wav": "많이 안 떨고 잘 할 수 있을 것 같다 왜냐면", "190429_01.0029.wav": "저를", "190429_01.0030.wav": "반겨 주시는 분들이 익숙한 분들이 굉장히 많이 계세요 일단 오픈 스튜디오에도", "190429_01.0032.wav": "끝날 때까지 있을 거예요", "190429_01.0033.wav": "가서 봐서 알았어요 잘 할게요 잘 할게요 10시까지 같이 해요 알겠죠 뭐야", "190429_01.0034.wav": "이렇게 막 직접 오기까지 하시고 그리고 지금 문자 창에 만 봐도 익숙한 이름들이 정말 정말 많아요", "190429_01.0035.wav": "진짜많다 다 읽고 싶은데", "190429_01.0036.wav": "모델과 요", "190429_01.0040.wav": "오 칠 칠 사장님이 1000일이 넘도록 기다렸습니다 오늘도 보고 싶었어요 잘 지냈어요 하셨는데", "190429_01.0041.wav": "내일 잘 지냈어요 7744님", "190429_01.0042.wav": "잘 지내고요", "190429_01.0043.wav": "박영미님도 그리웠던 목소리 유유", "190429_01.0044.wav": "작전 일리 눈물 난다 쩜쩜쩜", "190429_01.0046.wav": "아찬 찬일 씨 약간 요정도 감정은 아니니 하", "190429_01.0047.wav": "이따가 한 번 다 같이 슬픈 분위기를 몰아가 봐요 우리 다 같이 우는 분위기로", "190429_01.0048.wav": "감성적인 타임도 한 번 갖도록 해 보고", "190429_01.0049.wav": "김혜민 씨가 이 나긋나긋한 목소리는 누구인가 했는데 유인나 언니네요", "190429_01.0050.wav": "캬 반가워요 목소리 너무 이쁜 이웃이라고 하셨어요", "190429_01.0051.wav": "감사합니다", "190429_01.0052.wav": "어 뚜", "190429_01.0053.wav": "강신영 씨가 고2 때 막 방이었는데 어느새 대학생이 되었고", "190429_01.0054.wav": "지금은 휴학하고 유럽 여행에 와 있어요 25", "190429_01.0055.wav": "지금은 독일 3년 너무 빨리 갔어요 크 크", "190429_01.0056.wav": "3년 빨리 갔어요", "190429_01.0057.wav": "이인 유인나의 볼륨이 없는 데도", "190429_01.0058.wav": "그러면 진짜 잘 했다 잘 하셨어요", "190429_01.0059.wav": "아주 바쁘게", "190429_01.0060.wav": "우와 대학생도 되고 유럽 여행도 가고", "190429_01.0061.wav": "내가 그 나이 때 못했던 것이 낚시를 하고 계시네요", "190429_01.0062.wav": "멋지다", "190429_01.0063.wav": "아니 정말 요즘에도 여러분이 주신 편지들 가끔 열어 보고 또 지금도 저한테 편지 많이 보내 주시거든요", "190429_01.0064.wav": "아니", "190429_01.0065.wav": "사실 말이 아니야", "190429_01.0066.wav": "왜 많이 안 보내줘요", "190429_01.0067.wav": "내가 그때 분명히 우리 회사 주소를 알려 주고 떠났건만 많이는 안 보내 지더라고", "190429_01.0068.wav": "그러면 볼륨으로라도 보내면 피디님", "190429_01.0069.wav": "이게 뭐야", "190429_01.0070.wav": "우리 정해진 피디님 만약에 볼륨 앞으로 유인나는 편지가 오면은", "190429_01.0071.wav": "이렇게 했다 저한테 주시나요", "190429_01.0072.wav": "그 고민을 하는 거야", "190429_01.0073.wav": "아궁이를 하신다", "190429_01.0074.wav": "여러분 울림으로라도 저한테 혹시 할 말이 있으시면 편지 좀 써 주세요", "190429_01.0075.wav": "아주 그게 행복하더라구요 여러분이 편지 보낸 게", "190429_01.0076.wav": "이거 넘어가야 해 넘어가야 하는데 제가 지금 감을 못 잡고 있어서요", "190429_01.0077.wav": "코너 안내를 지금 할까요 하 할게요", "190429_01.0078.wav": "월요일에는 노 라이프 노 라이프가 준비되어 있어요 여러분 아시겠지만", "190429_01.0079.wav": "볼륨 청취자들의 사연에 어울리는 노래를 골라 주는 시간인데요", "190429_01.0080.wav": "어울리는 노래를 청취자 분들이 골라 주시는 게 아니죠", "190429_01.0081.wav": "아니 저희가 골라 드리는", "190429_01.0082.wav": "네", "190429_01.0083.wav": "디에이 드가 3부에 나오실 건데요 디에이드가 맞춤 선곡을 해 주고", "190429_01.0084.wav": "한국은 무려 라이브로 들려 주신대요", "190429_01.0085.wav": "저도 가끔씩 들어봤는데 라이브 정말 아름답고 너무너무 잘 하시더라고요", "190429_01.0086.wav": "이따가 3부에서 모셔 보도록 할 거고요", "190429_01.0087.wav": "나누고 싶은 일상들 듣고 싶은 노래도 저한테 하고 싶은 말", "190429_01.0088.wav": "자 좋습니다", "190429_01.0089.wav": "여러분들이 저 반겨 주시고 긴장하지 말라고 응원해 주시면", "190429_01.0090.wav": "제가 엄청 엄청 힘이 날 것 같아요 그런데 괜히", "190429_01.0091.wav": "오랜만이라고 느끼하게 좋은 말 만 하고 그런 거 말고 그냥 그냥 원래대로 저", "190429_01.0092.wav": "만 올려 주시고", "190429_01.0093.wav": "그냥 할 말 다 해 주시고 막 대해 주시고 그러면 저는 더 편하고", "190429_01.0094.wav": "힘이 될 것 같아요", "190429_01.0095.wav": "나도 가만히 있지 않는다", "190429_01.0096.wav": "아빠 이 문자 보는 샤 8910 1통에 단문 50원 장문 100원이에요", "190429_01.0097.wav": "50원 100원 정도 써 주실 수 있으시죠", "190429_01.0098.wav": "come과 my k 톡은 무료입니다 50원 100원 지금 조금 나한테 무리다 하시는 분들은", "190429_01.0099.wav": "무료로 많이 많이 보내 주세요 자 그럼 광고 댓글 게요", "190429_01.0100.wav": "초야와", "190429_01.0101.wav": "볼륨을 야", "190429_01.0102.wav": "볼륨을 높여요", "190429_01.0103.wav": "내 바버 레츠 로고송이죠", "190429_01.0104.wav": "이거 진짜 좋아요", "190429_01.0105.wav": "저거 들을 때마다 막 심취해서 불러요", "190429_01.0106.wav": "아아 눈이 이렇게 감는 거 뭔지 알지 눈썹 높이 들고 이렇게 하면서", "190429_01.0107.wav": "그런 거요", "190429_01.0108.wav": "집에서 이거 들으며 열심히 따라 불러요 진짜 좋은 거야", "190429_01.0109.wav": "볼륨을 높여요 스페셜 dj 유인나입니다 저는 유디의 목소리를 함께 듣고 계시죠 여러분인데 어디 계신가요", "190429_01.0110.wav": "8시 15분 14초 지나고 있어요 제가 이런 이야기를 할 때는 항상 나는 생방송 중이다라는 것", "190429_01.0111.wav": "티 내기 위해서 하는 말인데", "190429_01.0112.wav": "무슨", "190429_01.0113.wav": "아니면 참견 좀 해 달라 문자를 보내 달라 그런 뜻입니다", "190429_01.0114.wav": "어우 안 그래도 진짜 문자 많이 보내 주고 계세요", "190429_01.0116.wav": "진짜 기쁘다 너무 고마워요 여러분들이 만약에 막", "190429_01.0117.wav": "모른 척하구", "190429_01.0118.wav": "말도 안 걸고 그랬으면 진짜 슬펐을 거예요", "190429_01.0119.wav": "문성준씨가 박 변님 지금 스위스에 애견과 함께 있어요라고 하셨는데", "190429_01.0120.wav": "진짜", "190429_01.0121.wav": "아니 내가 왔는데 박별 여기 안구 스위스에 있다고요", "190429_01.0123.wav": "와 진짜 좋겠다 애견과 수", "190429_01.0124.wav": "스위스라니", "190429_01.0125.wav": "특별시 만약에 제가 목요일까지 있는데", "190429_01.0126.wav": "전에 이 소식을 들으시면", "190429_01.0127.wav": "요즘에는 뭐가 많잖아요 연락", "190429_01.0128.wav": "할 수 있는", "190429_01.0129.wav": "아 하긴 sns가 아니더라도 우리에겐 전화기도 있지", "190429_01.0132.wav": "샛별 씨한테 뭐 같은 거 막 그 이렇게 해서 사진 좀 보내달 라고 스위스의 풍경과", "190429_01.0133.wav": "강아지와 함께 이렇게 해서 사진은 작별도 나오게 해서", "190429_01.0134.wav": "사진 좀 보내 달라고 부탁을 해 보세요 여러분 박효신 너무 보고 싶네요", "190429_01.0135.wav": "그리고 최찬식이야 제천시 오랜만이에요", "190429_01.0136.wav": "인나 누나 3년 만이네요", "190429_01.0137.wav": "3년 동안 저는 대학 졸업도 하고 취직도 하게 되었어요", "190429_01.0138.wav": "그래서 지금 퇴근하면서 오랜만에 라디오 들으며 이렇게 문자를 보내네요", "190429_01.0139.wav": "정말 너무", "190429_01.0140.wav": "많이 보고 싶었어요 하셨어요", "190429_01.0141.wav": "제천시", "190429_01.0142.wav": "아닌 게 아니라 정말 너무 보고 싶네요", "190429_01.0143.wav": "최 천 씨가 제 사진에", "190429_01.0144.wav": "정말 예쁘게 찍어 주시는 분이에요 제가 가진", "190429_01.0145.wav": "얘는", "190429_01.0146.wav": "나 긴 한데 얘는 내가 아니다 너무 예쁘다 하는 사진들은 대부분 최 천 씨가 찍어 주신 거거든요", "190429_01.0147.wav": "어느 날부터 안 보이는 거야", "190429_01.0148.wav": "부하", "190429_01.0149.wav": "뭐 걸그룹이 있다고 하는데 뭐 이렇게 기억이 잘 안 나는데 걸그룹 불지 사진 찍으러 갔다 왔다고요", "190429_01.0150.wav": "아 최천식 지금은 지금 누구 사진을 찍어 주고 계신가요", "190429_01.0151.wav": "아무튼 정말 보고 싶네요", "190429_01.0152.wav": "세찬 씨", "190429_01.0153.wav": "궁금한데요", "190429_01.0154.wav": "음", "190429_01.0155.wav": "오 소개할게 진짜 많은 데", "190429_01.0156.wav": "코너가 있어 우리가", "190429_01.0157.wav": "6월 요일의 코너가 있어요", "190429_01.0158.wav": "매일 코너 뭘까요", "190429_01.0160.wav": "이거 이렇게 하는거 맞나요", "190429_01.0161.wav": "수근수근", "190429_01.0163.wav": "귓속말 코너로 가봅시다 하면", "190429_02.0000.wav": "이거 미적분 그래프 그리는 소리네요", "190429_02.0001.wav": "아 그 소리 내가 아닌데 이게 아니야 이거 이 소리랑 조금 달라요", "190429_02.0002.wav": "공부 좀 하세요 색시", "190429_02.0003.wav": "김민주 씨 아니 이게 퀴즈였어요 윤이 목소리 때문에 못했는데", "190429_02.0004.wav": "장혜진 언니 좀 조용히 해봐요 연필 소리 안 들잖아요", "190429_02.0006.wav": "나한테 말 좀 하자 나도 지금 디제이 논 건데 왜", "190429_02.0007.wav": "그렇잖아요 얼마나 오랫동안 말을 못 하고 있었는데요", "190429_02.0008.wav": "3419 유인나 씨는 누 난나", "190429_02.0009.wav": "땡", "190429_02.0010.wav": "지금 뭐라고 썼냐고 헐 떠든 사람 악동 수현이", "190429_02.0011.wav": "아", "190429_02.0013.wav": "아니에요 아니에요 저한테 하고 싶은 말을 써 주셨다고 했거든요", "190429_02.0014.wav": "응 케이 7413님이", "190429_02.0015.wav": "유인나 씨 빵 먹는다 카츠", "190429_02.0016.wav": "뭐 내가 그렇게 예전의 나처럼 뭐 아직도 빵 만 먹는 줄 아냐야", "190429_02.0017.wav": "전 빵을 좀 줄였습니다 빵을 너무 먹는 거 같아서", "190429_02.0018.wav": "조금 줄였어요", "190429_02.0019.wav": "078원 은행 62 언제까지 예쁠 건가요 하고", "190429_02.0020.wav": "뭐야 뭐야 자기가 더 이쁘면서 이런 여자들의 대화", "190429_02.0021.wav": "이준 씨가 8개 유인나 씨 만반 잘 부", "190429_02.0022.wav": "만반 잘 부", "190429_02.0023.wav": "뭐야 뭐야", "190429_02.0025.wav": "유인나 바보다 다 내 욕이야 지금", "190429_02.0026.wav": "유인나 씨 뒤에 귀신이 이런 거 하지 마요", "190429_02.0027.wav": "야 내 요리 얘기가 진짜 많구나", "190429_02.0028.wav": "맞아 여러분 시간이 너무 없어요 그래서 일단 정답을 발표해 드릴게요 정답은 요", "190429_02.0029.wav": "많이 맞춰 주셨어요 그쳐", "190429_02.0030.wav": "그러나 주는 계신 거죠", "190429_02.0031.wav": "유인나 씨", "190429_02.0032.wav": "어", "190429_02.0033.wav": "이거 유인나 씨 반가워요", "190429_02.0034.wav": "맞아요", "190429_02.0035.wav": "오빠야", "190429_02.0037.wav": "우와", "190429_02.0038.wav": "여러분 제가 오늘 첫날이잖아요 이해해 주셔서 절 좀 도와주세요", "190429_02.0039.wav": "어서 참여해 주신 분들 중에 정답자 오답자 상관없이 열 분 추첨하겠습니다 추첨해서요", "190429_02.0040.wav": "어이 분들께는 아이스 아메리카노 교환권 쏘고요 소리 파일 보내 주신 코카님께는 치킨 교환권 보내드리도록 하겠습니다", "190429_02.0041.wav": "당첨되신 분들은 볼륨 홈페이지에 당첨 선물 게시판에 연락처 꼭 남겨 주셔야 해요 그래야지 보내드릴 수 있습니다", "190429_02.0042.wav": "자이브 넘어가죠 지금 아직 일부죠 아니야 아 이브야 아냐 19야", "190429_02.0043.wav": "유베가야", "190429_02.0044.wav": "나는 좋아요", "190429_02.0045.wav": "우산 불만 목소리가 좋아요", "190429_02.0046.wav": "어린아이와", "190429_02.0047.wav": "저게 날아가요", "190429_02.0048.wav": "오 오늘의 씬나", "190429_02.0049.wav": "악동뮤지션 수현에", "190429_02.0050.wav": "볼륨을 높여요", "190429_02.0051.wav": "나이가 얼마든 사람은 성장한다", "190429_02.0052.wav": "성장하는 한 모두가 청춘이다", "190429_02.0053.wav": "성장 일기 어쨌거나 청춘", "190429_02.0054.wav": "제 221화", "190429_02.0055.wav": "그녀가 돌아왔다", "190429_02.0056.wav": "의의 천신이 5층이 게스트하우스에 오는 그 손님이 아는 사람이다 무소", "190429_02.0057.wav": "아 이 과장님이요", "190429_02.0058.wav": "어렸을 적에 한 동네 살던 언니인데요", "190429_02.0059.wav": "집에 인테리어를 새로 하나봐요", "190429_02.0060.wav": "며칠 살 길을 들여야 돼서 제가 할머니 게스트하우스 추천해줬어요", "190429_02.0061.wav": "할머니네 게스트하우스 손님도 없는데 이제 구매도 좋고 잘 했죠", "190429_02.0062.wav": "웃교 그란디 우리 우리 쪽에 만났는데 무슨 놈의 유 과장이라고 부르는겨", "190429_02.0063.wav": "아", "190429_02.0064.wav": "그때 언니네 아버지가 회사에서 과장님이었거든요", "190429_02.0065.wav": "혹시 자기가 아빠 딸이니까 유 과장이 나나 그렇게 불 러 달랬어요", "190429_02.0066.wav": "나중에는 근데 진짜 회사에서 육아 쟁이 되더라고요", "190429_02.0067.wav": "응 여기어 그래서 유 과장이랑은 친한 겨", "190429_02.0068.wav": "너 직장 생활하고 언니가 결혼하면서 지금은 뭐죠", "190429_02.0069.wav": "때문에 때문인데요", "190429_02.0070.wav": "그래도 각별한 정이 있는 사이죠", "190429_02.0072.wav": "변이를 말할 것 같으면", "190429_02.0073.wav": "저한테 동화책을 읽어 주는 언니였어요", "190429_02.0075.wav": "지금 드 언니가 동화책 읽어 주는 목소리가 들리는 것 같네요", "190429_02.0076.wav": "야야야 대박 야 내가 제법 도와 들려줄게 너도 드림팀 악 옛날에 세상에서 대박 이쁜 백설공주가 난쟁이들이랑 대박 잘 살고 있었어 언니 대박 나는 것을 드는 관기가 되는 거지는 치킨 닭다리 두 개 반 먹고 대박 쓰다", "190429_02.0077.wav": "응 야 그것 좀 요상하네 백설공주는 사과 먹고 쓰러지는 거 아니요", "190429_02.0078.wav": "응 현이가 치킨을 좋아했거든요 2", "190429_02.0079.wav": "무 주인공 이그가 사과대의 홀 려서 쓰러질 리가 없다는 거예요", "190429_02.0080.wav": "나도 그렇게 생각 했지만", "190429_02.0081.wav": "언니 때문에 제가 동화책 내용을 잘 못하는 경우가 얼마나 많았는데요", "190429_02.0082.wav": "백설공주가 먹고 쓰러진 건 닭다리", "190429_02.0083.wav": "선녀와나무꾼에서 나무꾼이 훔쳐간 건 딱지", "190429_02.0084.wav": "인어공주는 물거품이 아니라 무네가 되었다나", "190429_02.0085.wav": "어어 어겨 재밌는 양반이구먼 그려 동화책도 읽어줄 불고 아주 착한 언니 고마워요 온", "190429_02.0086.wav": "응 왜 왜요 싸우기도 많이 썼죠", "190429_02.0087.wav": "발레 학원을 같이 다녔거든요 오래", "190429_02.0088.wav": "발바닥 사탕 한입 만 달라는 거예요", "190429_02.0089.wav": "제가 앉으면 책상 금고 놓고 그랬어요 언니가", "190429_02.0090.wav": "아 대박 한입을 안 주냐 입을 너무 보이지 보이지 그럼 너무 다 듣고 다 대박 방금 연필로 나와서 이제 연필 냈고 대박이 내 거", "190429_02.0091.wav": "태릉", "190429_02.0092.wav": "소중한 손가락에 하나 넘어간 적이 있었어요", "190429_02.0093.wav": "와 제이 버 새끼 넘었다 넘었다 야근을 하게 될 거라 맞겠거니 하고 봐야지 대박은 먼 거니 봐야지", "190429_02.0095.wav": "그날 유로", "190429_02.0096.wav": "새끼손가락으로 코 안 파자나요", "190429_02.0097.wav": "그날 유가족 언니의 코딱지 촉감이 잊혀지지가 않아요", "190429_02.0098.wav": "쫄랑쫄랑 크 거야", "190429_02.0099.wav": "한층 갖고 놀았는데", "190429_02.0100.wav": "엄마 언니가 왔나 봐요", "190429_02.0102.wav": "언니", "190429_02.0103.wav": "에디 바야지 지도 반겼다 대박 얼굴이 이게 뭐냐 하면 나이를 정통으로 맞았네 일 봐", "190429_02.0107.wav": "야 피치", "190429_02.0108.wav": "그러게", "190429_02.0109.wav": "네 성장 일기 어쨌거나 청춘에 이어서", "190429_02.0110.wav": "그런 날에는 윤하의 별에서 온 그대였습니다", "190429_02.0111.wav": "그때 정말 오랜만이네요 가자 기래", "190429_02.0112.wav": "이거 진짜야", "190429_02.0113.wav": "아주 인기가 많은 친구였는데", "190429_02.0114.wav": "3419 님이 목소리 연기", "190429_02.0116.wav": "그래요 이은주 씨 육아 당 오랜만이네 지금 사장 좀 됐어야 하는데", "190429_02.0117.wav": "아니야 이 유 과장은 영원히 영원히 영원한 건 육아 지 않아", "190429_02.0118.wav": "하하 유 과장은 영원히 유 과장입니다", "190429_02.0119.wav": "김혜정 씨유 가장 대박 여전하다 김나영 씨 대박 유인나 대박 이유 대박", "190429_02.0120.wav": "박병석 씨 역시 1인 6 목소리까지 수고하신 배우 클라스", "190429_02.0121.wav": "할머니 목소리 반가워요 하셨어요", "190429_02.0122.wav": "아니 3년 동안 한 분의 목소리를 한 번도 해본 적이 없거든요", "190429_02.0123.wav": "그런데", "190429_02.0124.wav": "뭐야 이 원고를 보는데 할머니 목소리가 저절로 튀어나왔어", "190429_02.0126.wav": "어류 숙영 씨가 있나 있나 언니 살아 있네 하셨고요", "190429_02.0127.wav": "현준 씨가 우리 초딩 마니 컷네 이제 고딩이니", "190429_02.0128.wav": "아 그래 우리 초딩도 있었는데", "190429_02.0129.wav": "그리고 또 많은 분들이", "190429_02.0130.wav": "혹시 지금 애들이 맞춰야 할 것 같은 기분 뭐지", "190429_02.0131.wav": "1시간 오늘도 애들은 마치 가나요 하셨는데", "190429_02.0132.wav": "그치 그치 여러분을 기대할 것 같아서 제가 억지로 애들이 몇 마디 넣어 놨었어요", "190429_02.0133.wav": "근데 그게 생각이 안 나 우리가 애드리브를 여러분들 맞췄잖아요", "190429_02.0134.wav": "진짜 귀신같이 잘 맞춰", "190429_02.0135.wav": "아주 진짜 대단해요 여러분들은", "190429_02.0136.wav": "근데 맞추면", "190429_02.0137.wav": "내가", "190429_02.0138.wav": "뭘 해줬어", "190429_02.0139.wav": "선물을 드렸어요", "190429_02.0140.wav": "선물", "190429_02.0141.wav": "진짜", "190429_02.0142.wav": "오 그래 어차피 나 이거 4일 동안", "190429_02.0143.wav": "그리고 다 퍼주고 가야지", "190429_02.0144.wav": "다 퍼줄 거야 여러분", "190429_02.0145.wav": "맞춰 주세요 맞춰 주시면 선물 드릴게요", "190429_03.0000.wav": "어어", "190429_03.0001.wav": "응 대박을 몇 번 했는가 그거는 진짜 어려울 거 같은 데", "190429_03.0002.wav": "노래", "190429_03.0003.wav": "뭐야", "190429_03.0004.wav": "우리가 지금 8시 39분 57초 58초 지나고 있는데요", "190429_03.0005.wav": "몇 분까지 떠들 수가 있냐면요", "190429_03.0006.wav": "음 가만히 있어요 제가 알아요 47분", "190429_03.0007.wav": "아니 이분 이분 42분 아 그렇구나", "190429_03.0008.wav": "사실 지금까지 우리가 자유롭게 이야기를 나눌 수 있고요", "190429_03.0009.wav": "나는 노래를 한국 드릴 건데요", "190429_03.0010.wav": "막 문자가 진짜 많아", "190429_03.0011.wav": "어", "190429_03.0012.wav": "이거 말고 몇 가지 더 있었던 것 같긴 한데", "190429_03.0013.wav": "저도 마", "190429_03.0014.wav": "그래서 나온 말이라 기억이 안 나고요", "190429_03.0015.wav": "박경석 씨 선물 보내 드릴게요", "190429_03.0016.wav": "어디에다가 어디라고 올려 주세요", "190429_03.0017.wav": "음", "190429_03.0018.wav": "몸도 작년 시가", "190429_03.0019.wav": "유디 잘 지냈죠", "190429_03.0020.wav": "오랜만에 만나는건데 기분 좋은 사연이 아니라서 미안해요", "190429_03.0021.wav": "응 뭔 데 유리 요즘 이런저런 생각에 많이 힘들었거든요", "190429_03.0022.wav": "세상의 기준과 내가 안 맞는 것 같아서요", "190429_03.0023.wav": "이제 따뜻한 목소리로 위로받고 싶어서 사연 올려요", "190429_03.0024.wav": "신청곡도 있어요 제이레빗의 특별한 바램", "190429_03.0025.wav": "작렬 씨 오랜만이에요 그런데 장례 시", "190429_03.0026.wav": "이렇게 우리는", "190429_03.0027.wav": "실제로 봐도 목소리를 들을 수가 있고", "190429_03.0028.wav": "그래서 서로의 표정을 알 수가 있는 사이잖아요", "190429_03.0029.wav": "네", "190429_03.0030.wav": "많이 힘든 거 같아 하자 그래 이해해요 그런데", "190429_03.0031.wav": "기준 최상의 기준과 내가 안 맞는 안 맞을 수 있어요", "190429_03.0032.wav": "맞다고 좋고 뭐 안 맞다고 안 좋고 그럴 수가 있을까요", "190429_03.0033.wav": "우리가 지금 이 짧은 시간 안에", "190429_03.0034.wav": "당시 어떤 고민을", "190429_03.0035.wav": "그럴 수 없겠지만", "190429_03.0036.wav": "제 생각에는 저는 요즘에 그런 생각 많이 하거든요 그냥", "190429_03.0037.wav": "세상의 기준에 내가 맞춰 가는데 중요한 게 아니라 그냥 일단 내가 나를 사랑하는 첫 번째고", "190429_03.0038.wav": "내가 내 기준을 잘 세워서 나의 루틴에 맞게 잘 하루하루를 보내다 보면", "190429_03.0039.wav": "자신감이라는 건 무조건 다 나오게 돼 있더라고요", "190429_03.0040.wav": "뭐든지 그냥 깊이 장미윤 씨의 생각에 그 깊이를", "190429_03.0041.wav": "파헤쳐 보면 아마 결국 바닥에는", "190429_03.0042.wav": "지금", "190429_03.0043.wav": "어떤 종류의 자신감이 결여된 걸거라고 저는 생각해요", "190429_03.0044.wav": "그 자신감을 채우는 것", "190429_03.0045.wav": "1번으로 생각하고 그 방법이 뭔지는 우리가 다시 제가 월화수목 있는 동안에", "190429_03.0046.wav": "장윤 씨랑 다시 한 번 이야기를 나눠 보면 좋을 것 같아요", "190429_03.0047.wav": "이렇게 힘들면 안 되지 너무 속상해요 그러면", "190429_03.0048.wav": "물론 저도 힘들 때", "190429_03.0049.wav": "있지만 뭐야 시간", "190429_03.0050.wav": "시간이 여기 시간 뭔가 잘못 됐어 나를 속이고 있는데", "190429_03.0051.wav": "목요일까지 잡아내고 말겠다는 식의 비밀", "190429_03.0052.wav": "노래를 듣고 오겠습니다 작렬 씨 신청곡 띄워드릴께요 들으면서 1단지", "190429_03.0053.wav": "그래도 힘내 보세요", "190429_03.0054.wav": "제이레빗의 특별한 바램", "190429_03.0055.wav": "네 우리 소중한 장윤 씨의 신청곡 제이레빗에 특별한 바램 듣고 왔습니다", "190429_03.0056.wav": "응 여러분 여러분 뭐 하고 있어요 궁금해요 8시 46분 40초예요", "190429_03.0057.wav": "여러분 학교에도 계시고 우리 오픈 스튜디오에도 바뀌시고", "190429_03.0058.wav": "응 엄마 버스나 지하철에도 계실 테고", "190429_03.0059.wav": "그렇겠죠", "190429_03.0060.wav": "음", "190429_03.0061.wav": "lgt9934 님 윷이 최근에 나온 드라마에서 엄청 요리 잘 하시는 역할이던데", "190429_03.0062.wav": "혹시 서운하게 요리 실력이 되셨나요", "190429_03.0064.wav": "한동근 씨가 유지", "190429_03.0065.wav": "진짜 진짜로 유리 너무 오랜만에 봐서 궁금한 게 있는데 이제 미역국 끓일 줄 알아요 아 진짜 놀리는 거 절대 아니에요 라고 보내 주셨어요", "190429_03.0066.wav": "이 얘기 너무해 주고 싶은데 레시피까지 완벽하게 얘기해주고 싶은데 시간이 없어서 시간이 있다 시간이 나면 제가 아주 자세하게 구체적으로 설명을 드리도록", "190429_03.0067.wav": "할게요", "190429_03.0068.wav": "케이 7413님이 있나 언니 딸기님 중국 팬이에요 반가워요", "190429_03.0069.wav": "전 우주가 언니 라디오 듣고 있다", "190429_03.0070.wav": "하", "190429_03.0071.wav": "아 귀여워라", "190429_03.0072.wav": "야", "190429_03.0073.wav": "아 이름이 뭘까 이름이 너무 부르고 싶은데 7414님", "190429_03.0074.wav": "감사합니다아 지금 간간히 보여 중국 팬 분들께서 어떻게 이렇게", "190429_03.0075.wav": "한글로 보내 주시는 건지 진짜 신기하고", "190429_03.0076.wav": "정말 정말 반갑고 너무너무 감사합니다", "190429_03.0077.wav": "진짜 너무 감사합니다", "190429_03.0078.wav": "그래서 지금도 수십억 남명의 우리 청취자 분들이 듣고 계시잖아요", "190429_03.0079.wav": "선물을 보내드릴께요 제가 웬만한 지금 훌륭한 분들은 다 선물을 드린다고 생각하시면 됩니다", "190429_03.0080.wav": "몰라 나는 기둥을 다 보고 갈 거니까", "190429_03.0081.wav": "그렇지 창고를 비워 줘야 지라고", "190429_03.0082.wav": "되게 긴 님이 보내 주셨고요", "190429_03.0083.wav": "김효준 씨가 저를 잘 알거든요", "190429_03.0084.wav": "단언컨대 4일 만에 kbs 기둥 거덜 납니다 흑", "190429_03.0086.wav": "그리고 또요", "190429_03.0087.wav": "응 또 음 무게", "190429_03.0088.wav": "딸기 언니 전 중국의 팬이에요 방송에 다시 돌아온 거 축하해요", "190429_03.0089.wav": "많이 나가서 활동하세요 중국 진짜 많이 분들이 좋아해 주시고 힘내세요 우린 더 행복해질 거예요", "190429_03.0090.wav": "위임", "190429_03.0091.wav": "감사합니다 맞아요 우리는 분명히 더 행복해질 거예요", "190429_03.0092.wav": "와 진짜 진짜 기분이 너무너무 좋네요", "190429_03.0093.wav": "아 그리고 아이유 씨를 찾는 분들이 굉장히 많아요", "190429_03.0094.wav": "여기로 씨가", "190429_03.0095.wav": "아이유 언제 나오죠 선 부에 나오나요", "190429_03.0096.wav": "밖에 있는 거 다 알아요 으 설렌다 하셨어요", "190429_03.0097.wav": "그러니까요 나도 살림 나오겠지 분명히", "190429_03.0098.wav": "응 나 올까지 나 올 까지야 아이유 듣고 있냐와", "190429_03.0099.wav": "월화수목 중에 한 번은 나타나겠지 다크 템플러인데 설마 안 올까", "190429_03.0100.wav": "여러분들 어디에 아이유 씨가 어디에 계신지 모르겠는데 소식을 전해 주세요", "190429_03.0101.wav": "여기에서 한 번 조회를 해야 하지 않겠냐고", "190429_03.0102.wav": "많은 분들이 이렇게 기다리고 계신데", "190429_03.0103.wav": "저도 여러분이랑 똑같이 되게 기대하고 있어요", "190429_03.0104.wav": "언제 저 문을 열고 들어올지 모른다고요", "190429_03.0105.wav": "이 소식을 좀 전해 주시고요", "190429_03.0106.wav": "어어어 또 오오오", "190429_03.0109.wav": "진짜 진짜 많다 이거 문자가 워워", "190429_03.0110.wav": "오 여기가 숫자가 이거구나", "190429_03.0112.wav": "우두 많은 분들을 소개해 드리고 싶은데", "190429_03.0113.wav": "지 지금 지금이 49 아 그래", "190429_03.0114.wav": "여 러븐", "190429_03.0115.wav": "다음으로 넘어가야 할 시간이에요 아시죠 중요한 거 이거 듣고 와야 합니다 광고 듣고 바로 돌아오도록 하겠습니다 잠깐만요", "190429_03.0116.wav": "4월의 협찬품 소개해드릴께요 제가 한 번 더듬을 때마다 한 분께 선물 드립니다 뜨끔", "190429_03.0117.wav": "홍대 메이크업 브러쉬 플라 니아에서 블랙 에디션 세트 예쁘고 부모님 예님에서 카멜리아 오일 폼 클렌저", "190429_03.0118.wav": "사라의 선인장을 닮은 확실히에서 오일 에센스", "190429_03.0119.wav": "현지 맞춤 솔루션 티엔에서 극세사 마스크 팩 세트 피부 전문가가 만든 환호하기 위해서 호일 마스크팩", "190429_03.0120.wav": "프리미엄 뷔페 청라 들어봐 드마리스에서 식사권 감성 뷰티 핏", "190429_03.0121.wav": "쉿", "190429_03.0122.wav": "keep in touch에서 mlb 벨벳 틴트", "190429_03.0123.wav": "한 번 빠져드는 청년 피자 피자 시식권 천연 화장품 봉프레에서 보고 모바일 상품권", "190429_03.0124.wav": "파워 플렉스에서 시의 나이트 크림 자연을 노래하는 피부 라벨 영에서 쇼킹 필링젤", "190429_03.0125.wav": "손상모 케어의 모든 것 모레 모에 헤어 에센스 메디칼 주 파코메리 스틱이 아니죠 코스메틱 타워에서 비비 크림은 물만으로 클렌징 크림에서 클렌징 퍼프 트러블 팩", "190429_03.0126.wav": "콩스타일에서 여성 의류 쇼핑몰 이용권", "190429_03.0127.wav": "아름다운 비주얼 아디 주에서 뷰티 상품권 바디 허벌 지에서 천연 화 뿌", "190429_03.0128.wav": "천 염화", "190429_03.0129.wav": "후 화장품 세트", "190429_03.0130.wav": "수연이가 선택한 향기 브랜드 더블유 더블유 드레스룸에서 핸드크림 5종 세트를 드립니다와", "190429_05.0001.wav": "어우 주말 아니 솜사탕 같다고 하잖아요 솜사탕 계의 1 인자인 것 그것은 정말 목소리가 솜사탕에 쌓여 있는 것 같아요 너무 달다 3 하하 박경숙 씨가 내 복 넘치는 꿀 덕분에 da분들의 라이브 듣게 되다니 감동 김형석 씨 숙취와 라이브는 병기와", "190429_05.0002.wav": "어어 문상준 씨가 홍대 술 요정의 왕좌를 물려받은 시죠왕자가 지금 그 왕가는 지금 누구의 머리예요 하하 궁금하 네요와 정말 너무너무 잘 하시네요 와아 감사합니다", "190429_05.0003.wav": "응", "190429_05.0004.wav": "소녀 씨가 그레이 노래 가사가 너무 예쁘다고만 생각했는데 아름 씨가 말씀해 주시기를", "190429_05.0005.wav": "근데 헤어지면", "190429_05.0006.wav": "같았던 것들이 얼마나 힘들어 지는지 몰라요 자꾸 생각나서 아 오 끝이 함께 했을 때", "190429_05.0007.wav": "우리 왜 그런 순간들많은 사람이 있잖아요 똑같이 말하거나 똑같은 거라서 하였거나 어 어 역시 우리는 하이파이브하는 그런 느낌 근데 그런 것들이 헤어지고 나면은", "190429_05.0008.wav": "정말 슬프죠", "190429_05.0009.wav": "생각이 계속 나는 것", "190429_05.0010.wav": "맞아요 네", "190429_05.0011.wav": "a ado", "190429_05.0013.wav": "이 중국 팬 분이 아까부터 보내 주고 계세요 너무 감사하게도 이 가격은 참 듣기 좋다 하", "190429_05.0014.wav": "어 전 세계 우리 수십억 만 명의 가족들과", "190429_05.0015.wav": "외계인까지 듣고 있거든요 그니까는", "190429_05.0016.wav": "그런데 누가 들어도 정말 듣기 좋은 소리였던 것에 감사합니다 이수만 씨 가요 2 노래 너무 좋아서 검색해 보니까", "190429_05.0017.wav": "작사 작곡 편곡 김 유야 온", "190429_05.0018.wav": "은행 어떤 사람입니까", "190429_05.0019.wav": "그런데 섬세하고 다정한 사람이라고 믿어요", "190429_06.0000.wav": "네 악동뮤지션 수현의 볼륨을 높여요 함께 하고 계십니다", "190429_06.0001.wav": "저는 스페셜 dj 유인나입니다", "190429_06.0002.wav": "음 저는 요 오늘 첫날이라서 약간 감을 못 잡고 있어요", "190429_06.0003.wav": "하지만 여러분들이 이해해 주실 거라 믿고 내일은 완벽한 디제이가 돼서 올 거라는 거 여러분 아시죠 솔직히", "190429_06.0004.wav": "어", "190429_06.0005.wav": "목요일까지 생방송으로 함께 합니다", "190429_06.0006.wav": "목요일까지 꼭 여러분이 필요해요", "190429_06.0007.wav": "도와주세요", "190429_06.0008.wav": "야우리 사연을 조금 더 만나 볼 여유가 있어요 디에이드 분들은 잘 보내 드렸고요", "190429_06.0009.wav": "응", "190429_06.0010.wav": "아니 막", "190429_06.0011.wav": "막 벙 맛있는 걸 주신 거예요 저한테 딸기 주스랑 술", "190429_06.0012.wav": "아유 닭이 자나요", "190429_06.0013.wav": "네 오래전 일 이지만 제가 억지로 된 거예요 그거 나도 딸기라고 불러 달라 여러분들이 지어 주신 예쁜 별명이 아니라", "190429_06.0014.wav": "어느 날 어떤 사연의", "190429_06.0015.wav": "연인들끼리 뭐 예쁜 어떤 애칭 같은 거", "190429_06.0016.wav": "그래서 앵두야 자두야 이렇게 부른다 그래서 나도 딸기로 불러 달라", "190429_06.0017.wav": "여러분들이 그렇게 다음 날부터 제가 내일 내가 오면", "190429_06.0018.wav": "아무 일도 없었다는 듯이 원래부터 난 닭이었다는데 딸기로 불러 달라", "190429_06.0019.wav": "했더니 얼마나 쿵짝 이 잘 맞는지 다음날 딸기왔냐면서 반겨 주셨다는 말이에요", "190429_06.0020.wav": "그렇게 해서 제가 잇따르게 되는데", "190429_06.0021.wav": "어우 오늘도 정말 역시 저렇게 딸기도", "190429_06.0022.wav": "많이 많이 맞아 주고 계세요", "190429_06.0023.wav": "어", "190429_06.0024.wav": "혜리가 진짜인지 알았는데", "190429_06.0025.wav": "그만 놀리고 진짜 요리로", "190429_06.0026.wav": "제가 그때 당시 요리를 좀 못 했었던가", "190429_06.0027.wav": "내가 유래 뜻", "190429_06.0028.wav": "못 했어 어우 웅", "190429_06.0029.wav": "그래서 요리를 잘 할 예정이다 라고 항상 말씀을 드렸는데", "190429_06.0030.wav": "내가 너무 그 사유리 잘 해줬으면 여러분 어때요", "190429_06.0031.wav": "어때요 문자 좀 보내줘", "190429_06.0035.wav": "너무 섭섭하지", "190429_06.0036.wav": "내가 잘 해줬으면 너무 섭섭하죠", "190429_06.0037.wav": "일단 일단은 비밀로 하고 제가 내일 알려줘도 되는 거잖아요", "190429_06.0038.wav": "기다려봐요 요리에 대해서 어떻게 되는지 알려드릴 거고요", "190429_06.0039.wav": "사연을 좀 소개해 드릴게요", "190429_06.0041.wav": "구육 육 삼님이 오늘 퇴근길은 임원 분 모셔다 드리느라 이제서야 라디오 켰는데", "190429_06.0042.wav": "윷이다 윷이 홍콩 반가워서 실시간 눈물 흘리는 중 유유", "190429_06.0043.wav": "응 눈을 이렇게 찍어서 사진 좀 보내 주세요", "190429_06.0044.wav": "눈물 흘리는 거 실제로 보고 싶으니까", "190429_06.0046.wav": "아니 요즘에는", "190429_06.0047.wav": "사진도 볼 수 있죠 마", "190429_06.0049.wav": "저는 방법은 모르지만", "190429_06.0050.wav": "볼 수 있다는 건 알고 있습니다", "190429_06.0051.wav": "눈물 흘리는 눈물일까", "190429_06.0052.wav": "너무 보고 싶어", "190429_06.0053.wav": "응 윤미 씨가 유독 내일 보라에는 꼭 빵 들고 와요", "190429_06.0054.wav": "아니야 나 이제 빵", "190429_06.0056.wav": "너 사람 말은 못 되지만 빵 좀 줄였어요 너무 먹어 가지고 안 되겠더라고요", "190429_06.0057.wav": "그래서 요즘에는 열심히 밥 먹고 있어요 작곡 밥 같은 거 내가 만들어 놓아 여기까지 요리 얘기까지 했다", "190429_06.0058.wav": "너무 여러분이 여기에 관심을 가지십니까", "190429_06.0059.wav": "내일을", "190429_06.0060.wav": "내일 할 얘기가 많네 아이유도 내려오지 않을까 왜 오늘 알았지", "190429_06.0061.wav": "마더", "190429_06.0062.wav": "나 솔직히 올 줄 알았는데", "190429_06.0065.wav": "응 또 1094님이 몇 년 전 볼륨을 높여요 유디에게 수줍게 연애 상담을 하던 제가 절 진심으로 좋아해 주는 남자를 만나 2주 전에 결혼이라는 것", "190429_06.0066.wav": "했어요", "190429_06.0067.wav": "전원생활을 즐기고 있답니다 하루하루 꿈같고 행복해요", "190429_06.0068.wav": "전이렇게 잘 지내고 있답니다 당", "190429_06.0071.wav": "이런 사연이 가장 반갑습니다 어떻게 지내고 계셨는지 여러분 내일은 저 오자마자 이런 거 보내 주세요", "190429_06.0072.wav": "없었던 몇 년간의 나 어떻게 컸고 나무가 됐고 네 어떤 즐거운 일이 있었고 나 어떤 힘든 일이 있어", "190429_06.0073.wav": "이런 거 우리 같이 얘기 나눠요", "190429_06.0074.wav": "한동근 씨가 아유님 어제 게임 행사 가시고 수요일에 백상 후보 올라가서 거기 참석한데요라고", "190429_06.0077.wav": "너무 잘 알고 계신다", "190429_06.0078.wav": "그러면 사실", "190429_06.0079.wav": "앞으로 백성보다 나 아닐까", "190429_06.0081.wav": "기대해 보자고요 우리", "190429_06.0082.wav": "어우 우 오 오 칠님3년 전에 백수였고 지금도 백수가 되었어요 유리에 변함없이 놀라고 제 상황에 놀랍니다 저도 놀라면서 광고 듣고 있어요", "190430_01.0000.wav": "4월 30일 화요일 볼륨을 높여요", "190430_01.0001.wav": "오늘의 문을 열어준 거거든요 체내의 make it count입니다", "190430_01.0002.wav": "흠 안녕하세요 여러분 저는", "190430_01.0003.wav": "스페셜 디제이 유 댓글 두유 딸기 유치 자 뉴트는 시카 유인나입니다", "190430_01.0004.wav": "어울리는 수연 씨 자리잖아요 수연 씨가 일을 하러 갔어요 그 틈을 타서 제가 여러분을 만나러 이렇게 왔고요", "190430_01.0005.wav": "반갑죠 오늘 벌써 이틀째예요", "190430_01.0006.wav": "응", "190430_01.0008.wav": "height 오늘 이틀째인데 그 말인즉슨 이틀밖에 안 남았다 우리가 만날 날이", "190430_01.0009.wav": "유두", "190430_01.0010.wav": "응 지금 8시 5분 38초", "190430_01.0011.wav": "어제 막 시작했으니까 오늘 두 시간 동안 아주 아주 만약에 꽉", "190430_01.0012.wav": "눌러서", "190430_01.0013.wav": "해 보도록 합시다", "190430_01.0014.wav": "제가 그랬죠 가면서 3시간 있다 만나자고 했잖아요", "190430_01.0015.wav": "맞어", "190430_01.0016.wav": "뭐 했어요 여러분은", "190430_01.0017.wav": "어제 10시 이후로 지금까지 뭐하셨는지 좀 보내 주세요", "190430_01.0018.wav": "안녕", "190430_01.0019.wav": "음 저는 끝나고", "190430_01.0020.wav": "엉 또 뭘 먹었어 하", "190430_01.0021.wav": "우와 더블 먹고 집에 가서", "190430_01.0022.wav": "요즘에 책 정리하고 있거든요 집에 책이 막", "190430_01.0023.wav": "와 그 진짜", "190430_01.0024.wav": "책 사는 걸 좋아하잖아요 제가", "190430_01.0025.wav": "그러니까 책을 막", "190430_01.0026.wav": "여기저기 다 쌓여 있는데 너무 꼴 보기 싫어서", "190430_01.0027.wav": "다만 정리하고 있는데 그 작업을 또", "190430_01.0028.wav": "며칠째 어제도 하다가", "190430_01.0029.wav": "응 나 새벽에", "190430_01.0030.wav": "또봇 먹었어", "190430_01.0031.wav": "해외 또 배고프잖아요 그래서 뭘 또 막 먹다가", "190430_01.0032.wav": "요즘에 일이 없으니까 이럴 때는", "190430_01.0033.wav": "빵 먹어요", "190430_01.0034.wav": "그렇게 하루를 보내고", "190430_01.0035.wav": "오늘 오늘 뭐했지 오늘", "190430_01.0036.wav": "오늘도 친구를 만났어요 친구랑 같이", "190430_01.0037.wav": "다른 친구가", "190430_01.0038.wav": "청국장을 해다 준 거예요", "190430_01.0039.wav": "먹으라구 층 극장에서 갖다 줬어요 그래서 그거를 또 다른 친구랑 함께", "190430_01.0040.wav": "같이 먹었어요 진짜 진짜 맛있게 밥을 먹고", "190430_01.0041.wav": "음", "190430_01.0042.wav": "그리고", "190430_01.0043.wav": "호두 잠깐 또 아이유 씨랑", "190430_01.0044.wav": "잠깐 수다를 떨다가", "190430_01.0045.wav": "떡볶이를 먹었어요", "190430_01.0046.wav": "그냥 떡볶이를 먹고", "190430_01.0047.wav": "음 그리고 또 매니저들이랑 함께", "190430_01.0048.wav": "돈가스를 먹고", "190430_01.0049.wav": "그리구", "190430_01.0050.wav": "어우 왔어요", "190430_01.0051.wav": "여러분은 뭐 드셨어요야", "190430_01.0053.wav": "어", "190430_01.0054.wav": "어 그래 맞아 김혜수 씨가", "190430_01.0055.wav": "어삼 년 전에 유리 각도 보라 각도다", "190430_01.0056.wav": "여러분 오늘은 보이는 라디오로 함께 하고 있습니다", "190430_01.0057.wav": "보이는 라디오예요", "190430_01.0059.wav": "안 봐도 되고", "190430_01.0060.wav": "봐도 되는데", "190430_01.0061.wav": "보지만 말고 들어줘요 말도 걸어 주시고요", "190430_01.0062.wav": "6004 님이 쉿", "190430_01.0063.wav": "하하하", "190430_01.0064.wav": "와 그래 맞아 저도 제 휘파람 실력에 대해서 잊고 있었는데", "190430_01.0065.wav": "내가 휘파람을 잘 불었다", "190430_01.0066.wav": "그래서 우리 한참 그거 불렀잖아요", "190430_01.0067.wav": "기다려봐요", "190430_01.0075.wav": "그렇지", "190430_01.0079.wav": "어떡해요 휘파람 어디가 서 있다가 제가 노래나 근사에 다시 연습해서 데리고 올게요 휘파람 실력을", "190430_01.0080.wav": "으아아 또요", "190430_01.0081.wav": "서초 씨가 오늘의 얼룩말 같고 너무 좋네요 오셨어요 어디가 내가 어디가 얼룩말 같애", "190430_01.0082.wav": "어어 어디가 자", "190430_01.0084.wav": "여러분 오늘 있잖아요", "190430_01.0085.wav": "음 그분 오신대요", "190430_01.0086.wav": "안 그래도 이 화요일에 차밍 스쿨", "190430_01.0087.wav": "여러분들은 익숙한 코너인가요", "190430_01.0088.wav": "볼륨 어들의 스타일을 업그레이드시켜 들 인데요 스타일 준 뮤지션 서사 무이리", "190430_01.0089.wav": "서사무엘 씨가", "190430_01.0090.wav": "3일 서", "190430_01.0091.wav": "서사무엘 씨가 조언을 해 주신다고 합니다 주 주말에 특별한 약속이 있어서 옷 신경 써야 하는 분들", "190430_01.0092.wav": "옷 못 입는 분들", "190430_01.0093.wav": "창피하지 마시고 사연 보내 주세요", "190430_01.0094.wav": "저는 뭐 땜에 안 되겠지만 아마 서사무엘 씨가", "190430_01.0095.wav": "예 이거 좀 패션 꿀팁으로 거들어 드릴 거예요 음", "190430_01.0096.wav": "그럼 오늘 모 드셨는지 어제부터 오늘까지 뭐하셨는지 오늘 있었던 깨알 같은 사연들", "190430_01.0097.wav": "지금 나왔으면 하는 노래들 듣고 싶은 노래 여기로 보내 주시면 됩니다", "190430_01.0098.wav": "문자 보는 샤 8910 한 통에 50원이에요 근데 그걸 길게 쓰면 퇴근이다 여름은 짧은 강의 지만 귀권 배우 김건 백 원", "190430_01.0099.wav": "사진도 100원이에요 사진도 좀 많이 많이 보내 주세요", "190430_01.0100.wav": "오늘 사진 보는 거 배웠거든요", "190430_01.0101.wav": "이 게시판이 이를 누르면", "190430_01.0102.wav": "사진이 보이는데요 게시판 누르면 사진이 안 보이고 이따가 만약에 내가 어떻게 하는 거였더라 따먹었다 하면 여러분이 게시판 눌러라 라고 저한테도 알려 주세요", "190430_01.0103.wav": "그럼 홍과 톡 마이크 무료니까 많이 많이 걸어 주시고요", "190430_01.0104.wav": "음", "190430_01.0105.wav": "이렇게 말 걸어 주시니 오늘도 제가 국가 않답니다 불린 국가는 kbs 기둥 뽑고 갑니다", "190430_01.0106.wav": "광고지 곳", "190430_02.0000.wav": "네 제가 아주 오랜만에 들고 온 노래죠", "190430_02.0001.wav": "좋아해", "190430_02.0002.wav": "처음 만든 사랑 노래", "190430_02.0003.wav": "그렇습니다", "190430_02.0004.wav": "예전에는 제가", "190430_02.0005.wav": "일주일에 한 번이었나요 그", "190430_02.0006.wav": "네에", "190430_02.0007.wav": "추천곡", "190430_02.0008.wav": "시간이 있었던 거 같아요", "190430_02.0009.wav": "그래서 여러분들한테 들려주고 싶은 노래 가 너", "190430_02.0010.wav": "너무많아서", "190430_02.0011.wav": "허물을 들려드릴까 생각을 하다가", "190430_02.0012.wav": "아침이 소근소근", "190430_02.0013.wav": "어 이 코너에", "190430_02.0014.wav": "딱 맞는 거 같아서 여러분들한테", "190430_02.0015.wav": "제가 봤는데 어때요", "190430_02.0016.wav": "이건 대창 가사가 여러분 생각해봐요", "190430_02.0017.wav": "이 순간이 이대로 멈추길 바라는 이런 우리의 마음과", "190430_02.0018.wav": "어 약 속이란 말이 없어도 꿈을 꾸는 우리가 참 행복하다", "190430_02.0019.wav": "우리 우리랑 행복이란", "190430_02.0020.wav": "단오랑 가깝잖아요", "190430_02.0021.wav": "우리는 더 행복해질 거라는 이야기를", "190430_02.0022.wav": "우리는 한 5년 정도는 매일매일 같이 했죠", "190430_02.0024.wav": "아유 근데 다 들 지금", "190430_02.0025.wav": "너는 먹는 얘기 뿐이냐 온통 먹는 얘기냐 하", "190430_02.0027.wav": "1382 님 이하 유지 목소리는 반드시 유네스코 문화유산으로 지정해야 합니다", "190430_02.0028.wav": "그래서 우리끼리 그렇게 하기로 결정합시다", "190430_02.0029.wav": "픽셀을 냅시다 오늘", "190430_02.0032.wav": "강신혁 씨가 가사였구나 어쩐지 유리가 만들어낸 건 아닌 것 같아", "190430_02.0034.wav": "다음에 만들어서 갖고 올게요", "190430_02.0035.wav": "김형근 씨가 어 이거 공기 반 소리 반인데 왜 이 코너는 pd님 이거 누가 만든 코너예요 진짜", "190430_02.0036.wav": "되게 좋다 진짜 요즘", "190430_02.0037.wav": "좋다 요즘 사람들이 원하는 그런 코너인 거 같은 데", "190430_02.0038.wav": "하는 사람은 약간 길게 못해요 단소 오래 보면 어지러운 거 잇잖아요 뭔지 알죠 여러분", "190430_02.0039.wav": "어지러워 지려고 이거", "190430_02.0040.wav": "음 정해질 님이 걸어 가다가 잠들겠어요 목소리 대박", "190430_02.0041.wav": "아 재워 주고 싶네요 여러분들", "190430_02.0042.wav": "진짜 잘 지어줄 수 있을 것 같긴 한데", "190430_02.0043.wav": "전영미 씨 반남 예정에 벌써 하나요", "190430_02.0044.wav": "하하 만날 예정이에요 참 저도 좋아했던 코너였었는데", "190430_02.0045.wav": "응 0942 님이 너무 귀여운 거 아니에요 인났을 때 거금 200원 써요", "190430_02.0046.wav": "어땠어요 0942님", "190430_02.0047.wav": "200원 오타 소", "190430_02.0048.wav": "사진은 긴 문자 두 개 보내 주셨어요 역시", "190430_02.0049.wav": "굽네", "190430_02.0050.wav": "음", "190430_02.0051.wav": "6314 님이", "190430_02.0052.wav": "전 오늘 퇴근길에 혼자 멍게를 사서 먹었어요", "190430_02.0053.wav": "친구가 통영 가서 해산물 사진을 보냈는데", "190430_02.0054.wav": "너무 먹고 싶어서", "190430_02.0055.wav": "잘 했어", "190430_02.0056.wav": "먹고 싶은 거 있을 때는 찾아가서 드세요", "190430_02.0057.wav": "옛날에는 안 그랬거든요", "190430_02.0058.wav": "아니야 이런 말 함부로 하면 옛날에 다 알고 있어", "190430_02.0059.wav": "밖에서 날 쳐다보고 있어", "190430_02.0061.wav": "맞아요 저는 먹고 싶은 게 있으면 그날 그날 찾아가서 먹었던 것 같아요", "190430_02.0062.wav": "근데 이런 열정까진 없었던 거 같아요 막와 멍게를 먹으러 간다는 거는 정성이 필요한 거잖아요 어디인가를 반드시 찾아가야 되니까", "190430_02.0063.wav": "근처에서", "190430_02.0064.wav": "막 여기저기 막 우리 여기 앞에서도 팔고 그런 건 아니잖아요", "190430_02.0065.wav": "잘 하셨네요 정말", "190430_02.0067.wav": "조민구 씨가 꿀 디 누나 응", "190430_02.0068.wav": "더덕 무침이 있는데요와", "190430_02.0069.wav": "더덕 좋아요", "190430_02.0070.wav": "좋아하는 반찬이 궁금해요 저는 꼬막 많이 좋아합니다라고", "190430_02.0071.wav": "또 민구 맘 내맘", "190430_02.0072.wav": "더덕 꼬막", "190430_02.0073.wav": "너무 좋아해요", "190430_02.0074.wav": "저는", "190430_02.0075.wav": "그리고요", "190430_02.0076.wav": "군것질은 아예 다른 거고 막 그냥 케익이라든지 빵이라든지 단것들 있잖아요 아이스크림", "190430_02.0077.wav": "이런 거 이런 거 대로 좋아 하는데", "190430_02.0078.wav": "저는 식사 만큼은 진짜 어른이 좋아하는 거 좋아해요", "190430_02.0079.wav": "파워나 진짜 파 잘 먹고", "190430_02.0080.wav": "마늘", "190430_02.0081.wav": "양파", "190430_02.0082.wav": "적 양파 적 양파를 진짜 좋아해요", "190430_02.0083.wav": "깻잎 깻잎을 세상에서 제일 좋아해요", "190430_02.0084.wav": "깻잎", "190430_02.0085.wav": "그리고 또", "190430_02.0087.wav": "이런 건 재료 지금 반찬이라기보단", "190430_02.0089.wav": "다 좋아요 저는 먹을 거는 얼만 한 건 하지 마요", "190430_02.0090.wav": "추석 시각", "190430_02.0091.wav": "멍게에 소주 끝장나는데", "190430_02.0093.wav": "그동안 내가 술꾼이 됐잖아", "190430_02.0095.wav": "그래요 제가 이런 말을 하면 주위에서 너무 부실해요 너무", "190430_02.0097.wav": "제가 요즘에 여러분", "190430_02.0098.wav": "맥주 맛을 알았어요", "190430_02.0099.wav": "예전에는 진짜 술 한잔도 못 한 잔이면 반장도 못 마셨잖아요", "190430_02.0100.wav": "요즘에는 일하다 보면", "190430_02.0101.wav": "와", "190430_02.0102.wav": "시원하게 맥주 한잔 마시고 싶다", "190430_02.0103.wav": "제가 이런 말을 저도 모르게 막 하는 거예요", "190430_02.0104.wav": "그럼 저희 스탭들은 10년간 저랑 같이 있기 때문에 갑자기 빵 터져요", "190430_02.0105.wav": "웃긴가 봐요 전 술 마시는 거 본 적이 없으니까", "190430_02.0106.wav": "요즘에는", "190430_02.0107.wav": "우와", "190430_02.0108.wav": "맥주 한잔 진짜 차가운 맥주 한잔 꿀꺽에", "190430_02.0111.wav": "요즘 즐겨 하고 있아", "190430_02.0112.wav": "야 이수민 씨가 이얼 신조 썼다 하셨는데 뭐라 못 썼어", "190430_02.0113.wav": "나도 저렇게 썼나 봐 실수 실수", "190430_02.0114.wav": "뭐죠", "190430_02.0115.wav": "신조어", "190430_02.0117.wav": "뭐하나 배운 게 있었는데", "190430_02.0118.wav": "아 뭐더라", "190430_02.0120.wav": "참 노래하지 마 봐요", "190430_02.0121.wav": "아우 잠깐 만 노래할 시간이 한참 지났잖아 안 그래 할 시간이 아니라 노래 틀 시간", "190430_02.0122.wav": "노래는 못해요", "190430_02.0123.wav": "아니야 노래는 못해", "190430_02.0124.wav": "여러분 이거 우리도 추억의 노래죠", "190430_02.0125.wav": "커피 소년의", "190430_02.0126.wav": "행복의 주문 듣고 있어요", "190430_03.0000.wav": "아니 끝 그는 세 번 이상 쓰지 마세요", "190430_03.0001.wav": "지금 난리가 났어 크 크 타", "190430_03.0004.wav": "정말 창피하다 와 진짜", "190430_03.0005.wav": "내가 알기로는 이 코너에서", "190430_03.0006.wav": "이 정도로 는 작가님이 그동안 쓰셨던 거 같은 데", "190430_03.0007.wav": "어디 유인나도 한 번 당해봐라 해서 하하", "190430_03.0008.wav": "너무 하는 거 아닙니까 정말", "190430_03.0010.wav": "전영민 씨가 권사님 창법이라고", "190430_03.0011.wav": "권현정 씨 유디 보라 잠시 돌린 이유 알겠네요 민망함은 잠시예요 이지", "190430_03.0012.wav": "아니 얼굴이 진짜 너무 빨개져서", "190430_03.0013.wav": "정말요", "190430_03.0014.wav": "그러나 19세기", "190430_03.0015.wav": "근데 보이는 라디오에는 내가 지금 야광인 거 같은데 여러분들도 제가 제가 보는 것처럼 보이는 건가요", "190430_03.0016.wav": "하하 야당이지", "190430_03.0018.wav": "a word", "190430_03.0020.wav": "아잇", "190430_03.0021.wav": "사부도 있겠다 긴 님이 케이긴 님임", "190430_03.0022.wav": "매물 주면 안디야 크 큭", "190430_03.0023.wav": "이주연 씨가 할미 잘 한다 응 고마워요 그렇게나 칭찬해줘야 해요", "190430_03.0024.wav": "너무 창피하니까", "190430_03.0025.wav": "박민아 씨가 할머니 레게 강 같은 평화 한 번 해 주세요", "190430_03.0026.wav": "4개 하세요 뜸붕어", "190430_03.0027.wav": "이날 둥글고까지", "190430_03.0029.wav": "시키는 거 다해요 여러분 아시죠 제가 여기서 뭐 하러 왔겠어요 듣고 싶으신 거", "190430_03.0030.wav": "들려 드리려 왔습니다", "190430_03.0031.wav": "응", "190430_03.0032.wav": "7078 니 무슨 노래였어요 TWICE  노래가 아니었던 건 분명한데 말이에요", "190430_03.0033.wav": "아 당연하지 무슨 트와이스 노래야 그건 아니었어", "190430_03.0034.wav": "저도 잘 모르는데 뭐 있나 봐 요즘 워낙 아이돌이 많잖아요", "190430_03.0035.wav": "있어 있어", "190430_03.0036.wav": "설마 제가 설마 트와이스 님들께", "190430_03.0037.wav": "아니에요 아니에요", "190430_03.0038.wav": "주진희 씨가 음치래 드립이네 유리 노래 잘 하는데", "190430_03.0039.wav": "묻겠어", "190430_03.0040.wav": "어우", "190430_03.0043.wav": "왜 향니 노래 잘 하시는구만", "190430_03.0044.wav": "근데 애드립을 진짜 1분도 못 맞춰 주시고 계시는데요", "190430_03.0045.wav": "손성미씨가 오픈 스튜디오에 오신 분들이랑 다같이 웃었대요", "190430_03.0046.wav": "난리 났었어요", "190430_03.0048.wav": "하", "190430_03.0049.wav": "와 그런데 너무 행복하다 표정이 진짜 맑아졌어요 여러분", "190430_03.0051.wav": "여러분 들면 그렇게 행복해질 수 있다면 제가", "190430_03.0052.wav": "매일매일", "190430_03.0053.wav": "내일 내일", "190430_03.0054.wav": "내일 내가 그 뭐지", "190430_03.0056.wav": "비 오면", "190430_03.0057.wav": "뭐 할게", "190430_03.0058.wav": "비 오면", "190430_03.0059.wav": "좋은 거 뭐 뭐 할껴", "190430_03.0060.wav": "대신 비가 와야 돼", "190430_03.0061.wav": "데 혹시 그 기능 가지고 계신 분 계시면 내일 비 좀 켜 주세요", "190430_03.0062.wav": "비킬 줄 아는 사람은", "190430_03.0065.wav": "김원빈 씨가 트와이스와 다크 템플러에게 사과는 제가 대신할게요", "190430_03.0066.wav": "괜찮아 유 딩", "190430_03.0067.wav": "고마워요 진짜 여러분 덕분에 제가 여기 서만 이런 것도 해요", "190430_03.0069.wav": "우리나라", "190430_03.0070.wav": "제대로 된 취업 들을까요 사실 이노랜 정말좋은 노래잖아요", "190430_03.0071.wav": "지금 들을 수 있죠 피디님", "190430_03.0072.wav": "좋아요 트와이스의 cheer up 듣고 오겠습니다 죄송해요", "190430_03.0073.wav": "네에", "190430_03.0074.wav": "여러분", "190430_03.0077.wav": "재밌어요 있으니까 진짜 진짜 재밌다", "190430_03.0078.wav": "볼륨을 높여요 함께 하고 계시고요", "190430_03.0079.wav": "8시 45분 55초 지나고 있습니다 저는 여유 있네요 여러분들의 놀이 불륜 가족 분들이시고요", "190430_03.0080.wav": "응 아마 수연 씨도", "190430_03.0081.wav": "잘 계실 거예요 그래두", "190430_03.0082.wav": "나두", "190430_03.0083.wav": "가족이 왔으니까 아마", "190430_03.0084.wav": "아주 편안하게", "190430_03.0085.wav": "자리를 잘 마무리하고 오셨으면 좋겠고요", "190430_03.0086.wav": "음 시나몬 레스님이 언니 사투리 가르쳐 주세요", "190430_03.0087.wav": "사투리 못 해요", "190430_03.0088.wav": "헐 할머니가 하는 사투리잖아", "190430_03.0089.wav": "내가 어떻게 됐지", "190430_03.0090.wav": "오늘 사투리 못하는데", "190430_03.0091.wav": "할머니는 잘 하신단 말이지", "190430_03.0092.wav": "응", "190430_03.0093.wav": "그래요 나중에 알려줄께요 지나버렸으니님", "190430_03.0094.wav": "무슨 수로", "190430_03.0095.wav": "맞나요", "190430_03.0097.wav": "a key7414 님이 중국 대학생이다 사랑해 언니", "190430_03.0100.wav": "나는 한국 배우다 사랑 해나도 하", "190430_03.0101.wav": "나는 볼륨 dj다 사랑해", "190430_03.0102.wav": "응 케이 7300 님이 띄우면 레이니즘 예", "190430_03.0103.wav": "그게 그거죠 스엠아 3이거야", "190430_03.0104.wav": "아 안돼요 그건 못 해 주고요", "190430_03.0105.wav": "비 오면", "190430_03.0106.wav": "요즘 준비해올게요", "190430_03.0107.wav": "좋은 거 들려 드리고 하려구요 그냥", "190430_03.0108.wav": "내가 안 되나", "190430_03.0109.wav": "응", "190430_03.0110.wav": "음", "190430_03.0111.wav": "아 이런 귀여운 문제가 너무 많아요", "190430_03.0112.wav": "해외 주문들이 문자를 많이 주셔서 너무 반가운데요", "190430_03.0113.wav": "소개해 드릴게요", "190430_03.0114.wav": "언니 사장님이에요", "190430_03.0115.wav": "언니 내가 어젯밤에 너를 꿈 꿨는데 나는 너를 만나서 매우 설레고", "190430_03.0116.wav": "우리 함께 얘기하니까 마치 친구인 것 같아", "190430_03.0118.wav": "아니야 아니야 마치 친구인 것 같은 게 아니라 이미 우리는 친구야", "190430_03.0120.wav": "니가 났길래 난데 기분 나쁘면", "190430_03.0121.wav": "저는 존댓말 잘 해요", "190430_03.0122.wav": "오 어제 끝", "190430_03.0123.wav": "저기 어디지 태국에서", "190430_03.0124.wav": "꽃다발을 보내 주신 거예요 근데 그 꽃다발이 아마", "190430_03.0125.wav": "정말 태국에서부터 온 것 같더라고요 표정을 보니까", "190430_03.0126.wav": "그래서 제가", "190430_03.0127.wav": "아니야 pd 님 왜 우세요 아니야", "190430_03.0128.wav": "태국에서부터 온 것 맞춰 꽃다발이", "190430_03.0129.wav": "꽁꽁 숨어서 포장이 아주 아주 꽁꽁 싸여져왔어요 제 손안에 잘 도착했고요", "190430_03.0130.wav": "그래서 제가 그거 이렇게 이렇게 해서 꽃병에다가 이렇게 이렇게 해서 물 이렇게 이렇게", "190430_03.0131.wav": "되게 예쁘게나 생각대로", "190430_03.0132.wav": "사진 찍어놨는데 보여 주고 싶은데 제가", "190430_03.0133.wav": "활동하는 그런 sns 그런 게 없어 가지고", "190430_03.0134.wav": "그래요 어우 보이는 라디 오니까 보여줄까", "190430_03.0135.wav": "근데 나 지금", "190430_03.0136.wav": "노래 들어야 하는데", "190430_03.0137.wav": "그래도 이건 보여줘", "190430_03.0138.wav": "음 제가 꽃 빨간 꽃은 실력 보여", "190430_03.0139.wav": "바다 여러분", "190430_03.0140.wav": "와", "190430_03.0141.wav": "이렇게", "190430_03.0142.wav": "잘 안 보이는구나", "190430_03.0143.wav": "이게 이렇게", "190430_03.0144.wav": "이렇게 꼬치", "190430_03.0145.wav": "어 이렇게"}
tmp_dict = {}
for key,val in inna_json.items():
    tmp_dict[f'/content/gdrive/My Drive/stt플젝/datasets/inna/audio/{key}'] = val

with open('inna.json','w',encoding='utf-8') as f:
    json.dump(tmp_dict, f, ensure_ascii=False)

## KSS Dataset json화

In [ ]:
import pandas as pd
data = pd.read_csv('transcript.v.1.4.txt',sep='|',names=['audio','text','text1','text2','num','eng'])
audio_list = data['audio']
text_list = data['text']
tmp = {}
for i in range(len(data)):
    tmp[f'/content/drive/My Drive/stt플젝/datasets/kss/{audio_list[i]}'] = f'{text_list[i]}'
    with open('kss_원래.json','w',encoding='utf-8') as f:
        json.dump(tmp, f, ensure_ascii=False)

## Google API key

In [ ]:
!pip install --upgrade google-cloud-speech

Requirement already up-to-date: google-cloud-speech in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/gdrive/My Drive/stt플젝/sttapi/stt-project-292919-a02efdc10005.json'

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/gdrive/My Drive/stt플젝/sttapi/stt-project-292919-a02efdc10005.json


In [ ]:
!gcloud auth application-default login


The environment variable [GOOGLE_APPLICATION_CREDENTIALS] is set to:
  [/content/gdrive/My 
Drive/stt플젝/sttapi/stt-project-292919-a02efdc10005.json]
Credentials will still be generated to the default location:
  [/content/.config/application_default_credentials.json]
To use these credentials, unset this environment variable before
running your application.

Do you want to continue (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=yPcA9xHAS_4tpEZMwqKkTtf0t6YGwnEFrR5oWDRq6n0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/1AfDhmrjbrp8n34x5i

In [ ]:
#!gcloud beta auth application-default set-quota-project

In [ ]:
#!gcloud auth application-default set-quota-project

In [ ]:
#!gcloud init

In [ ]:
#!gcloud auth application-default set-quota-project --help